## QUESTION 2

A heatmap painted over the map of Gracia district was considered the most visual and human understandable source of information to made explicit the quality of bike stations across the district. A first approach would have led to produce some histograms or density plots depicting the distribution of available bikes for each station. But, heatmaps, besides giving a quick first glance of quality and depletion, can introduce space and inter-station neighbourhood information. If I stick only to depletion metrics concerning each station I get an insolated perspective. I could get a biased decision because a good quality station could be surrounded by bad bike stations. So, my B-plan could be catasthopic in case one morning I find my station depleted of bikes, because my nearby surrounding station most probably will also be depleted. On the other hand, a heatmap will expose the quality of each station plus its nearby surrounding stations. So a good decission is based on chosing a good station surrounded by other good/mid quality station which work as a good alternative plans. 



# Strategy

The script will access to a bunch of data which was sampled along 24h. The samples are spaced by 5 minutes intervals. The script is not focusing on counting mean of free bikes but counting amount of segments where the station had zero bikes to offer. The database is a list of snapshots consisting of a timestamp, the list of Gracia district stations and another list with the neighbour stations whether they belong to Gracia or not. For each snapshot a table will sum the zerosegments (no bikes during 5 minutes). Such amount will induce a "hot spot" which will contribute to the heat of each station. The spots are spread randomly uniform around each station. The more zerosegments a station has, the more heat is amassed around the station.  

# Heatmap Interpretation

Stations with high amount of zerosegments will sum up high amounts of heat. The good choice is to settle nearby cold stations, blueish or greenish. Very high quality stations will be extremely cold and will almost paint no heat trace on the map. This will make seem similar two kind of streets. First kind are streets very near to top quality stations; and second kind of streets which are far away of any station. The second type is indeed a not good one. The user would live away from any station giving him a bad commute time. Such issue motivates to introduce "inverse heat". 

# Inverse Heat

Heatmaps depict and stress interesting entities giving a positive incremental qualitative measure. Stressing bad quality stations with a heatmap gives us an uncomplete picture of the setting. You can see above it is ambiguous about marking streets nearby high quality stations from isolated streets away of any station. The proposal is to give heat to high quality stations. The measure of amount of zeroSegments is updated to a complementary value. High quality stations will be enclosed in a cloud of heat, and thereby all the interesting streets around it will also be stressed by the heat cloud. 

# Inverse Heatmap Interpretation

The new metric will show with equal cold temperature streets near a bad station as streets far away from any station.
The user will guide its decission on highest temperature areas. It should be advisable to land on some mid point between high quality station, but not too far away from a prefered one.



In [ ]:
import requests
import urllib, json

import datetime

import pymongo as mongo
from pymongo import MongoClient

import folium
from folium import plugins
from folium.plugins import HeatMap
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import matplotlib.pyplot as plt
import random as rnd

import bson

__author__ = "Alexis Torrano"
__email__ = "a.torrano.m@gmail.com"
__status__ = "Production"

%matplotlib inline

In [ ]:
def countZeroBikeStations(snapshot,tablCounter):
    
    GraciaStations = snapshot['gracia']
    neighbourStations = snapshot['neighbours']    
    
    # Increase the counter of each station when bikes are depleted.
    # If station is stil not registered in the counters list, it
    # sets a counter to 0 for the station.
    for x in GraciaStations:        
        if not x['id'] in tablCounter:
            tablCounter[x['id']] = 0
                
        if 0 == x['free_bikes']:            
            tablCounter[x['id']] += 1
        

    for x in neighbourStations:        
        if not x['id'] in tablCounter:
            tablCounter[x['id']] = 0
                
        if 0 == x['free_bikes']:            
            tablCounter[x['id']] += 1

In [ ]:
def getStaticFeaturesOfStations(snapshot,volatileFeatures):
    # returns tables with the static part of the data model associated to stations    
    
    GraciaStations = snapshot['gracia']
    neighbourStations = snapshot['neighbours']       
    
    pandasGraciaStations = pd.DataFrame.from_dict(GraciaStations)
    pandasNeighbourStations = pd.DataFrame.from_dict(neighbourStations)
        
    pandasGraciaStations.drop(volatileFeatures,axis=1)
    pandasNeighbourStations.drop(volatileFeatures,axis=1)
    
    return pandasGraciaStations,pandasNeighbourStations

In [ ]:
def getHeatSpots(pandasDF,counterDF,stats,m,inverseHeat,heatSpotsList):
    
    '''
    For each station <getHeatSpots> produces a list of spots for a heatmap. 
    Each spot originally represents a segment of time where the station had 0 free_bikes.
    The more time a station has been depleted the hotter it should appear in
    the heatmap.
    
    There is an option for the inverseHeat version of the heatmap. Then, <heat>
    measures availability of bikes in the station.
    
    The amount of time segments of depletion induces the color of the station marker
    in the city map. The color assignation follows the quantiles in the distribution
    of depleted time segments. And such assignation is inmovable whether <inverseHeat>
    is chosen or not.
    '''
    
    for index, row in pandasDF.iterrows():
        heat = counterDF[row.id]
        
        # Basic semantics of <heat> is applied in marker color assignation 
        if heat <= stats['min']:
            st_marker_color='blue'
        elif heat < stats['25%']:
            st_marker_color='green'            
        elif heat < stats['75%']:
            st_marker_color='orange'            
        elif heat < stats['max']:
            st_marker_color='red'
        elif heat == stats['max']:
            st_marker_color='black'
                     
        # Once color marker is assigned, <inverseHeat> may be activated
        if inverseHeat:
            # The parameters for <inverseHeat> were found uppon manual factor exploration
            # based on visual identification for better distinguishable separation area.
            heat = stats['max'] - heat            
            ratioHeat = float(heat) / float(stats['max'])            
            stdev = 2.0 * ratioHeat * ratioHeat * ratioHeat
            scale = 1000.0            
        else:
            ratioHeat = float(heat) / float(stats['max'])
            stdev = 1.0
            scale = 1000.0

        
        folium.CircleMarker([row['latitude'], row['longitude']],
                            radius=15,
                            popup=row['name'],
                            fill_color="#3db7e4", # divvy color
                           ).add_to(m)
        
        folium.Marker([row['latitude'], row['longitude']],                        
                            popup=str(row['name']+"::"+str(heat)),
                            icon=folium.Icon(color=st_marker_color)
                           ).add_to(m)
         
         
        # produce a list of coordinates for each bike in order to feed the heatmap        
        for s in range(int(100.0*ratioHeat)):        
            disturbLat = ((-stdev+2.0*stdev*rnd.random())/scale)
            disturbLon = ((-stdev+2.0*stdev*rnd.random())/scale)
            heatSpotsList.append([row['latitude']+disturbLat, row['longitude']+disturbLon])
        
        '''
        For each station, each 5' segment with 0 bikes will entail an occurrency 
        in the heatmap giving a random perturbation to original station coordinates.
        '''       

In [ ]:
def heatmap(pandasGraciaStations,neighboursDF,counterDF,inverseHeat=False):
    ## Must paint in map all x in list <GraciaStations> and all y in y.extra.NearbyStationList
    
    '''
    I got Gracia district coordinates from openstreetmap: 
    https://nominatim.openstreetmap.org/details.php?place_id=198819829
    https://www.openstreetmap.org/relation/3773080#map=14/41.4102/2.1599
    
    map center: 41.41023,2.15087 view on osm.org
    map zoom: 14
    viewbox: 2.08989,41.42632,2.21177,41.39413
    '''    
    
    m = folium.Map([41.41023, 2.15087], zoom_start=14)
      
    # mark each station as a point: put a marker and a pop-up with station name
    # todo : add free_bikes at the pop-up
    zeroBikesList = []    
    #count,mean,std,mini,pct25,pct50,pct75,maxi=counterDF.describe()
    stats=counterDF.describe()
    
    # Get coordinates for occurences to add to heatmap and at markers in map for each Bicing Station
    getHeatSpots(pandasGraciaStations,counterDF,stats,m,inverseHeat,zeroBikesList)    
    getHeatSpots(neighboursDF,counterDF,stats,m,inverseHeat,zeroBikesList)        
    
    ## HEATMAP CALL
    if inverseHeat:
        min_opacity=0.1
        max_val=0.8
    else:
        min_opacity=0.5
        max_val=1.0
    
    
    m.add_child(plugins.HeatMap(zeroBikesList, radius=50,blur=70,min_opacity=min_opacity,max_val=max_val))    
    
    return(m)
    
    #TODO Heatmap radius : add a slider to Jupyter

In [ ]:
## Please remember advice from question_0, the leeching process.
## Jupyter notebook will generate results not from a mongoDB query but from 
## a query to mongoDB database dump file.
## All code line preceded by "### MONGO ###" was used in case of direct mongoDB interaction.

### MONGO ### mongoC = MongoClient('mongodb://localhost:27017/')
### MONGO ### dbHosco = mongoC['HOSCO']
### MONGO ### timeBikeAllocation = dbHosco['timeBikeAllocation']

bsonfilePath = 'data/HOSCO/'
bsonfileName = 'timeBikeAllocation.bson'

try:
    with open(bsonfilePath+bsonfileName,'rb') as f:
        timeBikeAllocation_list = bson.decode_all(f.read())
except Exception as e:
    print(str(e))   


# Get the first snapshot to build the dataframe with station's 
# static features.
### MONGO ### snapshot = timeBikeAllocation.find_one()
snapshot = timeBikeAllocation_list[0]

volatileFeatures = ['free_bikes', 'empty_slots', 'timestamp']
pandasGraciaStations,pandasNeighbourStations = getStaticFeaturesOfStations(snapshot,volatileFeatures)
# remove from pandasNeighbourStations any station contained at pandasGraciaStations 
pandasNeighbourStations = pandasNeighbourStations[~ pandasNeighbourStations.id.isin(pandasGraciaStations.id)]
    
tablCounter = {}
### MONGO ### cursor = timeBikeAllocation.find({})
timeSliceCount = 0
### MONGO ### for snapshot in cursor:
for snapshot in timeBikeAllocation_list:
    countZeroBikeStations(snapshot,tablCounter)
    timeSliceCount += 1

## Prepare data for graphical report;
## paint heatmap from tablCounter

pandasTablCounter = pd.Series(tablCounter, name='countTimeSliceZeroBikes')
pandasTablCounter.index.name = 'id'
pandasTablCounter.reset_index()

m = heatmap(pandasGraciaStations,pandasNeighbourStations,pandasTablCounter)      



## First Heatmap.





Each station locus is tagged with a mark in the map. The quality/utility of each station is induced by the probability of depletion. Depletion is estimated from the amount of 5 minute time segments where the station has no bikes available. Based on ratio, such amount, belonging to the distribution of all “zero segments” of all stations, will receive an assigned color. See, please, the adjoining figure to discover the correspondence of color and ratio bin where the station is assigned. Such code color will mark the better station to select a good nearby house. A station with a ratio of 0% zero segments is ideal.

<html>
<head>
	<meta http-equiv="content-type" content="text/html; charset=utf-8"/>
	<title></title>
	<meta name="generator" content="LibreOffice 5.1.6.2 (Linux)"/>
	<meta name="created" content="00:00:00"/>
	<meta name="changed" content="2018-11-22T22:34:19.945470688"/>
	
</head>
<body lang="en-US" dir="ltr">
<table width="290" cellpadding="4" cellspacing="0" style="page-break-before: always">
	<col width="283">	
	<tr valign="top">
		<td width="183" style="border-top: 1px solid #000000; border-bottom: 1px solid #000000; border-left: 1px solid #000000; border-right: none; padding-top: 0.04in; padding-bottom: 0.04in; padding-left: 0.04in; padding-right: 0in">
			<p> <img src="img/legend.png" alt="legend" width="70%"> </p>
		</td>		
	</tr>
</table>
<p style="margin-bottom: 0in; line-height: 100%"><br/>

</p>
</body>
</html>

In [ ]:
m

In [ ]:
m = heatmap(pandasGraciaStations,pandasNeighbourStations,pandasTablCounter,inverseHeat=True)      

## Heatmap of inverse heat

In [ ]:
m

<!DOCTYPE html>
<head>    
    <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
    <script>L_PREFER_CANVAS=false; L_NO_TOUCH=false; L_DISABLE_3D=false;</script>
    <script src="https://cdn.jsdelivr.net/npm/leaflet@1.2.0/dist/leaflet.js"></script>
    <script src="https://ajax.googleapis.com/ajax/libs/jquery/1.11.1/jquery.min.js"></script>
    <script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/js/bootstrap.min.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js"></script>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet@1.2.0/dist/leaflet.css"/>
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap.min.css"/>
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap-theme.min.css"/>
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.6.3/css/font-awesome.min.css"/>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css"/>
    <link rel="stylesheet" href="https://rawgit.com/python-visualization/folium/master/folium/templates/leaflet.awesome.rotate.css"/>
    <style>html, body {width: 100%;height: 100%;margin: 0;padding: 0;}</style>
    <style>#map {position:absolute;top:0;bottom:0;right:0;left:0;}</style>
    
    <style>#map_e9a0ac71acee47749a2d0b689bffffa3 {
        position: relative;
        width: 100.0%;
        height: 100.0%;
        left: 0.0%;
        top: 0.0%;
        }
    </style>
    <script src="https://leaflet.github.io/Leaflet.heat/dist/leaflet-heat.js"></script>
</head>
<body>    
    
    <div class="folium-map" id="map_e9a0ac71acee47749a2d0b689bffffa3" ></div>
</body>
<script>    
    
    
        var bounds = null;
    

    var map_e9a0ac71acee47749a2d0b689bffffa3 = L.map(
        'map_e9a0ac71acee47749a2d0b689bffffa3', {
        center: [41.41023, 2.15087],
        zoom: 14,
        maxBounds: bounds,
        layers: [],
        worldCopyJump: false,
        crs: L.CRS.EPSG3857,
        zoomControl: true,
        });

    
    
    var tile_layer_c04ed4ef95b249c58b259d0cd72268ca = L.tileLayer(
        'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png',
        {
        "attribution": null,
        "detectRetina": false,
        "maxNativeZoom": 18,
        "maxZoom": 18,
        "minZoom": 0,
        "noWrap": false,
        "subdomains": "abc"
}).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
    
            var circle_marker_15a058ce23274d019bb69b6807fd4302 = L.circleMarker(
                [41.402535, 2.152519],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_6248037211bc4bec907bf599c10b3e6b = L.popup({maxWidth: '300'
            
            });

            
                var html_ce3c263f66ed4467abab88b6a048cc24 = $('<div id="html_ce3c263f66ed4467abab88b6a048cc24" style="width: 100.0%; height: 100.0%;">221 - GRAN DE GRÀCIA, 155 (METRO FONTANA)</div>')[0];
                popup_6248037211bc4bec907bf599c10b3e6b.setContent(html_ce3c263f66ed4467abab88b6a048cc24);
            

            circle_marker_15a058ce23274d019bb69b6807fd4302.bindPopup(popup_6248037211bc4bec907bf599c10b3e6b)
            ;

            
        
    
        var marker_db8584da64f34f9c874a87deec0d3abe = L.marker(
            [41.402535, 2.152519],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_f0651221fe664695a4f6b0e1af7a31ef = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'green',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_db8584da64f34f9c874a87deec0d3abe.setIcon(icon_f0651221fe664695a4f6b0e1af7a31ef);
            
    
            var popup_671d3095a25947e89788af04350e413e = L.popup({maxWidth: '300'
            
            });

            
                var html_15880d0d68874a36b597fa4e28a0ca3a = $('<div id="html_15880d0d68874a36b597fa4e28a0ca3a" style="width: 100.0%; height: 100.0%;">221 - GRAN DE GRÀCIA, 155 (METRO FONTANA)::250.0</div>')[0];
                popup_671d3095a25947e89788af04350e413e.setContent(html_15880d0d68874a36b597fa4e28a0ca3a);
            

            marker_db8584da64f34f9c874a87deec0d3abe.bindPopup(popup_671d3095a25947e89788af04350e413e)
            ;

            
        
    
            var circle_marker_741f1607a7a74f9f849b7d80cf65ce50 = L.circleMarker(
                [41.40124, 2.157483],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_ad1d6418c8f14393877027ff792b7e5c = L.popup({maxWidth: '300'
            
            });

            
                var html_95427642acbd4710baf757240e69d49d = $('<div id="html_95427642acbd4710baf757240e69d49d" style="width: 100.0%; height: 100.0%;">222 - C/ DEL CANÓ, 1</div>')[0];
                popup_ad1d6418c8f14393877027ff792b7e5c.setContent(html_95427642acbd4710baf757240e69d49d);
            

            circle_marker_741f1607a7a74f9f849b7d80cf65ce50.bindPopup(popup_ad1d6418c8f14393877027ff792b7e5c)
            ;

            
        
    
        var marker_56fe443fcdbe4cd99ab60595b8e8f4bc = L.marker(
            [41.40124, 2.157483],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_bdd04fe069e444abb690d48d5c36b5b3 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'black',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_56fe443fcdbe4cd99ab60595b8e8f4bc.setIcon(icon_bdd04fe069e444abb690d48d5c36b5b3);
            
    
            var popup_65959a05371249f09620e7290b03db13 = L.popup({maxWidth: '300'
            
            });

            
                var html_cccc49b7bf5e4be894b396516cd602f7 = $('<div id="html_cccc49b7bf5e4be894b396516cd602f7" style="width: 100.0%; height: 100.0%;">222 - C/ DEL CANÓ, 1::0.0</div>')[0];
                popup_65959a05371249f09620e7290b03db13.setContent(html_cccc49b7bf5e4be894b396516cd602f7);
            

            marker_56fe443fcdbe4cd99ab60595b8e8f4bc.bindPopup(popup_65959a05371249f09620e7290b03db13)
            ;

            
        
    
            var circle_marker_ae3976c512b047c09d6f2e169550901a = L.circleMarker(
                [41.398311, 2.159865],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_b6b3ef290e644eb488940ac9c34fa9ed = L.popup({maxWidth: '300'
            
            });

            
                var html_c566c2485c2a4e3798c309647df6f4e6 = $('<div id="html_c566c2485c2a4e3798c309647df6f4e6" style="width: 100.0%; height: 100.0%;">223 - C/ DE BONAVISTA, 14</div>')[0];
                popup_b6b3ef290e644eb488940ac9c34fa9ed.setContent(html_c566c2485c2a4e3798c309647df6f4e6);
            

            circle_marker_ae3976c512b047c09d6f2e169550901a.bindPopup(popup_b6b3ef290e644eb488940ac9c34fa9ed)
            ;

            
        
    
        var marker_924d0863e19f477886b97730cd528eb7 = L.marker(
            [41.398311, 2.159865],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_9a91397e5a4549ec92db25324e381f34 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'orange',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_924d0863e19f477886b97730cd528eb7.setIcon(icon_9a91397e5a4549ec92db25324e381f34);
            
    
            var popup_cf66fca2849940b1a2b40a800e8dbf70 = L.popup({maxWidth: '300'
            
            });

            
                var html_c6f842e2c9ba40f294955fb3503b6438 = $('<div id="html_c6f842e2c9ba40f294955fb3503b6438" style="width: 100.0%; height: 100.0%;">223 - C/ DE BONAVISTA, 14::197.0</div>')[0];
                popup_cf66fca2849940b1a2b40a800e8dbf70.setContent(html_c6f842e2c9ba40f294955fb3503b6438);
            

            marker_924d0863e19f477886b97730cd528eb7.bindPopup(popup_cf66fca2849940b1a2b40a800e8dbf70)
            ;

            
        
    
            var circle_marker_a7d4916bd0114decb99aa560481f4acb = L.circleMarker(
                [41.403465, 2.161096],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_1a2c34809517431abee77b1f69685706 = L.popup({maxWidth: '300'
            
            });

            
                var html_8013b0ec05504f72acbbe3e5df959a3a = $('<div id="html_8013b0ec05504f72acbbe3e5df959a3a" style="width: 100.0%; height: 100.0%;">226 - C/MONTMANY, 1</div>')[0];
                popup_1a2c34809517431abee77b1f69685706.setContent(html_8013b0ec05504f72acbbe3e5df959a3a);
            

            circle_marker_a7d4916bd0114decb99aa560481f4acb.bindPopup(popup_1a2c34809517431abee77b1f69685706)
            ;

            
        
    
        var marker_ca5c8d26d7914ca590c8942943ad4216 = L.marker(
            [41.403465, 2.161096],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_4ec295587b534fc6a3f56da58d4ac91e = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'orange',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_ca5c8d26d7914ca590c8942943ad4216.setIcon(icon_4ec295587b534fc6a3f56da58d4ac91e);
            
    
            var popup_492f03f75c4a43f6aca17f7dee899934 = L.popup({maxWidth: '300'
            
            });

            
                var html_c0bd7314c63b4bf9b4590331100d4b0a = $('<div id="html_c0bd7314c63b4bf9b4590331100d4b0a" style="width: 100.0%; height: 100.0%;">226 - C/MONTMANY, 1::200.0</div>')[0];
                popup_492f03f75c4a43f6aca17f7dee899934.setContent(html_c0bd7314c63b4bf9b4590331100d4b0a);
            

            marker_ca5c8d26d7914ca590c8942943ad4216.bindPopup(popup_492f03f75c4a43f6aca17f7dee899934)
            ;

            
        
    
            var circle_marker_a2d18d8f62b7433a8d04e6c081eee1a3 = L.circleMarker(
                [41.405986, 2.151633],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_da360bf86e9d4e61b2adb431cb47dd20 = L.popup({maxWidth: '300'
            
            });

            
                var html_1cc4ea8b9b754b33a6f6937be74a0c2e = $('<div id="html_1cc4ea8b9b754b33a6f6937be74a0c2e" style="width: 100.0%; height: 100.0%;">230 - C/ DE NIL FABRA, 16-20</div>')[0];
                popup_da360bf86e9d4e61b2adb431cb47dd20.setContent(html_1cc4ea8b9b754b33a6f6937be74a0c2e);
            

            circle_marker_a2d18d8f62b7433a8d04e6c081eee1a3.bindPopup(popup_da360bf86e9d4e61b2adb431cb47dd20)
            ;

            
        
    
        var marker_08e6128d2ead4753b470f1dac4208a5b = L.marker(
            [41.405986, 2.151633],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_eda419ded3174081958fe5717affd4cc = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'orange',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_08e6128d2ead4753b470f1dac4208a5b.setIcon(icon_eda419ded3174081958fe5717affd4cc);
            
    
            var popup_f2e40bdfeb2642dc8bff2fa39833aa60 = L.popup({maxWidth: '300'
            
            });

            
                var html_0fef82b9341b4cf9bc9a931a92befef3 = $('<div id="html_0fef82b9341b4cf9bc9a931a92befef3" style="width: 100.0%; height: 100.0%;">230 - C/ DE NIL FABRA, 16-20::232.0</div>')[0];
                popup_f2e40bdfeb2642dc8bff2fa39833aa60.setContent(html_0fef82b9341b4cf9bc9a931a92befef3);
            

            marker_08e6128d2ead4753b470f1dac4208a5b.bindPopup(popup_f2e40bdfeb2642dc8bff2fa39833aa60)
            ;

            
        
    
            var circle_marker_1935b342cb3b42459c8dfde8e823743c = L.circleMarker(
                [41.38551, 2.152],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_24832a2be44f49ae8ae604ae14dc63e0 = L.popup({maxWidth: '300'
            
            });

            
                var html_cd3430c0aeb04883b182657b1ab918f6 = $('<div id="html_cd3430c0aeb04883b182657b1ab918f6" style="width: 100.0%; height: 100.0%;">405 - C/ COMTE BORRELL, 198</div>')[0];
                popup_24832a2be44f49ae8ae604ae14dc63e0.setContent(html_cd3430c0aeb04883b182657b1ab918f6);
            

            circle_marker_1935b342cb3b42459c8dfde8e823743c.bindPopup(popup_24832a2be44f49ae8ae604ae14dc63e0)
            ;

            
        
    
        var marker_3d9c81720e064f24b2cd2c723226bb07 = L.marker(
            [41.38551, 2.152],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_0124b950f540464193c9244081a6b2e4 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'orange',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_3d9c81720e064f24b2cd2c723226bb07.setIcon(icon_0124b950f540464193c9244081a6b2e4);
            
    
            var popup_ec849234a52c4d8e809bde159f776874 = L.popup({maxWidth: '300'
            
            });

            
                var html_f7e1482bda7f417d8406c1bd399d5dc8 = $('<div id="html_f7e1482bda7f417d8406c1bd399d5dc8" style="width: 100.0%; height: 100.0%;">405 - C/ COMTE BORRELL, 198::228.0</div>')[0];
                popup_ec849234a52c4d8e809bde159f776874.setContent(html_f7e1482bda7f417d8406c1bd399d5dc8);
            

            marker_3d9c81720e064f24b2cd2c723226bb07.bindPopup(popup_ec849234a52c4d8e809bde159f776874)
            ;

            
        
    
            var circle_marker_17cc42dfc33e4de8bd1dd8a98001d348 = L.circleMarker(
                [41.407353, 2.151341],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_887a2b26df094bcd955032e64b4f07a1 = L.popup({maxWidth: '300'
            
            });

            
                var html_6509548179a34e36bcd7db7645159f5e = $('<div id="html_6509548179a34e36bcd7db7645159f5e" style="width: 100.0%; height: 100.0%;">472 - (PK) C/ DEL TORRENT DE L´OLLA, 221</div>')[0];
                popup_887a2b26df094bcd955032e64b4f07a1.setContent(html_6509548179a34e36bcd7db7645159f5e);
            

            circle_marker_17cc42dfc33e4de8bd1dd8a98001d348.bindPopup(popup_887a2b26df094bcd955032e64b4f07a1)
            ;

            
        
    
        var marker_623edfa6eb2d41d09976f9c6aea4d100 = L.marker(
            [41.407353, 2.151341],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_0fbaef28e85c4cc1ad64b57ae5c92938 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'blue',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_623edfa6eb2d41d09976f9c6aea4d100.setIcon(icon_0fbaef28e85c4cc1ad64b57ae5c92938);
            
    
            var popup_40f37e4289f64cde8508cb41c3926e89 = L.popup({maxWidth: '300'
            
            });

            
                var html_40e25089959c49d6bbb69168738b8505 = $('<div id="html_40e25089959c49d6bbb69168738b8505" style="width: 100.0%; height: 100.0%;">472 - (PK) C/ DEL TORRENT DE L´OLLA, 221::266.0</div>')[0];
                popup_40f37e4289f64cde8508cb41c3926e89.setContent(html_40e25089959c49d6bbb69168738b8505);
            

            marker_623edfa6eb2d41d09976f9c6aea4d100.bindPopup(popup_40f37e4289f64cde8508cb41c3926e89)
            ;

            
        
    
            var circle_marker_a8c39ff73833459abb1669e5b5b2b727 = L.circleMarker(
                [41.401752, 2.160748],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_b7ecef024b914d0e91bc8e060eea384d = L.popup({maxWidth: '300'
            
            });

            
                var html_211b90a6c8ac4f7ca7f5dd18df3fbf1b = $('<div id="html_211b90a6c8ac4f7ca7f5dd18df3fbf1b" style="width: 100.0%; height: 100.0%;">483 - (PK) C/ DE SIRACUSA, 39-51</div>')[0];
                popup_b7ecef024b914d0e91bc8e060eea384d.setContent(html_211b90a6c8ac4f7ca7f5dd18df3fbf1b);
            

            circle_marker_a8c39ff73833459abb1669e5b5b2b727.bindPopup(popup_b7ecef024b914d0e91bc8e060eea384d)
            ;

            
        
    
        var marker_8f9d47b6b2dc4a0cae17fd19d766ad33 = L.marker(
            [41.401752, 2.160748],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_8120fe7934e94f67a9a03f8ebdf320de = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'blue',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_8f9d47b6b2dc4a0cae17fd19d766ad33.setIcon(icon_8120fe7934e94f67a9a03f8ebdf320de);
            
    
            var popup_1c8e84a02ec94008ae62ba639913decc = L.popup({maxWidth: '300'
            
            });

            
                var html_3112b960b37343a18691ee0411f0da87 = $('<div id="html_3112b960b37343a18691ee0411f0da87" style="width: 100.0%; height: 100.0%;">483 - (PK) C/ DE SIRACUSA, 39-51::266.0</div>')[0];
                popup_1c8e84a02ec94008ae62ba639913decc.setContent(html_3112b960b37343a18691ee0411f0da87);
            

            marker_8f9d47b6b2dc4a0cae17fd19d766ad33.bindPopup(popup_1c8e84a02ec94008ae62ba639913decc)
            ;

            
        
    
            var circle_marker_e7ea295c38144818bec8e5875a414f1f = L.circleMarker(
                [41.384956, 2.150814],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_3a304ea67e474e14a5a3ac910d1193e6 = L.popup({maxWidth: '300'
            
            });

            
                var html_488250e842f848f4b944b2a019bfd3ea = $('<div id="html_488250e842f848f4b944b2a019bfd3ea" style="width: 100.0%; height: 100.0%;">225 - C/MALLORCA, 84</div>')[0];
                popup_3a304ea67e474e14a5a3ac910d1193e6.setContent(html_488250e842f848f4b944b2a019bfd3ea);
            

            circle_marker_e7ea295c38144818bec8e5875a414f1f.bindPopup(popup_3a304ea67e474e14a5a3ac910d1193e6)
            ;

            
        
    
        var marker_bc8ab54f8cba4fa582cf87b8b801c692 = L.marker(
            [41.384956, 2.150814],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_697bd985cc184436bcbeb4b9d0256cec = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'orange',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_bc8ab54f8cba4fa582cf87b8b801c692.setIcon(icon_697bd985cc184436bcbeb4b9d0256cec);
            
    
            var popup_ae49ab20749b4fe1a0b713e13130d18c = L.popup({maxWidth: '300'
            
            });

            
                var html_907951bccfbf433fb016e23d67f7e7e3 = $('<div id="html_907951bccfbf433fb016e23d67f7e7e3" style="width: 100.0%; height: 100.0%;">225 - C/MALLORCA, 84::222.0</div>')[0];
                popup_ae49ab20749b4fe1a0b713e13130d18c.setContent(html_907951bccfbf433fb016e23d67f7e7e3);
            

            marker_bc8ab54f8cba4fa582cf87b8b801c692.bindPopup(popup_ae49ab20749b4fe1a0b713e13130d18c)
            ;

            
        
    
            var circle_marker_aa7d8c5c40ae4ddba4440f32a5979d2c = L.circleMarker(
                [41.395364, 2.157356],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_de59bc3e803245208645fccd1a43bdd6 = L.popup({maxWidth: '300'
            
            });

            
                var html_4123d18ff0874fabadf0e2f8c804c88f = $('<div id="html_4123d18ff0874fabadf0e2f8c804c88f" style="width: 100.0%; height: 100.0%;">68 - RAMBLA CATALUNYA, 133 /  CÒRSEGA</div>')[0];
                popup_de59bc3e803245208645fccd1a43bdd6.setContent(html_4123d18ff0874fabadf0e2f8c804c88f);
            

            circle_marker_aa7d8c5c40ae4ddba4440f32a5979d2c.bindPopup(popup_de59bc3e803245208645fccd1a43bdd6)
            ;

            
        
    
        var marker_fe974fbdbff9406f90244da7fdb4ff6d = L.marker(
            [41.395364, 2.157356],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_266c8e43cb7d4d2c9f65e411e267e02c = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'red',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_fe974fbdbff9406f90244da7fdb4ff6d.setIcon(icon_266c8e43cb7d4d2c9f65e411e267e02c);
            
    
            var popup_c87c1a814b504d06adb0a89815ccfb8c = L.popup({maxWidth: '300'
            
            });

            
                var html_522347c43e6a4f58ba8e61e627f3a0e7 = $('<div id="html_522347c43e6a4f58ba8e61e627f3a0e7" style="width: 100.0%; height: 100.0%;">68 - RAMBLA CATALUNYA, 133 /  CÒRSEGA::182.0</div>')[0];
                popup_c87c1a814b504d06adb0a89815ccfb8c.setContent(html_522347c43e6a4f58ba8e61e627f3a0e7);
            

            marker_fe974fbdbff9406f90244da7fdb4ff6d.bindPopup(popup_c87c1a814b504d06adb0a89815ccfb8c)
            ;

            
        
    
            var circle_marker_bed09b45af494ec6bff37025424ace34 = L.circleMarker(
                [41.384075, 2.153654],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_320c5ccb39ac4f579a64f45dad56f585 = L.popup({maxWidth: '300'
            
            });

            
                var html_9b676f2bec5f4ac09646b2a1e62b91ca = $('<div id="html_9b676f2bec5f4ac09646b2a1e62b91ca" style="width: 100.0%; height: 100.0%;">83 - C/ COMTE BORRELL 177</div>')[0];
                popup_320c5ccb39ac4f579a64f45dad56f585.setContent(html_9b676f2bec5f4ac09646b2a1e62b91ca);
            

            circle_marker_bed09b45af494ec6bff37025424ace34.bindPopup(popup_320c5ccb39ac4f579a64f45dad56f585)
            ;

            
        
    
        var marker_19221659c3454ef6ad4b4b51abb7b7c9 = L.marker(
            [41.384075, 2.153654],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_fa458165727e47468f3306bc767edf85 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'orange',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_19221659c3454ef6ad4b4b51abb7b7c9.setIcon(icon_fa458165727e47468f3306bc767edf85);
            
    
            var popup_098b929284ba4b74a2bb54e2db3134e4 = L.popup({maxWidth: '300'
            
            });

            
                var html_5a15b3a8c83d4c28885a08e729e1b768 = $('<div id="html_5a15b3a8c83d4c28885a08e729e1b768" style="width: 100.0%; height: 100.0%;">83 - C/ COMTE BORRELL 177::220.0</div>')[0];
                popup_098b929284ba4b74a2bb54e2db3134e4.setContent(html_5a15b3a8c83d4c28885a08e729e1b768);
            

            marker_19221659c3454ef6ad4b4b51abb7b7c9.bindPopup(popup_098b929284ba4b74a2bb54e2db3134e4)
            ;

            
        
    
            var circle_marker_bd9bf998ebf84e84921f3f672f8c58dc = L.circleMarker(
                [41.383156, 2.148506],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_10115bea6a6c4abd8c0f9a54fdea6e52 = L.popup({maxWidth: '300'
            
            });

            
                var html_2445034e4cb74935abfd2433048014d9 = $('<div id="html_2445034e4cb74935abfd2433048014d9" style="width: 100.0%; height: 100.0%;">87 - C/ MALLORCA 41-43</div>')[0];
                popup_10115bea6a6c4abd8c0f9a54fdea6e52.setContent(html_2445034e4cb74935abfd2433048014d9);
            

            circle_marker_bd9bf998ebf84e84921f3f672f8c58dc.bindPopup(popup_10115bea6a6c4abd8c0f9a54fdea6e52)
            ;

            
        
    
        var marker_3382f9fbfbf64db493eeb2558756bbc8 = L.marker(
            [41.383156, 2.148506],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_e421c9919d7e42e28159e3bf2e3cb926 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'orange',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_3382f9fbfbf64db493eeb2558756bbc8.setIcon(icon_e421c9919d7e42e28159e3bf2e3cb926);
            
    
            var popup_b1e760ec404a4b9c9ac60c70fdc7f4df = L.popup({maxWidth: '300'
            
            });

            
                var html_63190c78cfe14401a0d78a6e8d757fda = $('<div id="html_63190c78cfe14401a0d78a6e8d757fda" style="width: 100.0%; height: 100.0%;">87 - C/ MALLORCA 41-43::219.0</div>')[0];
                popup_b1e760ec404a4b9c9ac60c70fdc7f4df.setContent(html_63190c78cfe14401a0d78a6e8d757fda);
            

            marker_3382f9fbfbf64db493eeb2558756bbc8.bindPopup(popup_b1e760ec404a4b9c9ac60c70fdc7f4df)
            ;

            
        
    
            var circle_marker_efdd786a997d4b8dafd3b8b261dcfa0b = L.circleMarker(
                [41.387953, 2.150169],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_0f377ef04cec43698c2e41bd58716b77 = L.popup({maxWidth: '300'
            
            });

            
                var html_3eb7f55de0194edc8124f05958d441e8 = $('<div id="html_3eb7f55de0194edc8124f05958d441e8" style="width: 100.0%; height: 100.0%;">89 - C/ ROSSELLÓ, 101</div>')[0];
                popup_0f377ef04cec43698c2e41bd58716b77.setContent(html_3eb7f55de0194edc8124f05958d441e8);
            

            circle_marker_efdd786a997d4b8dafd3b8b261dcfa0b.bindPopup(popup_0f377ef04cec43698c2e41bd58716b77)
            ;

            
        
    
        var marker_73aa914361a3479d8353fb3505a67d7e = L.marker(
            [41.387953, 2.150169],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_ec571cd0696347198d11ee7a34c6b5b5 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'green',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_73aa914361a3479d8353fb3505a67d7e.setIcon(icon_ec571cd0696347198d11ee7a34c6b5b5);
            
    
            var popup_f523e0c8232b4de2b80d7e0af7e0c536 = L.popup({maxWidth: '300'
            
            });

            
                var html_023c7b1111fc44b6821823032d285e71 = $('<div id="html_023c7b1111fc44b6821823032d285e71" style="width: 100.0%; height: 100.0%;">89 - C/ ROSSELLÓ, 101::263.0</div>')[0];
                popup_f523e0c8232b4de2b80d7e0af7e0c536.setContent(html_023c7b1111fc44b6821823032d285e71);
            

            marker_73aa914361a3479d8353fb3505a67d7e.bindPopup(popup_f523e0c8232b4de2b80d7e0af7e0c536)
            ;

            
        
    
            var circle_marker_aa0b93801dad4298b5c827e1eee236cd = L.circleMarker(
                [41.405587, 2.162317],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_b17fb692a3c24380b82bd02c61960b59 = L.popup({maxWidth: '300'
            
            });

            
                var html_7d4cf79c67754f34a0f34d2b683b88f2 = $('<div id="html_7d4cf79c67754f34a0f34d2b683b88f2" style="width: 100.0%; height: 100.0%;">106 - PL. JOANIC- C/ BRUNIQUER, 59</div>')[0];
                popup_b17fb692a3c24380b82bd02c61960b59.setContent(html_7d4cf79c67754f34a0f34d2b683b88f2);
            

            circle_marker_aa0b93801dad4298b5c827e1eee236cd.bindPopup(popup_b17fb692a3c24380b82bd02c61960b59)
            ;

            
        
    
        var marker_98333144b2f9461f96fd754a6d52d23d = L.marker(
            [41.405587, 2.162317],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_d8e9cf92252b48c29b2e2fb7f5f7c333 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'red',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_98333144b2f9461f96fd754a6d52d23d.setIcon(icon_d8e9cf92252b48c29b2e2fb7f5f7c333);
            
    
            var popup_026d7016aecd47028487d605198e4445 = L.popup({maxWidth: '300'
            
            });

            
                var html_ccb343d6bd2545d69cb988847239b4fc = $('<div id="html_ccb343d6bd2545d69cb988847239b4fc" style="width: 100.0%; height: 100.0%;">106 - PL. JOANIC- C/ BRUNIQUER, 59::182.0</div>')[0];
                popup_026d7016aecd47028487d605198e4445.setContent(html_ccb343d6bd2545d69cb988847239b4fc);
            

            marker_98333144b2f9461f96fd754a6d52d23d.bindPopup(popup_026d7016aecd47028487d605198e4445)
            ;

            
        
    
            var circle_marker_e61bced7212d4ecfbf4e3fec4aa10c58 = L.circleMarker(
                [41.402314, 2.164961],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_98de3bb6d5e84f78975ee8e3f8114b94 = L.popup({maxWidth: '300'
            
            });

            
                var html_74f2098239864d34b4b8c8db7c205579 = $('<div id="html_74f2098239864d34b4b8c8db7c205579" style="width: 100.0%; height: 100.0%;">108 - C/ INDÚSTRIA 10</div>')[0];
                popup_98de3bb6d5e84f78975ee8e3f8114b94.setContent(html_74f2098239864d34b4b8c8db7c205579);
            

            circle_marker_e61bced7212d4ecfbf4e3fec4aa10c58.bindPopup(popup_98de3bb6d5e84f78975ee8e3f8114b94)
            ;

            
        
    
        var marker_7f25d92f8a8b461781c06d311a20cfc5 = L.marker(
            [41.402314, 2.164961],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_dac86a68de61417f9d2f2545f9274e69 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'orange',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_7f25d92f8a8b461781c06d311a20cfc5.setIcon(icon_dac86a68de61417f9d2f2545f9274e69);
            
    
            var popup_3578eedc0e58452b9270eb2736388f96 = L.popup({maxWidth: '300'
            
            });

            
                var html_1a499271d3934a8caf4b146a2ff3a368 = $('<div id="html_1a499271d3934a8caf4b146a2ff3a368" style="width: 100.0%; height: 100.0%;">108 - C/ INDÚSTRIA 10::241.0</div>')[0];
                popup_3578eedc0e58452b9270eb2736388f96.setContent(html_1a499271d3934a8caf4b146a2ff3a368);
            

            marker_7f25d92f8a8b461781c06d311a20cfc5.bindPopup(popup_3578eedc0e58452b9270eb2736388f96)
            ;

            
        
    
            var circle_marker_4b91393c685d497a89194f3a7e548dca = L.circleMarker(
                [41.385377, 2.154878],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_f478d3ea519b4b76ae04084af9fb8e70 = L.popup({maxWidth: '300'
            
            });

            
                var html_0b4ab427398c48b59ede2e8957e4998b = $('<div id="html_0b4ab427398c48b59ede2e8957e4998b" style="width: 100.0%; height: 100.0%;">110 - AV. ROMA, 136</div>')[0];
                popup_f478d3ea519b4b76ae04084af9fb8e70.setContent(html_0b4ab427398c48b59ede2e8957e4998b);
            

            circle_marker_4b91393c685d497a89194f3a7e548dca.bindPopup(popup_f478d3ea519b4b76ae04084af9fb8e70)
            ;

            
        
    
        var marker_b02f1248d6fd4c3fad7f5db88be4d534 = L.marker(
            [41.385377, 2.154878],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_4c1f9694b74c415989bd04626af6104a = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'orange',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_b02f1248d6fd4c3fad7f5db88be4d534.setIcon(icon_4c1f9694b74c415989bd04626af6104a);
            
    
            var popup_2a194dfd90f544b3bc0f80f2213b1853 = L.popup({maxWidth: '300'
            
            });

            
                var html_ecb2a21b1c0442359f1261f72480ef70 = $('<div id="html_ecb2a21b1c0442359f1261f72480ef70" style="width: 100.0%; height: 100.0%;">110 - AV. ROMA, 136::231.0</div>')[0];
                popup_2a194dfd90f544b3bc0f80f2213b1853.setContent(html_ecb2a21b1c0442359f1261f72480ef70);
            

            marker_b02f1248d6fd4c3fad7f5db88be4d534.bindPopup(popup_2a194dfd90f544b3bc0f80f2213b1853)
            ;

            
        
    
            var circle_marker_453e0fa78c1c4f98885c0ef20277d92b = L.circleMarker(
                [41.405435, 2.16624],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_f9991f48aa9c456092318adce3e25b6d = L.popup({maxWidth: '300'
            
            });

            
                var html_3ec80a8584bc48f3a56aa724e9e13745 = $('<div id="html_3ec80a8584bc48f3a56aa724e9e13745" style="width: 100.0%; height: 100.0%;">122 - C/NÀPOLS, 344</div>')[0];
                popup_f9991f48aa9c456092318adce3e25b6d.setContent(html_3ec80a8584bc48f3a56aa724e9e13745);
            

            circle_marker_453e0fa78c1c4f98885c0ef20277d92b.bindPopup(popup_f9991f48aa9c456092318adce3e25b6d)
            ;

            
        
    
        var marker_62fd33f968b14f1ab7126dfff87b9d77 = L.marker(
            [41.405435, 2.16624],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_929abd3b707b40b1bc453976ffd434ed = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'green',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_62fd33f968b14f1ab7126dfff87b9d77.setIcon(icon_929abd3b707b40b1bc453976ffd434ed);
            
    
            var popup_1122cdad70364edcb296f76d373bc7d7 = L.popup({maxWidth: '300'
            
            });

            
                var html_644a49afa1be49d2a6394bb848c16b2c = $('<div id="html_644a49afa1be49d2a6394bb848c16b2c" style="width: 100.0%; height: 100.0%;">122 - C/NÀPOLS, 344::251.0</div>')[0];
                popup_1122cdad70364edcb296f76d373bc7d7.setContent(html_644a49afa1be49d2a6394bb848c16b2c);
            

            marker_62fd33f968b14f1ab7126dfff87b9d77.bindPopup(popup_1122cdad70364edcb296f76d373bc7d7)
            ;

            
        
    
            var circle_marker_e205671c44804ffa8a8e7cfeb3caa6e6 = L.circleMarker(
                [41.382564, 2.149094],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_10184c34c0b640d4b2426b09ff37eef4 = L.popup({maxWidth: '300'
            
            });

            
                var html_00720cb458164160a449df328aec62c0 = $('<div id="html_00720cb458164160a449df328aec62c0" style="width: 100.0%; height: 100.0%;">191 -C/ ROCAFORT, 167</div>')[0];
                popup_10184c34c0b640d4b2426b09ff37eef4.setContent(html_00720cb458164160a449df328aec62c0);
            

            circle_marker_e205671c44804ffa8a8e7cfeb3caa6e6.bindPopup(popup_10184c34c0b640d4b2426b09ff37eef4)
            ;

            
        
    
        var marker_ddf21f649a13474e8f2242ff9779f6c0 = L.marker(
            [41.382564, 2.149094],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_624afdfcaa2e4a1488f1ea357bff4962 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'orange',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_ddf21f649a13474e8f2242ff9779f6c0.setIcon(icon_624afdfcaa2e4a1488f1ea357bff4962);
            
    
            var popup_4c476f60d0244a6b907f70049dc491d2 = L.popup({maxWidth: '300'
            
            });

            
                var html_cc49e136e3be446e92a900a55cddfbd6 = $('<div id="html_cc49e136e3be446e92a900a55cddfbd6" style="width: 100.0%; height: 100.0%;">191 -C/ ROCAFORT, 167::199.0</div>')[0];
                popup_4c476f60d0244a6b907f70049dc491d2.setContent(html_cc49e136e3be446e92a900a55cddfbd6);
            

            marker_ddf21f649a13474e8f2242ff9779f6c0.bindPopup(popup_4c476f60d0244a6b907f70049dc491d2)
            ;

            
        
    
            var circle_marker_b889fbe7a6bb440984097554774023d3 = L.circleMarker(
                [41.399945, 2.164389],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_a312dd922a0a469babea91c9c4f0b9b2 = L.popup({maxWidth: '300'
            
            });

            
                var html_7cc2cb5fffff447fa4c061544de4488f = $('<div id="html_7cc2cb5fffff447fa4c061544de4488f" style="width: 100.0%; height: 100.0%;">224- C /GIRONA, 176</div>')[0];
                popup_a312dd922a0a469babea91c9c4f0b9b2.setContent(html_7cc2cb5fffff447fa4c061544de4488f);
            

            circle_marker_b889fbe7a6bb440984097554774023d3.bindPopup(popup_a312dd922a0a469babea91c9c4f0b9b2)
            ;

            
        
    
        var marker_107693e9da2549558c2c08c5a9f91a0f = L.marker(
            [41.399945, 2.164389],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_5fe74d5bfca647d1b8b2863ae474529c = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'orange',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_107693e9da2549558c2c08c5a9f91a0f.setIcon(icon_5fe74d5bfca647d1b8b2863ae474529c);
            
    
            var popup_a8cab2e15cc84016ac9386746e9e4c03 = L.popup({maxWidth: '300'
            
            });

            
                var html_b1a3ed0b840749ab8d5a31a18b88507e = $('<div id="html_b1a3ed0b840749ab8d5a31a18b88507e" style="width: 100.0%; height: 100.0%;">224- C /GIRONA, 176::236.0</div>')[0];
                popup_a8cab2e15cc84016ac9386746e9e4c03.setContent(html_b1a3ed0b840749ab8d5a31a18b88507e);
            

            marker_107693e9da2549558c2c08c5a9f91a0f.bindPopup(popup_a8cab2e15cc84016ac9386746e9e4c03)
            ;

            
        
    
            var circle_marker_583fefe99206455981c75650328ccc9b = L.circleMarker(
                [41.40694, 2.155794],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_574a206e71f148fcbe77f857e556b3ad = L.popup({maxWidth: '300'
            
            });

            
                var html_a736f6c9d8424e56ba86ed6ec55b49f6 = $('<div id="html_a736f6c9d8424e56ba86ed6ec55b49f6" style="width: 100.0%; height: 100.0%;">228 - PL.DEL NORD, 5</div>')[0];
                popup_574a206e71f148fcbe77f857e556b3ad.setContent(html_a736f6c9d8424e56ba86ed6ec55b49f6);
            

            circle_marker_583fefe99206455981c75650328ccc9b.bindPopup(popup_574a206e71f148fcbe77f857e556b3ad)
            ;

            
        
    
        var marker_6a73453597f1460fa48024273e2bffbc = L.marker(
            [41.40694, 2.155794],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_e1693b65897348ed929977f542c46928 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'red',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_6a73453597f1460fa48024273e2bffbc.setIcon(icon_e1693b65897348ed929977f542c46928);
            
    
            var popup_d5a3bf29e5064d4183b7fc44694b5301 = L.popup({maxWidth: '300'
            
            });

            
                var html_b1ab408e5452403da5bd48850fed8da7 = $('<div id="html_b1ab408e5452403da5bd48850fed8da7" style="width: 100.0%; height: 100.0%;">228 - PL.DEL NORD, 5::132.0</div>')[0];
                popup_d5a3bf29e5064d4183b7fc44694b5301.setContent(html_b1ab408e5452403da5bd48850fed8da7);
            

            marker_6a73453597f1460fa48024273e2bffbc.bindPopup(popup_d5a3bf29e5064d4183b7fc44694b5301)
            ;

            
        
    
            var circle_marker_0ce94d376a3f4157ae791ae3c38407f6 = L.circleMarker(
                [41.405107, 2.156874],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_280035ca626f4074a7867abd58bf384e = L.popup({maxWidth: '300'
            
            });

            
                var html_b3e44f60e4aa4f7e87f8afc1867fed78 = $('<div id="html_b3e44f60e4aa4f7e87f8afc1867fed78" style="width: 100.0%; height: 100.0%;">229 - C/ DE LA SANTACREU, 2 (PL.DE LA VIRREINA)</div>')[0];
                popup_280035ca626f4074a7867abd58bf384e.setContent(html_b3e44f60e4aa4f7e87f8afc1867fed78);
            

            circle_marker_0ce94d376a3f4157ae791ae3c38407f6.bindPopup(popup_280035ca626f4074a7867abd58bf384e)
            ;

            
        
    
        var marker_6ec0a88d6d1546028d1ad3b742a5203c = L.marker(
            [41.405107, 2.156874],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_eb728e21c1644d39bbb9f1059d949f41 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'orange',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_6ec0a88d6d1546028d1ad3b742a5203c.setIcon(icon_eb728e21c1644d39bbb9f1059d949f41);
            
    
            var popup_35a66624c8e04ad79885ef48b3385cea = L.popup({maxWidth: '300'
            
            });

            
                var html_75a035ef177b4fb19f7088107def74dc = $('<div id="html_75a035ef177b4fb19f7088107def74dc" style="width: 100.0%; height: 100.0%;">229 - C/ DE LA SANTACREU, 2 (PL.DE LA VIRREINA)::246.0</div>')[0];
                popup_35a66624c8e04ad79885ef48b3385cea.setContent(html_75a035ef177b4fb19f7088107def74dc);
            

            marker_6ec0a88d6d1546028d1ad3b742a5203c.bindPopup(popup_35a66624c8e04ad79885ef48b3385cea)
            ;

            
        
    
            var circle_marker_308899a02c5343e3b7fb1e7d088f1d54 = L.circleMarker(
                [41.407202, 2.163476],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_0fadae42d0644d01b529613e099e2f62 = L.popup({maxWidth: '300'
            
            });

            
                var html_b76ce519adc148e28ba2a3470667a02e = $('<div id="html_b76ce519adc148e28ba2a3470667a02e" style="width: 100.0%; height: 100.0%;">231- C/PI I MARGALL, 38</div>')[0];
                popup_0fadae42d0644d01b529613e099e2f62.setContent(html_b76ce519adc148e28ba2a3470667a02e);
            

            circle_marker_308899a02c5343e3b7fb1e7d088f1d54.bindPopup(popup_0fadae42d0644d01b529613e099e2f62)
            ;

            
        
    
        var marker_54a64402bd194a0491b868a41e2f96d3 = L.marker(
            [41.407202, 2.163476],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_d112d3b198bf4d01b190aa8ece04b077 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'red',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_54a64402bd194a0491b868a41e2f96d3.setIcon(icon_d112d3b198bf4d01b190aa8ece04b077);
            
    
            var popup_894ba2a604a84a9a8e254bd6dc7c3d1b = L.popup({maxWidth: '300'
            
            });

            
                var html_1f71133a44a74c159e9122a3089f45a5 = $('<div id="html_1f71133a44a74c159e9122a3089f45a5" style="width: 100.0%; height: 100.0%;">231- C/PI I MARGALL, 38::184.0</div>')[0];
                popup_894ba2a604a84a9a8e254bd6dc7c3d1b.setContent(html_1f71133a44a74c159e9122a3089f45a5);
            

            marker_54a64402bd194a0491b868a41e2f96d3.bindPopup(popup_894ba2a604a84a9a8e254bd6dc7c3d1b)
            ;

            
        
    
            var circle_marker_b45971a45c434c958f9c010f7cf2549f = L.circleMarker(
                [41.395, 2.16123],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_23380722c35f4c559daaefa471578198 = L.popup({maxWidth: '300'
            
            });

            
                var html_7fa254678ba14fb98f4b9ed1c2d39484 = $('<div id="html_7fa254678ba14fb98f4b9ed1c2d39484" style="width: 100.0%; height: 100.0%;">374 - PG. DE GRÀCIA, 89</div>')[0];
                popup_23380722c35f4c559daaefa471578198.setContent(html_7fa254678ba14fb98f4b9ed1c2d39484);
            

            circle_marker_b45971a45c434c958f9c010f7cf2549f.bindPopup(popup_23380722c35f4c559daaefa471578198)
            ;

            
        
    
        var marker_ffe30c79b54841a88e4d42e3df183f6d = L.marker(
            [41.395, 2.16123],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_e8903e531d664ec58e7d53e4ffef43e0 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'red',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_ffe30c79b54841a88e4d42e3df183f6d.setIcon(icon_e8903e531d664ec58e7d53e4ffef43e0);
            
    
            var popup_ebba61010c5e4d88bfa20a1dc084120c = L.popup({maxWidth: '300'
            
            });

            
                var html_e70f6128fc3a421c871602d14c4c430a = $('<div id="html_e70f6128fc3a421c871602d14c4c430a" style="width: 100.0%; height: 100.0%;">374 - PG. DE GRÀCIA, 89::167.0</div>')[0];
                popup_ebba61010c5e4d88bfa20a1dc084120c.setContent(html_e70f6128fc3a421c871602d14c4c430a);
            

            marker_ffe30c79b54841a88e4d42e3df183f6d.bindPopup(popup_ebba61010c5e4d88bfa20a1dc084120c)
            ;

            
        
    
            var circle_marker_84a97642112c42b6854319ee566713fb = L.circleMarker(
                [41.40542, 2.142456],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_5f0445e9993344ff9cfff743dd289c2e = L.popup({maxWidth: '300'
            
            });

            
                var html_6bab3e73aeb9485d9d8265477346ea46 = $('<div id="html_6bab3e73aeb9485d9d8265477346ea46" style="width: 100.0%; height: 100.0%;">465 - (PK) RDA. DEL GENERAL MITRE, 203</div>')[0];
                popup_5f0445e9993344ff9cfff743dd289c2e.setContent(html_6bab3e73aeb9485d9d8265477346ea46);
            

            circle_marker_84a97642112c42b6854319ee566713fb.bindPopup(popup_5f0445e9993344ff9cfff743dd289c2e)
            ;

            
        
    
        var marker_377dad61bfa14388a6d3b8ccc1208d90 = L.marker(
            [41.40542, 2.142456],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_2e0bdc4233254472bbf19b985d15ed0b = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'blue',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_377dad61bfa14388a6d3b8ccc1208d90.setIcon(icon_2e0bdc4233254472bbf19b985d15ed0b);
            
    
            var popup_7b04070791cf4da48d4c25f72bf56957 = L.popup({maxWidth: '300'
            
            });

            
                var html_316bfeb038bb48e597b6fc41c0fab9ff = $('<div id="html_316bfeb038bb48e597b6fc41c0fab9ff" style="width: 100.0%; height: 100.0%;">465 - (PK) RDA. DEL GENERAL MITRE, 203::266.0</div>')[0];
                popup_7b04070791cf4da48d4c25f72bf56957.setContent(html_316bfeb038bb48e597b6fc41c0fab9ff);
            

            marker_377dad61bfa14388a6d3b8ccc1208d90.bindPopup(popup_7b04070791cf4da48d4c25f72bf56957)
            ;

            
        
    
            var circle_marker_32b4ead969414845b0d71922ffee3710 = L.circleMarker(
                [41.3989, 2.153668],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_63d0a455c94c44c8b0ddfb136b1d8d8e = L.popup({maxWidth: '300'
            
            });

            
                var html_8624dea7b8664c0bb9359c09643a9197 = $('<div id="html_8624dea7b8664c0bb9359c09643a9197" style="width: 100.0%; height: 100.0%;">474 - (PK) PL. GAL·LA PLACÍDIA, 2</div>')[0];
                popup_63d0a455c94c44c8b0ddfb136b1d8d8e.setContent(html_8624dea7b8664c0bb9359c09643a9197);
            

            circle_marker_32b4ead969414845b0d71922ffee3710.bindPopup(popup_63d0a455c94c44c8b0ddfb136b1d8d8e)
            ;

            
        
    
        var marker_15604df242a149eba5c58acbc407d5d9 = L.marker(
            [41.3989, 2.153668],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_b74de9f7a40a4dcb96ee01d990bc0790 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'red',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_15604df242a149eba5c58acbc407d5d9.setIcon(icon_b74de9f7a40a4dcb96ee01d990bc0790);
            
    
            var popup_b35a717781fc4b6692ab230c716f17bf = L.popup({maxWidth: '300'
            
            });

            
                var html_96a45861c2ea484b81a3417bed1a9c00 = $('<div id="html_96a45861c2ea484b81a3417bed1a9c00" style="width: 100.0%; height: 100.0%;">474 - (PK) PL. GAL·LA PLACÍDIA, 2::57.0</div>')[0];
                popup_b35a717781fc4b6692ab230c716f17bf.setContent(html_96a45861c2ea484b81a3417bed1a9c00);
            

            marker_15604df242a149eba5c58acbc407d5d9.bindPopup(popup_b35a717781fc4b6692ab230c716f17bf)
            ;

            
        
    
            var circle_marker_60977951ef694ac29f6fd75dcc52127f = L.circleMarker(
                [41.396934, 2.159445],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_5a563dcfed1a41d59a32890ffe9a47d7 = L.popup({maxWidth: '300'
            
            });

            
                var html_7a77f88a51314a6ab4ce7061ecde1a24 = $('<div id="html_7a77f88a51314a6ab4ce7061ecde1a24" style="width: 100.0%; height: 100.0%;">486 - (PK) PG. DE GRÀCIA - DIAGONAL</div>')[0];
                popup_5a563dcfed1a41d59a32890ffe9a47d7.setContent(html_7a77f88a51314a6ab4ce7061ecde1a24);
            

            circle_marker_60977951ef694ac29f6fd75dcc52127f.bindPopup(popup_5a563dcfed1a41d59a32890ffe9a47d7)
            ;

            
        
    
        var marker_996fde0ab97d46c6acd78054c70de50d = L.marker(
            [41.396934, 2.159445],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_c40daf6959f04b778bd78c6465d4356c = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'blue',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_996fde0ab97d46c6acd78054c70de50d.setIcon(icon_c40daf6959f04b778bd78c6465d4356c);
            
    
            var popup_798ad9b66b5148669e00b97b68553edd = L.popup({maxWidth: '300'
            
            });

            
                var html_1c84136aff3041cd95f4e85428b0b1aa = $('<div id="html_1c84136aff3041cd95f4e85428b0b1aa" style="width: 100.0%; height: 100.0%;">486 - (PK) PG. DE GRÀCIA - DIAGONAL::266.0</div>')[0];
                popup_798ad9b66b5148669e00b97b68553edd.setContent(html_1c84136aff3041cd95f4e85428b0b1aa);
            

            marker_996fde0ab97d46c6acd78054c70de50d.bindPopup(popup_798ad9b66b5148669e00b97b68553edd)
            ;

            
        
    
            var circle_marker_20cfc2379e944615be4e5e08425b5ab2 = L.circleMarker(
                [41.398373, 2.153154],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_84d7499f76cd4ab6ae553dc939c82027 = L.popup({maxWidth: '300'
            
            });

            
                var html_f1e15a802ce1419590db3e4c61b27710 = $('<div id="html_f1e15a802ce1419590db3e4c61b27710" style="width: 100.0%; height: 100.0%;">107 - TRAV. DE GRÀCIA 92 / VIA AUGUSTA</div>')[0];
                popup_84d7499f76cd4ab6ae553dc939c82027.setContent(html_f1e15a802ce1419590db3e4c61b27710);
            

            circle_marker_20cfc2379e944615be4e5e08425b5ab2.bindPopup(popup_84d7499f76cd4ab6ae553dc939c82027)
            ;

            
        
    
        var marker_0649ef54b50148b3a2f0653fa262d71b = L.marker(
            [41.398373, 2.153154],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_7c3cec38bedd4534ae065e4edc23e570 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'orange',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_0649ef54b50148b3a2f0653fa262d71b.setIcon(icon_7c3cec38bedd4534ae065e4edc23e570);
            
    
            var popup_01b849c2370a4a59b43e460849c22456 = L.popup({maxWidth: '300'
            
            });

            
                var html_5ac13f255b0a450d9cd01c959d448c62 = $('<div id="html_5ac13f255b0a450d9cd01c959d448c62" style="width: 100.0%; height: 100.0%;">107 - TRAV. DE GRÀCIA 92 / VIA AUGUSTA::204.0</div>')[0];
                popup_01b849c2370a4a59b43e460849c22456.setContent(html_5ac13f255b0a450d9cd01c959d448c62);
            

            marker_0649ef54b50148b3a2f0653fa262d71b.bindPopup(popup_01b849c2370a4a59b43e460849c22456)
            ;

            
        
    
            var heat_map_aa07254fbe13437aae717ac9e1034219 = L.heatLayer(
                [[41.403309859099664, 2.1521850379155514], [41.40181954961136, 2.15212354191599], [41.40252399485162, 2.1541074417935593], [41.4024991371333, 2.1541473478159165], [41.40172818421211, 2.151513949455388], [41.401073322999046, 2.152520908777154], [41.40381775999537, 2.153006325000218], [41.4024752234313, 2.151270034155539], [41.40341477324329, 2.151849548113032], [41.40177956014948, 2.1521191904077686], [41.40124150335853, 2.1522693271467213], [41.40199086812038, 2.1527794968345293], [41.402512501976396, 2.1535180082218726], [41.40234178189931, 2.15393102246908], [41.40336871641285, 2.151830882377483], [41.402704144394, 2.1538624165962013], [41.40269006227144, 2.151228468035271], [41.40256584269451, 2.151930942148317], [41.40390440744452, 2.15376289185034], [41.40152557431332, 2.1512047908509038], [41.40417861419942, 2.1527943320111484], [41.40402335463647, 2.151119956383269], [41.40088556429481, 2.152851771627446], [41.403870996081174, 2.150928507050988], [41.401154864381525, 2.15093501258914], [41.402868347182576, 2.152039864814522], [41.40321518089324, 2.1514661614552417], [41.40246960867502, 2.1528308590154253], [41.401799655887125, 2.152605100637584], [41.40093041069562, 2.1519797122827447], [41.402084030009235, 2.1512196531035515], [41.40170678059006, 2.153326507679404], [41.40235308812968, 2.152975855356589], [41.4041155316387, 2.1520267345246813], [41.402720879373355, 2.152718839014712], [41.403046567443745, 2.151612466144095], [41.40261408553937, 2.151993191464785], [41.40210018359861, 2.1527742188234473], [41.40305347167135, 2.1521741508497083], [41.40399406740089, 2.1518146794810615], [41.40103476894876, 2.1522192298753384], [41.40129071027865, 2.1522980517433], [41.40274162137902, 2.151238841249528], [41.40187316629966, 2.1525639023204146], [41.403987123712064, 2.151623502839703], [41.402817730451574, 2.1532532558441098], [41.404155084888096, 2.15295941733257], [41.40217456715852, 2.1538625192274243], [41.40260855718433, 2.151188402016893], [41.40266261046022, 2.152361272798773], [41.40353899335425, 2.152716049697949], [41.40266948635823, 2.1519659240716846], [41.404156528915756, 2.1528235973870853], [41.401731335938486, 2.152025437248911], [41.40411070140945, 2.151587025023517], [41.40147778078129, 2.1525516907510607], [41.403271654594185, 2.1508616823957487], [41.404030328760825, 2.1514030851209665], [41.40155101875454, 2.151074085950952], [41.40091882971642, 2.1532590570671237], [41.4039593567602, 2.153612700229997], [41.40405361120166, 2.1529243303049137], [41.40132935993195, 2.1516978394780106], [41.401842912754105, 2.152474056872414], [41.40105745669867, 2.153977480323412], [41.4009547156994, 2.1514634633813094], [41.40329367798736, 2.1522563297388175], [41.40331321399714, 2.1528011439396715], [41.400958602820396, 2.1520950056345742], [41.40228665792718, 2.153062171471235], [41.40094474979203, 2.152455803283539], [41.40400155982582, 2.1541178104013183], [41.40161977442402, 2.152814286109641], [41.401867928814596, 2.1525530656931147], [41.402590963519884, 2.1533239412220553], [41.40310179366495, 2.152184115906431], [41.40176845107955, 2.153077086353103], [41.400899877818006, 2.153357635233846], [41.40158017043975, 2.1522692218565855], [41.40381717906632, 2.1513015174229735], [41.40088235104758, 2.154005044038152], [41.402627624984504, 2.153853674234475], [41.402233290484624, 2.152979707383645], [41.40412292917655, 2.1539464135892215], [41.40122264666583, 2.152170438273102], [41.40235304122522, 2.150979916553486], [41.401142553465, 2.153533205497393], [41.40207799361685, 2.1529830395492566], [41.40121674401017, 2.1537512181171015], [41.40265310305926, 2.1524354717826317], [41.401039735239415, 2.1515744426298147], [41.403409986164725, 2.151757958389885], [41.40240721329135, 2.151173348845519], [41.39845526958453, 2.1600417013435527], [41.398887316056424, 2.1592022007328113], [41.397721637933884, 2.159918250177366], [41.397963808016335, 2.1597314494859376], [41.397506523568325, 2.160667273366283], [41.398988384831306, 2.1605066748618045], [41.39899389806905, 2.1605099979967126], [41.39788912150108, 2.1591304805727973], [41.39908976089943, 2.1597906972673533], [41.39892867515554, 2.1603283239762527], [41.398403626903374, 2.160341477506525], [41.3986985218633, 2.1591323098533475], [41.39869578391833, 2.1592693094822986], [41.39832566270166, 2.159424050239344], [41.398897677872256, 2.1599083091265494], [41.39899567368082, 2.1600596583352503], [41.39804112468801, 2.1602091264061993], [41.3977754819771, 2.159688603810698], [41.39752680164683, 2.1595846562226355], [41.39788617236652, 2.1598442688316624], [41.39810686631593, 2.160478472220072], [41.397909350747106, 2.1600366269461895], [41.39862942549333, 2.160487286856497], [41.39860592488847, 2.159573464738476], [41.39802638638719, 2.1592792524463165], [41.39903067995754, 2.160534915297682], [41.397789173246295, 2.1591086853580297], [41.399045663980644, 2.1590828701545806], [41.39777558157899, 2.160026455088131], [41.398850168226325, 2.160347902477521], [41.39750693753295, 2.1605865880381523], [41.39859822731395, 2.160192728474962], [41.398515147924726, 2.159137438310477], [41.39892032054575, 2.1604667337572607], [41.39857024920867, 2.160390519439808], [41.39886806379018, 2.160019802427627], [41.39905459361096, 2.159976627996302], [41.39881288820952, 2.159700473387994], [41.398813563005916, 2.160089926738341], [41.3984131348692, 2.159656927539897], [41.39906257560975, 2.159263556882262], [41.39826861569694, 2.159314351880283], [41.39853797771344, 2.1597728035282255], [41.397760596130006, 2.1600080324946775], [41.39752043784951, 2.1592903500027343], [41.39779459469743, 2.1602170099896014], [41.39749917864044, 2.1594099751736295], [41.397901273813936, 2.1603910651676452], [41.39851668537966, 2.159613084176475], [41.398812490208826, 2.159079877952134], [41.39822565157083, 2.1595071720436487], [41.397626731453286, 2.1595492481624765], [41.39901858996959, 2.159427258676234], [41.39761359410468, 2.1599956084678262], [41.3984215985504, 2.1604235356710095], [41.397592319941865, 2.1596856350430125], [41.39859310733627, 2.1605259506480636], [41.39882529676395, 2.1604878403761796], [41.39909538326118, 2.1593648509554915], [41.398443789647786, 2.1599968991804297], [41.397588027764144, 2.1601374577931027], [41.39751603247476, 2.1592881472772376], [41.398883385545936, 2.1594683958345153], [41.398049210523, 2.160471134625439], [41.3975199217682, 2.159922342651037], [41.397739063382176, 2.1597215663168856], [41.39759096148397, 2.159741159967619], [41.39842435373546, 2.160357026162134], [41.39778771112276, 2.160262151243042], [41.398416206883475, 2.1604235691283673], [41.39873324759809, 2.1603899937470374], [41.398087351809494, 2.1595620197755605], [41.398139335142346, 2.1600898854864568], [41.39804724331322, 2.1603875571077817], [41.404277614004535, 2.1604788489831694], [41.40354022176881, 2.161782925726851], [41.40277761000748, 2.1609814455718057], [41.403309595976324, 2.161572046871532], [41.403176066923606, 2.161427925735381], [41.40300683057193, 2.16143823101957], [41.40360114412787, 2.1612944735960675], [41.40319490323098, 2.161174880291111], [41.40366809731147, 2.1606079227618324], [41.40382755050379, 2.161539407059219], [41.40404428527836, 2.1611835844984975], [41.40351207139945, 2.1607819395138455], [41.40360824118682, 2.161674548636279], [41.40291398076509, 2.161770900975857], [41.403143746088816, 2.161445119040361], [41.404237142591136, 2.160566502814751], [41.40380727231513, 2.1603514947280424], [41.40414644982953, 2.1607824057016782], [41.403098671339016, 2.1608763444334693], [41.40391135561831, 2.160673685319634], [41.40421541882773, 2.161732790871693], [41.403285742810176, 2.1604848567354327], [41.40360493966746, 2.1606563643028105], [41.40363060325563, 2.1609046367141103], [41.403116840226744, 2.161845915027155], [41.403876040313456, 2.161829363732375], [41.40281887899247, 2.1608679660259753], [41.40287228674918, 2.1613825961881017], [41.40318457528152, 2.1612096254402737], [41.40354901308208, 2.1613204831129766], [41.402891778446396, 2.1617114310539796], [41.40328677337129, 2.1607532750930716], [41.4031419678135, 2.160655216811301], [41.40316802267215, 2.1608813816567887], [41.40261715707497, 2.161936252705061], [41.40279912020073, 2.160489616509368], [41.40315786963061, 2.161284004987996], [41.40321814246047, 2.16165343882502], [41.40394830842676, 2.1603827971282055], [41.404000626274545, 2.1609410792105948], [41.40349742220965, 2.161742156531714], [41.40302704639066, 2.160290756903996], [41.4033455822812, 2.1605267943353055], [41.40340956376378, 2.161169156336684], [41.402637922285685, 2.1612786883277955], [41.40301648255696, 2.160656493412763], [41.403566171787624, 2.160928304512482], [41.40348227197941, 2.1613133841985634], [41.40300639573747, 2.16062449969351], [41.40374616065591, 2.1612790357395673], [41.40393801941969, 2.1603297585369896], [41.40382960554657, 2.161630288719332], [41.40274130176612, 2.1614929115649364], [41.40334256376648, 2.161068308724775], [41.40361554872653, 2.160485399843948], [41.40385925191359, 2.16051197285974], [41.403091231771505, 2.160528020287294], [41.40261655308726, 2.1603753254687064], [41.40379594513486, 2.161296387470128], [41.40335843540264, 2.161732152785018], [41.403650818514684, 2.161268493808056], [41.40331733812931, 2.1605547141737054], [41.40331903185501, 2.1602897454278334], [41.40397491056315, 2.161311575471427], [41.40361534595212, 2.160441087105113], [41.403722702898364, 2.1615504508065366], [41.40370165395605, 2.1603105982851725], [41.40353313929478, 2.1606347505057655], [41.40317278807604, 2.16168835372347], [41.40305732370609, 2.160306594962537], [41.40279976631259, 2.161311614169262], [41.403557078602994, 2.161422784265717], [41.40390808696355, 2.16193391274229], [41.404015035282555, 2.1619177859890084], [41.40297931579231, 2.161933295575228], [41.40644152611839, 2.1519194173459706], [41.4068324006764, 2.151878882429393], [41.40677041089853, 2.1526170727710214], [41.40523963844399, 2.1516720566421883], [41.4060321591056, 2.1526426365817914], [41.40586172668308, 2.1522331471282574], [41.40659198157717, 2.1520444474285703], [41.40705342226779, 2.151259072287588], [41.404709726800675, 2.150735259875619], [41.40551423326826, 2.152634048327058], [41.405252074544606, 2.151771680957149], [41.40477981526898, 2.151590169675974], [41.40701283455285, 2.150836752263629], [41.407185694014494, 2.1523331210116803], [41.40513925046628, 2.1524246729872063], [41.406911878734434, 2.1520188458390153], [41.405538060026714, 2.152651167412334], [41.405940419092055, 2.1517260618024574], [41.405479796690216, 2.150374795276415], [41.406644289674844, 2.1504992834176893], [41.40636081051026, 2.1528899723712045], [41.406326353493164, 2.151280764429729], [41.40652722440244, 2.151729134255275], [41.40653974425502, 2.1519454603000234], [41.40618713341051, 2.1504107226743714], [41.40486045420844, 2.1518275344403506], [41.40726417155395, 2.1520832434857997], [41.40695561563293, 2.1521543159434855], [41.40559495309653, 2.1511927583035937], [41.40621206454818, 2.1507862272856006], [41.40501207728905, 2.151357738386269], [41.404762698272975, 2.1515850981294835], [41.40501785216648, 2.1503735997921773], [41.40524036693033, 2.1527940007381647], [41.40504684000262, 2.1510030026142757], [41.40677979085238, 2.152652786328624], [41.404905346887986, 2.1511993143985033], [41.40471576495196, 2.150553812259905], [41.405944585930115, 2.15226664660277], [41.40639378616196, 2.15181253413186], [41.405103750419705, 2.152732483450278], [41.40507989574046, 2.1522883051765693], [41.40523540706458, 2.1519829395939643], [41.40566008262545, 2.1520802289750076], [41.40630663644352, 2.1518127365475697], [41.40730363091751, 2.1509614495941136], [41.405481680895164, 2.151363717488739], [41.40646047924659, 2.151350667592398], [41.40493934924103, 2.152505660546456], [41.40631022032517, 2.1509506263542675], [41.40670512395795, 2.1508201053045353], [41.40667700467811, 2.150760447165571], [41.40620750954298, 2.1508710242221345], [41.404795432278924, 2.151693480836934], [41.4063557714409, 2.1505827441941396], [41.40624610199736, 2.1520185857306133], [41.40659964950556, 2.150857831957577], [41.40570571130774, 2.1508838908004315], [41.40556281756326, 2.1511403170102437], [41.405400842342985, 2.151730637162761], [41.406333939685645, 2.152369254246332], [41.407131130460904, 2.1503554077966176], [41.405819832891616, 2.152937945473346], [41.40572468475542, 2.1522132973194563], [41.40473076218164, 2.1517909598390594], [41.404884266125734, 2.1516170947244873], [41.40642695970224, 2.152789771740792], [41.40606033484315, 2.1504637625740535], [41.40683841813715, 2.151572716596562], [41.405330891808774, 2.151626874309297], [41.406657494927735, 2.1528795379889623], [41.40631343510872, 2.1521315960859777], [41.40472173141101, 2.151525655514101], [41.40615503570398, 2.151889266908237], [41.4065871979082, 2.1517801898137], [41.40477449925095, 2.15229022573639], [41.40605162737675, 2.1524542437965426], [41.406089952294465, 2.151638799138341], [41.40706101672839, 2.1520992366977265], [41.406088634698115, 2.1512833223495385], [41.407269183807614, 2.152640970329308], [41.4067075478512, 2.1518216234523724], [41.405144279740085, 2.152733811790108], [41.40492308723074, 2.1526254702270213], [41.405522416986635, 2.1523511643000814], [41.40634349945524, 2.1526062160735444], [41.40669716346892, 2.1527099085539354], [41.38639809698555, 2.150970269147942], [41.385822930721964, 2.152737671378118], [41.38488459525908, 2.152774157910541], [41.38657684695827, 2.1514389388174915], [41.385159832201744, 2.151141112745494], [41.38575469921295, 2.1511700879373334], [41.384982910329924, 2.152934261649244], [41.38582309804549, 2.1512332857614065], [41.384417102200835, 2.1517587642996188], [41.38672257933406, 2.150851983738501], [41.38598027412942, 2.1507583340722563], [41.38633995928271, 2.1513715477590383], [41.38468209446866, 2.153132476469322], [41.3857689154052, 2.1532231853771884], [41.386492824736635, 2.15301513561729], [41.385738847025436, 2.1508664234674426], [41.38529118663718, 2.1532469385133295], [41.38656279159795, 2.1509432728013502], [41.38524994619641, 2.1516252640299682], [41.386057015601615, 2.153105744200442], [41.38671475210165, 2.1515916846743965], [41.38568962184647, 2.1516766050160627], [41.3849343474301, 2.1524276399875464], [41.38508463962558, 2.1516747581157696], [41.38667630544541, 2.153028560885883], [41.385216623232814, 2.153248882490933], [41.38493234908667, 2.152565956193447], [41.38568507644199, 2.151949523276346], [41.38526167093402, 2.1519775028305745], [41.384273057333886, 2.1509672140562106], [41.38546706537274, 2.153252194101209], [41.38514199888721, 2.1527890078431753], [41.385818661512054, 2.151625854528319], [41.385080556485605, 2.1518867213761426], [41.38455433001542, 2.1507757439797692], [41.38548587746705, 2.151758841183193], [41.38536924094048, 2.1513656534401044], [41.38493583217013, 2.1523592692675146], [41.384550097526635, 2.1507674913910857], [41.384386044760646, 2.153014983744717], [41.385357238435475, 2.1531802089256122], [41.38448743920101, 2.15152951426539], [41.38458645347369, 2.1510711429739313], [41.38649142078677, 2.152652755978154], [41.38542762925847, 2.1522168634540226], [41.38492857616967, 2.151719033703386], [41.38636967297341, 2.1530514892178063], [41.38522724854057, 2.150815542747494], [41.3853117299056, 2.1515637350066137], [41.38639052336978, 2.152038982038518], [41.38530477758146, 2.152584975228786], [41.386380794617686, 2.153056780103792], [41.38439620478474, 2.1510321112738735], [41.3859219635179, 2.151042646089798], [41.38585033233765, 2.152797041207778], [41.38629966608191, 2.151306963716314], [41.38668349858138, 2.1526147115284444], [41.38458288996203, 2.151400457961625], [41.385664879892595, 2.152082563382859], [41.38472596756561, 2.1514746731424883], [41.38619413465804, 2.1512629250173054], [41.3860645406936, 2.150842674209626], [41.38437786149917, 2.152431605254636], [41.38573002974374, 2.1514191443015647], [41.38487535740216, 2.152360648357365], [41.38609724855852, 2.1514443434304966], [41.38519228781621, 2.1514377581411157], [41.38438153657525, 2.152083008900039], [41.38427762947717, 2.151519406166777], [41.38473047924632, 2.1520797684905353], [41.38576914232958, 2.151454129899747], [41.38439487840945, 2.1509091543109213], [41.38531710530126, 2.1514099005938205], [41.38625086436089, 2.152056060777713], [41.38448734035936, 2.153228615508118], [41.38664319928271, 2.1531335244913783], [41.38465172088706, 2.152453144670268], [41.386628017204885, 2.151658553389696], [41.38533503054039, 2.1522743831216444], [41.38610250968993, 2.1523182310849003], [41.3853700946535, 2.1511800759111437], [41.38509395618504, 2.152102545189919], [41.384440120492116, 2.1526452318616944], [41.38466432307042, 2.15112349852162], [41.386315666989226, 2.1526432235990502], [41.4090644764057, 2.15082114756546], [41.40758974096342, 2.152086842456519], [41.40723332911488, 2.150952124912809], [41.406050742373644, 2.149649640033535], [41.40859271100594, 2.1506990505650694], [41.40900968204539, 2.153311901871669], [41.40774326706659, 2.151513481168723], [41.40662984393028, 2.1505849150755396], [41.40731671599635, 2.1505490831054623], [41.406723014599805, 2.1510260838859647], [41.406866865972226, 2.1518094295581727], [41.40829401431292, 2.1528608742990896], [41.40602666214014, 2.15164928017658], [41.40675276084461, 2.151335774475861], [41.40583911399572, 2.1502755864108023], [41.406068756305686, 2.1494067639432384], [41.40561031732419, 2.150539623561849], [41.407750046435275, 2.149612573752862], [41.40865842101285, 2.1494830196053516], [41.40896858184225, 2.1500411074863033], [41.40884832735874, 2.1503621139642757], [41.40861945767488, 2.150364483248976], [41.40865794933562, 2.1497311341918026], [41.407078445297714, 2.1520707800208076], [41.406660427719885, 2.150245343654168], [41.405520095139074, 2.152102978102844], [41.40636485340515, 2.152332741351455], [41.40898990264832, 2.150434495593709], [41.406744125634376, 2.1527730071374216], [41.40917602130824, 2.152916476435834], [41.40665966674197, 2.1529771943157145], [41.4066488343029, 2.151925887422803], [41.40599915262918, 2.151489090634049], [41.40693874873123, 2.150233749447014], [41.40800167429386, 2.14983332180792], [41.40694817967958, 2.1510619469031607], [41.40777241009424, 2.152486996155591], [41.40564691478877, 2.1521572024252973], [41.408006452653225, 2.15059167668007], [41.40598760126814, 2.1510718946451877], [41.40753025342291, 2.153144784497639], [41.406351368027664, 2.152082568135293], [41.406569801951264, 2.1510522771696916], [41.4093005212873, 2.1528710129882067], [41.40728544422018, 2.1530229168859427], [41.40840635774715, 2.1520889991371863], [41.40788488332105, 2.152677487848509], [41.4068013898334, 2.150086024916505], [41.409201989209755, 2.1532814135943377], [41.40683383967815, 2.151196775251741], [41.40555043958583, 2.150794298776757], [41.407792663642546, 2.151758693799443], [41.40662157605115, 2.1511027864833174], [41.407885234697645, 2.1511790726889912], [41.406217667243624, 2.1505024834076245], [41.40631071017763, 2.152445924391683], [41.4058148276367, 2.1497874708167544], [41.40832049046985, 2.151820592399873], [41.40729045998657, 2.14976196607203], [41.4060168541695, 2.150289280910973], [41.407918201855836, 2.1504055662077888], [41.40654389555119, 2.1509640072643625], [41.405566636449095, 2.149752371311225], [41.40679610698638, 2.149354366744953], [41.40923596255834, 2.1526191399825962], [41.40865455960387, 2.1529629762691287], [41.40736981897624, 2.1508051000230233], [41.40538887595449, 2.1529952839221873], [41.40916211048581, 2.1526781320308257], [41.40809455794592, 2.1527752376423837], [41.408692203186625, 2.153025933974529], [41.4068852902402, 2.1508793519840936], [41.4082713049766, 2.1520245281187154], [41.405593288200194, 2.152972051462695], [41.40758628409939, 2.1528237975449396], [41.406902501953056, 2.1529691550561605], [41.40609955452806, 2.1494591034279606], [41.4061716435809, 2.1499562009186275], [41.40822643664389, 2.1499577863439026], [41.406827658632004, 2.1516135024608802], [41.40765816838085, 2.1518998723309655], [41.405593990380574, 2.1498176167881664], [41.40795913971757, 2.152791135033763], [41.40739843171946, 2.1530122880138096], [41.4056323893208, 2.1498877385178785], [41.40822918101179, 2.150194264289694], [41.408565106134354, 2.150190699130879], [41.40697069474083, 2.1508519691156995], [41.40576930598777, 2.1495931109721482], [41.405974970743145, 2.1521683301400043], [41.40595546778112, 2.1500088977967247], [41.40669299957717, 2.1520915436416623], [41.4057804298561, 2.150534451142126], [41.40742780903431, 2.1512652054940142], [41.40596207090792, 2.1514942944496758], [41.40685802596177, 2.15074616998031], [41.406757361781416, 2.1522331469713856], [41.40922951362684, 2.1500255821778103], [41.40616764567058, 2.153209054209534], [41.40832521890961, 2.152318373389824], [41.401692363977595, 2.1625115386512825], [41.4015889136725, 2.160176480758623], [41.40227850317439, 2.1616141881365833], [41.40125445036982, 2.162498757204996], [41.40064375196943, 2.16254027490165], [41.401458291803735, 2.1604388831419135], [41.400965195878086, 2.1595264668616965], [41.402134113146055, 2.162636942722862], [41.40206235309173, 2.160356381948657], [41.40205792013363, 2.1596179963640294], [41.39979374234992, 2.159157958788122], [41.400037653612614, 2.1622382703103464], [41.40353096479848, 2.1601332463717355], [41.40318349875727, 2.1594019951852386], [41.40041715814241, 2.16088549020131], [41.40186418567178, 2.1623965302531474], [41.40094433164971, 2.1601627875332774], [41.4033795406082, 2.1606007973889154], [41.403110676328396, 2.162013410171255], [41.40024995100698, 2.159780814204227], [41.40365737463012, 2.16038158714599], [41.40268759712093, 2.1589576575235143], [41.40002627169764, 2.161457251994103], [41.40007087807684, 2.1621105224688457], [41.40261735731224, 2.162731528071257], [41.40281521666265, 2.1588294726115302], [41.400194617976894, 2.1622804479586204], [41.40051729972471, 2.16244631824359], [41.400430148211164, 2.162373448546154], [41.402375014239524, 2.1621306192673124], [41.402545031433924, 2.161149687946661], [41.40302588461524, 2.159476331102117], [41.40304288421597, 2.15952430377338], [41.402914706041656, 2.1606001058533955], [41.40198481815594, 2.159155383737232], [41.40173449038666, 2.1591401759899074], [41.39976118291463, 2.160825989362543], [41.4003044065289, 2.1597774896236483], [41.40199519371632, 2.159181654094821], [41.40143413456022, 2.1607328476568908], [41.40305360973829, 2.159860345823285], [41.40029625820713, 2.1627112207271013], [41.40066860841116, 2.159023385017818], [41.402529075370076, 2.160431015282612], [41.40337740392873, 2.161596792658619], [41.40313689507681, 2.160929990630885], [41.40353705815204, 2.162700959933349], [41.40152496774689, 2.161729195598278], [41.40180319039955, 2.1603357873002254], [41.40246215941571, 2.160656553363525], [41.400621630726874, 2.1605495356858815], [41.402062922134284, 2.162201868118134], [41.403587919155406, 2.160637020921058], [41.40264269351178, 2.162118907579622], [41.399943752455314, 2.160240744416638], [41.40364732993185, 2.161522834140067], [41.402326457517496, 2.1619821877096106], [41.40215623072723, 2.161473018569561], [41.402618721650065, 2.1604441683024938], [41.402725947938066, 2.16204092400555], [41.40026566793471, 2.158916310856001], [41.403374202073564, 2.1591536218236165], [41.40016457926327, 2.1618229180335193], [41.40338021529842, 2.1603233600066223], [41.40228260819478, 2.1599262689496563], [41.402732240806465, 2.160460065644676], [41.40154629189065, 2.158897402791057], [41.40253240537602, 2.1590131336354914], [41.40050680298047, 2.1605068192384036], [41.40054949468522, 2.162159616204635], [41.40271711732323, 2.159910927309682], [41.39979010644024, 2.1619257725337273], [41.400275587694004, 2.1594924738836645], [41.39982779814061, 2.1598967955152024], [41.399950723235136, 2.1593093824455987], [41.402976104814236, 2.1598971555700297], [41.40308803666979, 2.161152230209658], [41.403672596146706, 2.159555497677951], [41.40237290813808, 2.1601449400886263], [41.400125424041654, 2.16097163380595], [41.40143821929778, 2.161048011980262], [41.40104031397159, 2.1600172323596687], [41.40264361020152, 2.162368438653974], [41.403024299362976, 2.1603396499977285], [41.40014204419718, 2.159902161485765], [41.40252828692287, 2.162385487478122], [41.39983134635002, 2.1590671793309344], [41.40217306878345, 2.159292516122339], [41.40121557035502, 2.1622879879893877], [41.402914301663735, 2.1590955495017483], [41.40187442586569, 2.1606483428974195], [41.40357522647546, 2.1611864911852883], [41.400873394809146, 2.1602152414266254], [41.40239077096485, 2.162327143455709], [41.40208279794269, 2.1622652488801344], [41.40307859712762, 2.1621202538212065], [41.40013243850288, 2.1591237309589144], [41.40155640061185, 2.159340111085743], [41.40127061946413, 2.1598306456300733], [41.4005913662181, 2.1625659921795273], [41.38393028926368, 2.1518102204342404], [41.38455808172941, 2.151353231889968], [41.3858033270716, 2.1513730393938446], [41.38387568233463, 2.1517279929538025], [41.38395162382429, 2.1505839628674797], [41.384011456445386, 2.151536217530365], [41.38498460933166, 2.1509045940883564], [41.3848387613741, 2.149921810775092], [41.3855407606747, 2.150605607622149], [41.384767035637324, 2.151190219230952], [41.38395975720587, 2.1508858889001083], [41.38507556884559, 2.150269492942123], [41.384381037097135, 2.1503459328002346], [41.385179089963565, 2.1507073333673556], [41.38394305068935, 2.151742852619631], [41.385218938664735, 2.1517703874517284], [41.38578932599804, 2.1512314072126277], [41.38422576351659, 2.1497479055045603], [41.38511395983895, 2.150175400399138], [41.38421790249894, 2.151328270256411], [41.38424553831361, 2.1500846101764304], [41.3838246355796, 2.150224590328858], [41.384856933660565, 2.151533181783975], [41.38605450755226, 2.151804788936643], [41.384345016312764, 2.1503316607282557], [41.384494452247615, 2.1498315293860597], [41.38538942301937, 2.1509968864293487], [41.38398991021511, 2.1505139403178317], [41.383970282980044, 2.1499358862402955], [41.385250902147874, 2.1519500372146734], [41.38522097050899, 2.1499308600737366], [41.38594024789524, 2.1503360242453513], [41.385569126637996, 2.1496786965142287], [41.38607792848373, 2.150446672102504], [41.385189714738196, 2.1512760297605795], [41.385053336099745, 2.1517427013165147], [41.38504489787627, 2.1496769681141936], [41.38381471977092, 2.1504571237530348], [41.38407896601705, 2.151450881037276], [41.38438605276072, 2.151333730277656], [41.38404757281374, 2.1518497324136985], [41.38450546083624, 2.151136425991354], [41.3845604274104, 2.1514231311454677], [41.38514103258125, 2.1517535402667227], [41.38473382330806, 2.150989748744724], [41.385043188451576, 2.1507670594632122], [41.38408178411824, 2.149725009063788], [41.384591549677666, 2.15057088682769], [41.38452029809895, 2.1507265494720427], [41.38596655523638, 2.150789195243689], [41.386107328908, 2.1519130352886995], [41.38506430047135, 2.151046859912803], [41.38557863109135, 2.1516137820108816], [41.38451973754507, 2.1500888449888893], [41.38474100718659, 2.150033915351992], [41.385654122518716, 2.1501728243604443], [41.3850389355734, 2.151943360787425], [41.385503810471945, 2.1507013166168125], [41.38410300570509, 2.1503408494022964], [41.38485678238364, 2.149782147606375], [41.385262350019815, 2.151892543628514], [41.38579895891355, 2.149870522685095], [41.38439636703485, 2.1513176851018954], [41.384182450943904, 2.150763979937224], [41.385034535583365, 2.150492197540509], [41.3842926116573, 2.149652470639626], [41.385917116826406, 2.151190741889649], [41.38538699485498, 2.150371208494978], [41.383830433246985, 2.1511641755964437], [41.383927179152565, 2.1515104629051836], [41.38453302074247, 2.150859347845946], [41.38501150628814, 2.1510100132777343], [41.38387804054819, 2.1509283363949385], [41.38418465565478, 2.1500259546466167], [41.38508136414244, 2.151956016956066], [41.38416538348963, 2.1501729883633924], [41.38448238619017, 2.1505433216636693], [41.38462508366733, 2.151233251056251], [41.38400366883734, 2.1498287663619418], [41.38406446823931, 2.1516767920553566], [41.38574497527331, 2.151803587126755], [41.38522775427808, 2.151381204024877], [41.38453151242592, 2.1503395286417386], [41.39532783429778, 2.1576782630143083], [41.39482262342383, 2.1579860675890385], [41.39578678474565, 2.1568835287682386], [41.39492624608169, 2.1574111914855116], [41.39492413305222, 2.1577315021939207], [41.395904940752196, 2.157498579430298], [41.395063848929865, 2.1569286451214436], [41.3950111298865, 2.1574561423367666], [41.39490415348241, 2.157502773312913], [41.394941768955086, 2.1571303257615995], [41.394934550336274, 2.1569512622934237], [41.3947636773067, 2.157048739217961], [41.3947956264771, 2.1578984898550093], [41.394895259726326, 2.1568963717296854], [41.39535175483755, 2.157752549352411], [41.395430713679275, 2.1569755413370135], [41.39494738694469, 2.1567163853852964], [41.39513823152707, 2.1573463692978376], [41.395589754735425, 2.1578343755672647], [41.39577623326167, 2.1567807758720297], [41.395127715170894, 2.1569184395644756], [41.39477017687174, 2.1568909842676507], [41.39512107894621, 2.1567712987232612], [41.39473037157458, 2.156855355187201], [41.3955220200431, 2.1567558616413103], [41.39498903430368, 2.157625735069528], [41.3959084438074, 2.1576322356689133], [41.39520067584723, 2.1567291265478468], [41.395152034460196, 2.15744304918096], [41.39520413766349, 2.156887130299096], [41.39598742455084, 2.157229499982962], [41.39550900480644, 2.1567827869345657], [41.39503420617952, 2.157758002505526], [41.39582405488116, 2.157506167804096], [41.395391236219645, 2.1571699604189574], [41.39585780743692, 2.1570853382496167], [41.39479662670574, 2.1569313774743857], [41.39539418156373, 2.1578842539258916], [41.395435713354956, 2.1575227828818435], [41.395962058186704, 2.1576426711199583], [41.39587417637631, 2.156824934804435], [41.39517920716624, 2.1574354968749825], [41.39552641439873, 2.157063703204839], [41.39546395654558, 2.1568295468587753], [41.39517560785497, 2.157993677233774], [41.3947789837891, 2.157178915162419], [41.39541099388491, 2.1570641317769663], [41.39484120116789, 2.157693099670216], [41.39598385104655, 2.1576195674456073], [41.39478592623573, 2.1568232631696795], [41.39559902525574, 2.15764438552359], [41.39476767963173, 2.1573414654871077], [41.3950515748566, 2.157656507997112], [41.395253068416125, 2.156922257444084], [41.39581709385949, 2.1573900089718885], [41.39513027478309, 2.1578604626739044], [41.39595059855193, 2.1569000888524066], [41.395915671317525, 2.1579208696186565], [41.39578375214913, 2.1569990068023084], [41.39573077424698, 2.157211912151458], [41.39504272437058, 2.1576965696677695], [41.39596835375692, 2.157187899729404], [41.395004723858015, 2.1575342061291556], [41.39501094673379, 2.1569259232447164], [41.395528316533046, 2.156813656611792], [41.39531122948785, 2.157639257986013], [41.394734835833866, 2.1569902494580924], [41.39500286419651, 2.1570981760599537], [41.384997896721565, 2.15271967020053], [41.383884918297746, 2.1538982088269787], [41.38335395899965, 2.1546041250565455], [41.38361356181728, 2.1543978327181956], [41.38395056201935, 2.153040188542278], [41.383199871502704, 2.153447804223878], [41.38395837524731, 2.1537191145850674], [41.382975285528225, 2.152637176490939], [41.38448830383702, 2.1546237365740404], [41.38493434212314, 2.154241076244234], [41.38408659982919, 2.153669896418885], [41.38315323246468, 2.1526576323847277], [41.384471880400376, 2.1537503518977266], [41.38410471814908, 2.154425809739155], [41.38512173568216, 2.152568508874874], [41.384173777092506, 2.153657306292062], [41.38405173023132, 2.1527864871367446], [41.38490658746542, 2.154176967143779], [41.3843438355811, 2.1525327177338447], [41.38364838731042, 2.154227827536262], [41.38340437154412, 2.153986629511136], [41.38352113489181, 2.1539149705559466], [41.3830547360433, 2.1542549833619753], [41.38513835188892, 2.1534751988709337], [41.38428854122369, 2.154719734489195], [41.384109054127045, 2.153108202925007], [41.38460040822372, 2.1525829460504706], [41.38308973094531, 2.153462271196155], [41.38472419033926, 2.15298959666566], [41.383324596611864, 2.154426378326719], [41.38397525514562, 2.152886589670657], [41.38477368620151, 2.1533000097174777], [41.38397478163354, 2.153321877212769], [41.3844745688744, 2.1536760155789847], [41.384346665166575, 2.1540340427906703], [41.38347697833036, 2.1528170338139683], [41.384268279577725, 2.154267406700565], [41.38305950879386, 2.1537603490321184], [41.383556993592926, 2.1529909731282704], [41.38305168742027, 2.152799370946888], [41.38422493785108, 2.1526838694447803], [41.38412783712101, 2.1527818181300966], [41.38489691270322, 2.154075862492245], [41.384304333417006, 2.154329425647938], [41.38424370499685, 2.1528533355869266], [41.38517299208012, 2.1540227482495045], [41.38449433226019, 2.153613375005953], [41.38294572868408, 2.1534254866201907], [41.38372328745217, 2.15389081847049], [41.38321785122016, 2.154458516906962], [41.38470837850217, 2.152882077233971], [41.383013031253604, 2.1535001052699028], [41.384271471922204, 2.154374591770233], [41.38515103647622, 2.152564247318627], [41.38514938436187, 2.152882690935731], [41.38495934837687, 2.1527490319551603], [41.38497632665914, 2.154286696676409], [41.383722446545754, 2.1526505137318237], [41.38391030151948, 2.1541942140120414], [41.383262444534104, 2.153373823468045], [41.38357192600622, 2.152691079772575], [41.38472969907779, 2.1538957103356897], [41.38492438761373, 2.1545824880136983], [41.38472817530172, 2.1543830942170406], [41.3831768121001, 2.1545783644052747], [41.382999549875684, 2.1544853279964573], [41.38357961915313, 2.1536588065043296], [41.383247921556354, 2.1533287439670135], [41.38500964171338, 2.154339854589544], [41.384940170050875, 2.153695171432588], [41.38429179737036, 2.1540018696854126], [41.38333086182757, 2.154676096003429], [41.383583520700995, 2.1541635522489533], [41.38416394630566, 2.154495570213682], [41.383155371670284, 2.1544118634758176], [41.38483388055928, 2.1538556353667895], [41.38456373619284, 2.1530681289712232], [41.3838453865146, 2.1527098652744097], [41.38386719880285, 2.153048786431952], [41.3840932997208, 2.1533139792186105], [41.384055759204685, 2.153163486530541], [41.38486813694107, 2.1540170992992387], [41.38348077803072, 2.148128275688064], [41.38375022672473, 2.1486355183534354], [41.38240637980803, 2.1489933158906784], [41.382629643822646, 2.1481552331094798], [41.38288286988423, 2.1494531716141076], [41.383763283903676, 2.1493487817845427], [41.383874497340365, 2.148378609440122], [41.38293355357173, 2.14770442094871], [41.383533642576275, 2.147630104743727], [41.383410609227305, 2.149125868997377], [41.384118987359535, 2.147708611792642], [41.38245506000408, 2.1492041075879538], [41.38378231321412, 2.1492058591516754], [41.382258746196314, 2.1479033143860664], [41.383516545347156, 2.1495966560898783], [41.38335262373602, 2.147549416119961], [41.38297439502167, 2.147908514160222], [41.38259766519415, 2.149298305364932], [41.38390567237961, 2.1495955367005912], [41.38299875489475, 2.1484091497390403], [41.3829533599069, 2.1489799491358723], [41.382425065094694, 2.1495914618836722], [41.38213991660848, 2.147885742568186], [41.382400175126186, 2.148962180746362], [41.38297828593865, 2.1493661546799485], [41.38211249424851, 2.1489897673586245], [41.384174537739064, 2.1489521263850073], [41.382811758094306, 2.1486619748318367], [41.38375078051241, 2.148338799857859], [41.38287378834145, 2.149495709297374], [41.38265622637084, 2.1475752361971043], [41.38372020713633, 2.148113838389707], [41.3822058494135, 2.1474252164152117], [41.383734423995165, 2.148788519927032], [41.38393882164276, 2.1491911908350136], [41.38245848628382, 2.149220089606582], [41.38355964433477, 2.1479763881272502], [41.38346369460033, 2.149230468712136], [41.38295815466131, 2.1486061474175076], [41.38285144779709, 2.147639070025329], [41.38330213420787, 2.148228769130548], [41.38301979583528, 2.148652664421473], [41.38246911252586, 2.1484906679815463], [41.38299383115146, 2.148696151056272], [41.38362301556757, 2.1476622548754163], [41.38285185627066, 2.149242752060289], [41.3837249528373, 2.1494947689746047], [41.3836845928257, 2.1480660327740173], [41.38350897120191, 2.1475844589732778], [41.38258198690647, 2.148745843998723], [41.38401527848203, 2.1475240356486927], [41.38425592025659, 2.147576472877046], [41.383294114193326, 2.1475838452688665], [41.38341501804786, 2.148744331969941], [41.38334877671011, 2.147660494354395], [41.38282362022673, 2.1492244225672064], [41.38372313308362, 2.1485034085735912], [41.38335360604296, 2.149121823318764], [41.38280163433562, 2.1482690053550644], [41.382443515975176, 2.1495277197050866], [41.382249349726685, 2.148812003608653], [41.382598029262425, 2.1487072480725824], [41.38391692836138, 2.1480833145275176], [41.38389380852462, 2.148955919897333], [41.382149729416454, 2.1486192112771683], [41.384127503200496, 2.1484591021242765], [41.38389248270652, 2.1478941858228824], [41.383961286664096, 2.1478019822612473], [41.384263985844846, 2.147485580403581], [41.38337161397756, 2.1485891119224485], [41.38222036900825, 2.149575903316754], [41.38247588337667, 2.148686956320442], [41.38305675654744, 2.148917290771397], [41.38266424373938, 2.1477481488372283], [41.38264728315513, 2.1485246476881388], [41.38266640333685, 2.147719438317053], [41.38285225571949, 2.1493819138193944], [41.38382196552959, 2.1494825694814597], [41.38380735727004, 2.148567501089209], [41.383677333784455, 2.1490301310350826], [41.3822730240953, 2.147906745403656], [41.38251731776526, 2.148049211830514], [41.387684283687555, 2.1517877593933057], [41.3863595784784, 2.1492716871758533], [41.38812011540935, 2.1520483322555695], [41.387301298295796, 2.151003719780469], [41.38871007922742, 2.151331727593681], [41.38925037354475, 2.148956850263399], [41.387210241829045, 2.1513033379100888], [41.38664408591056, 2.151446061904314], [41.38781232800874, 2.1513450941866084], [41.387709163693636, 2.1520662399344204], [41.38920496521987, 2.1500332472893904], [41.388134509135384, 2.1483542571595042], [41.38666851092061, 2.1511996429853086], [41.386549041448134, 2.148532688042867], [41.38714430047806, 2.150976751176303], [41.38795372578085, 2.149825001188147], [41.389466260773716, 2.1502993193593785], [41.387422792286785, 2.1518352660557936], [41.386256375072804, 2.150844449309474], [41.38960432051394, 2.1495504545945683], [41.38729108702476, 2.1484138860507302], [41.38688159423733, 2.1516539243189414], [41.38873409424344, 2.1516718027394917], [41.38774552185618, 2.148893773711933], [41.38660418092731, 2.1515848561653654], [41.387440495432884, 2.148682280157276], [41.38896265817389, 2.1505780128050547], [41.38645117187577, 2.1495978290678197], [41.389545069004534, 2.1488835692968458], [41.38793123738561, 2.149533801629975], [41.38935068181464, 2.150572833797786], [41.38895892340069, 2.151905625265586], [41.38718540460314, 2.148895829063947], [41.38684377208543, 2.1516074936029024], [41.38637946229026, 2.1491769908540514], [41.387646316541336, 2.15069620305469], [41.38899857544307, 2.1511756962446205], [41.38622466122569, 2.151208199582778], [41.38622758402493, 2.149601634551385], [41.38714922087815, 2.150967402873025], [41.386717446632666, 2.148870665944493], [41.38924933395708, 2.150469046638094], [41.38894001957235, 2.15065252485759], [41.38977783510647, 2.1492361827306294], [41.38693632145782, 2.1484168485212694], [41.3883399945847, 2.149965090790962], [41.38734612021046, 2.1492850688099745], [41.38878846256289, 2.1483837614692995], [41.38952550144266, 2.1488924706587516], [41.38975591055159, 2.150407415568574], [41.38789074587904, 2.14927719159138], [41.38774077795059, 2.149646427648081], [41.386389542584844, 2.151682876396899], [41.38831327143455, 2.1484060794842748], [41.38839896079798, 2.151562295249873], [41.3885859880525, 2.15067640828312], [41.38705253736624, 2.148444236152823], [41.38665178046837, 2.151782931480179], [41.38672572319349, 2.1510944972581396], [41.387847916138654, 2.148498571303741], [41.386026536350414, 2.151547941562474], [41.38754532564917, 2.148701087744358], [41.38854458291911, 2.150666041028397], [41.386528877799606, 2.1494511593589034], [41.388177633102686, 2.1513718076728146], [41.38765545011077, 2.148940189369788], [41.3898381793297, 2.1482628767191994], [41.38735262098322, 2.14875117965881], [41.38878983906163, 2.1484675601638985], [41.388553107007105, 2.1503045304249238], [41.389139118893965, 2.1509241241825134], [41.38782582485769, 2.15120268309401], [41.38758746580195, 2.1493568180739078], [41.38867754568362, 2.150114689095235], [41.38986913554144, 2.150573369557211], [41.38964679753313, 2.1487289782104844], [41.3863130590335, 2.1486051447238275], [41.387777898292164, 2.151779935603198], [41.38838432443998, 2.149345133268114], [41.38920603021619, 2.1490486067192074], [41.38752208082132, 2.150317005891797], [41.38739539881854, 2.1484752597300223], [41.3885624471688, 2.151842967798945], [41.38798361093047, 2.15158175060003], [41.38607614891181, 2.150276353274533], [41.38884397838541, 2.1500236052160355], [41.38915361616459, 2.15105177165963], [41.387327087357946, 2.1519730542241673], [41.387927963023344, 2.150993314761709], [41.38700064561571, 2.1491282987825606], [41.386972545339084, 2.151665303018579], [41.38607162620114, 2.151053092719308], [41.38809037555289, 2.150660842381524], [41.38982904125283, 2.1485738183417453], [41.38927220158052, 2.151770707109309], [41.38713704205798, 2.1489946835085116], [41.38659078742794, 2.151969914909502], [41.38932064520307, 2.14963575089561], [41.405089963690635, 2.1624431522918304], [41.405545555631065, 2.1619697879066075], [41.405890012129454, 2.1619689413589263], [41.40577345974004, 2.1624357364140683], [41.40554979826474, 2.162678355250147], [41.40582265966811, 2.1617515752547805], [41.40584201592207, 2.1629430035823556], [41.4051842805721, 2.1628267293354386], [41.40593401596514, 2.1628086539161653], [41.40610920641195, 2.16267591733439], [41.40509107214006, 2.16218030989481], [41.4052121932538, 2.162220784284645], [41.406105982730274, 2.162303284211298], [41.40519988090159, 2.1621463283135465], [41.4060339505887, 2.1618176519170844], [41.4054978694399, 2.161754418735891], [41.40547189198935, 2.162200006917457], [41.40533450079913, 2.1618737510587795], [41.40539816744165, 2.1621183421614782], [41.40510943906011, 2.16187362171336], [41.405748725955796, 2.1624554142877894], [41.40548512270079, 2.162800434624612], [41.405599629189695, 2.162206693487306], [41.40596657671265, 2.1622984959148526], [41.4061432039774, 2.1618776865277463], [41.405033748147616, 2.1629460712262727], [41.40565774876997, 2.1622731823350994], [41.405547864242266, 2.1625719048434418], [41.40588678594056, 2.1622305820962424], [41.40575549182293, 2.162388362341375], [41.405823227773716, 2.162417841330207], [41.40614307634671, 2.161796912452633], [41.405262749358535, 2.162113654756958], [41.40605120719754, 2.162294038490071], [41.405186093231414, 2.1618872521762844], [41.405971208292, 2.1619455430450123], [41.40529207865931, 2.1626726871214212], [41.406120515849594, 2.1617987745693377], [41.40553689757118, 2.1621654877353054], [41.40588163253869, 2.162072612569354], [41.4061759121927, 2.161965958531352], [41.40543323550306, 2.161974640590162], [41.40564638658725, 2.1623522110136424], [41.40520205704773, 2.162751479716668], [41.405167665348245, 2.16261900960845], [41.406061677447305, 2.161890028856988], [41.405174018123844, 2.1620664691160276], [41.40604237319169, 2.162392384168631], [41.40616178320376, 2.1620910564081672], [41.40588558639663, 2.162822879263733], [41.405213782301, 2.1624233499153127], [41.405625937519254, 2.1619635267328907], [41.40619521747, 2.1629458051426687], [41.405799397546915, 2.1622532341988774], [41.404995786552924, 2.1621326962502248], [41.40550296086867, 2.16224621842213], [41.40582486343713, 2.1623802205086298], [41.405067241464586, 2.1624778606769217], [41.40538360059803, 2.1625589501545126], [41.40553164610022, 2.16294767505531], [41.40553646866778, 2.1624884302679894], [41.40520768103388, 2.1628207572154543], [41.40497062369509, 2.1619869272924555], [41.405492640479245, 2.1618027759725713], [41.40555875458569, 2.161838600656754], [41.406166891523455, 2.162503598022723], [41.405261349751854, 2.162491980690358], [41.405694056191656, 2.1628543642916487], [41.40108934255754, 2.1638860101263417], [41.40242299011954, 2.164701786902239], [41.40356750568431, 2.164383337947326], [41.40350214132941, 2.1656576336207243], [41.40292097769716, 2.1651180078696886], [41.40115526593091, 2.164138179634494], [41.40273540537094, 2.165083484612163], [41.40242830457865, 2.1648091573710713], [41.403281251522735, 2.165842306942827], [41.401328908289535, 2.163498904816967], [41.400847711606225, 2.16491701520013], [41.40202012396602, 2.1646276442509067], [41.40346894345011, 2.1660965389777935], [41.40300007577445, 2.1663145086393105], [41.40290722543817, 2.165739409548243], [41.40103955725934, 2.1649420355650393], [41.40095467093675, 2.164198032977687], [41.40183839595398, 2.166177018254002], [41.402598019426634, 2.163503393439449], [41.40219901409379, 2.166426041932876], [41.40187535953329, 2.1648766520108906], [41.401873135089446, 2.165110924078689], [41.40261528398456, 2.1642831865946994], [41.40261563674831, 2.16413823277103], [41.40106156456939, 2.165919758142986], [41.4021477819589, 2.1648602823718437], [41.401345352276394, 2.164189388450132], [41.40148848448519, 2.165176694270238], [41.40246635764607, 2.163937055038512], [41.40263405059047, 2.1654085958746028], [41.40296575325895, 2.166404358832419], [41.401981259858864, 2.164540352954386], [41.40181504215631, 2.1661735751377886], [41.40277756655422, 2.165376636925188], [41.40253636698965, 2.165094992718711], [41.40227511787097, 2.166340779229924], [41.40316012726436, 2.1637724098177644], [41.401867706859015, 2.1661284123962847], [41.402453541003936, 2.165842292921682], [41.40283165151643, 2.16546383301113], [41.401621627631755, 2.1644395110091743], [41.40099329445602, 2.1641180028302536], [41.401184548476266, 2.1636114316232926], [41.40131069275259, 2.165814278365023], [41.40083679162514, 2.165139695132233], [41.402539755631125, 2.1660599624381778], [41.40144079781553, 2.1659399927643297], [41.40375036745629, 2.164594445237823], [41.4025523891912, 2.1645867412083177], [41.403716273396746, 2.165342627632336], [41.40286291607262, 2.1656311280593203], [41.40365438073718, 2.1636781374585827], [41.40347707464566, 2.1657325397321983], [41.40102458467217, 2.1646525958000336], [41.40269338188512, 2.1647198139789747], [41.40224582135592, 2.1655577397928107], [41.40142786729042, 2.164431635899843], [41.40105123332717, 2.1647604325538077], [41.40170230528706, 2.1654641790907196], [41.40340227250802, 2.1661863703271975], [41.40108911275365, 2.1642056022455316], [41.40281790466956, 2.1662690203850494], [41.400860436613115, 2.1661604573770252], [41.40174616267218, 2.1650135692485843], [41.4033775301201, 2.1652885262193258], [41.402293537341116, 2.1645535787766645], [41.401873620767645, 2.163691157546173], [41.40239670511588, 2.165061475426948], [41.40379724715796, 2.1648480317296785], [41.402856981009656, 2.165572240659971], [41.40085056610504, 2.164631733078535], [41.403708191599776, 2.1647107070870244], [41.40093733851351, 2.1653159787235476], [41.40124557676544, 2.1662966247900757], [41.40286606554497, 2.1647054591020245], [41.403755464657785, 2.1641353280292184], [41.40153194254201, 2.1650784869882496], [41.40166178576177, 2.1635633417449265], [41.4023623122332, 2.1661942310578106], [41.40161789385782, 2.1650079403900486], [41.40365941696122, 2.1656314136110133], [41.402125058960465, 2.165324173951647], [41.40093226457452, 2.1648788478679575], [41.40146131000618, 2.165224536804995], [41.403780872150804, 2.165841302710562], [41.40338841552764, 2.1652408444880766], [41.401884317430124, 2.164721531628147], [41.40229948531538, 2.1662837220669204], [41.402508446391195, 2.1647536773617513], [41.40233520161491, 2.1640169095481547], [41.385631680637, 2.15458521791634], [41.38612884994634, 2.1547142212551273], [41.38645703490063, 2.154880723292298], [41.384131582184786, 2.1561803976864424], [41.385949985034955, 2.1543617324617177], [41.38485062458755, 2.155985203355961], [41.38461300178085, 2.15561351481663], [41.38473605546653, 2.1553891455038388], [41.38513806625, 2.1535803888390816], [41.38600888429466, 2.1538782285505533], [41.3858157779398, 2.1548116082254283], [41.38415584898839, 2.1544787705802944], [41.384133164153575, 2.154899198887689], [41.38451809960519, 2.1541596856327305], [41.385210477438704, 2.1554128794290364], [41.385765493811164, 2.155284294697209], [41.385936723098325, 2.1556310825936316], [41.38469212556021, 2.1550029024027824], [41.38429683413784, 2.154361767220602], [41.38544546057901, 2.153787197750198], [41.38416865330433, 2.1542873838516665], [41.3856809997659, 2.156065592898019], [41.38615610616707, 2.1556495482232343], [41.38581827887302, 2.1540222019399504], [41.38544688887032, 2.155156484374986], [41.384451465865, 2.154875318886347], [41.38668202163823, 2.155157442265244], [41.38663380718313, 2.1548758595662756], [41.38597442842512, 2.1555218004394745], [41.38469262262564, 2.1546410044606588], [41.386076732803296, 2.154869769729852], [41.38477238463497, 2.1548841290260494], [41.38521961681386, 2.1561691274467605], [41.3862134600769, 2.154325703302971], [41.384730869554396, 2.1554950153015806], [41.38652391903616, 2.1549091950222863], [41.384144597363466, 2.155283035053862], [41.38630556694017, 2.1543028668806157], [41.386004098288865, 2.1550881934575266], [41.38520724294867, 2.1553390948352247], [41.38455758996167, 2.154273604932883], [41.386343419847485, 2.1557853110540623], [41.38532497040522, 2.1546803458351973], [41.3863067063313, 2.154414812929439], [41.38445055526004, 2.155303677652091], [41.38633397118568, 2.155418073136225], [41.384716107225515, 2.155369738652125], [41.38407458092276, 2.1560127424977593], [41.386464012492546, 2.1553890430571934], [41.384493771772874, 2.153750635024193], [41.38525908832407, 2.153938196660829], [41.38415052522266, 2.154057593961519], [41.38662007364596, 2.1560497223420994], [41.38408487409436, 2.153864384357558], [41.38454229060136, 2.1553279255469495], [41.38464583158076, 2.154332334853333], [41.38654099701304, 2.154001218030851], [41.3860225751608, 2.15533717931182], [41.38466404684529, 2.1554073739134596], [41.38511631672342, 2.15430222631893], [41.385628884448806, 2.1541460871811946], [41.385843387679635, 2.154672541790621], [41.384745639628726, 2.1559313011344674], [41.38445303008433, 2.153603431783226], [41.38525874367262, 2.15383608414138], [41.38415245578681, 2.1559543374510026], [41.385405935471695, 2.1554741500791055], [41.38562124462412, 2.1545444947747523], [41.38594229645577, 2.155676358035085], [41.38467935755983, 2.155625140365478], [41.38449006103142, 2.154799668883813], [41.385468993098364, 2.154010750844251], [41.38656384932567, 2.1556991645131705], [41.38416543028207, 2.155217376370775], [41.38534463670532, 2.153631141125586], [41.384691511812534, 2.154850174868302], [41.38576486279982, 2.154537018188586], [41.38606524764228, 2.1536808928195073], [41.385399716407406, 2.1560158227507964], [41.3859628248566, 2.1543290394683687], [41.38424953748949, 2.155379214449925], [41.38476719939723, 2.154260782324928], [41.38577725805496, 2.155014103629798], [41.386463797187496, 2.1549462739973495], [41.38545755744255, 2.1555318366677354], [41.385387743467, 2.155169372604818], [41.40417905555304, 2.1660901473371763], [41.40577696673579, 2.167116278794125], [41.40606743470728, 2.1656358815781322], [41.40416088626925, 2.1647645225928107], [41.403820557660524, 2.16500332217701], [41.403812245927526, 2.1655048583115017], [41.40595714727225, 2.16520480649687], [41.40641656361575, 2.165682004195314], [41.40632641737219, 2.1669140616167275], [41.40420258718072, 2.1661525000268593], [41.40635147141503, 2.165014292246319], [41.40627732075876, 2.1656191045614306], [41.406293563458235, 2.164894270448981], [41.40407760946411, 2.167603546714501], [41.404684212488526, 2.165630543185883], [41.40651692455149, 2.1654922501636795], [41.40403321728363, 2.166607053965583], [41.404786794694125, 2.164869295225643], [41.404878385809006, 2.1657300566267397], [41.40393219860822, 2.1660223260610234], [41.4068764517892, 2.1660648949069956], [41.40446383496805, 2.1646160075164156], [41.40623713364174, 2.1677678397655766], [41.40470110074484, 2.16589956795757], [41.40544269306792, 2.1667046175116593], [41.404154435184296, 2.165324419442971], [41.405119234348625, 2.1656300702267353], [41.40506468866634, 2.16662976561736], [41.40519237898422, 2.16646269621438], [41.40534750747971, 2.167006718961366], [41.4053753961116, 2.1655148262111066], [41.40663345374183, 2.1656101128807257], [41.40563004983071, 2.164996393092457], [41.40453539187263, 2.165569575442283], [41.40489052554624, 2.16512200746976], [41.405602248187115, 2.1656115310200956], [41.40451018520382, 2.166313932392314], [41.404874094770186, 2.16716729486256], [41.406297777379734, 2.1663627817529982], [41.40529105577293, 2.1654652050190526], [41.4056965628644, 2.1650584326767497], [41.40563694621762, 2.1671271591498673], [41.405838405133345, 2.1651967697813133], [41.40435633209634, 2.16658589430116], [41.40419993779646, 2.166036417153116], [41.40523904740748, 2.1658106883898167], [41.40465675554736, 2.165550642382777], [41.406765444366606, 2.1646379816777634], [41.406386240893916, 2.166223137520266], [41.405462542322745, 2.1674490804664144], [41.40381139386361, 2.167784536055248], [41.40687713461707, 2.1671215089029277], [41.40650605638285, 2.166537273557297], [41.40693507867236, 2.167326147285945], [41.40667901576514, 2.167433479783558], [41.4064987092468, 2.1659532331029374], [41.40558548916092, 2.1651765089281496], [41.40540236598874, 2.1660344089107335], [41.4038445381034, 2.1661693938341635], [41.40704579432614, 2.166558307689344], [41.40614429430211, 2.165464844335671], [41.404162704850286, 2.1656799654737195], [41.40620759406137, 2.1650213469213337], [41.405248579418384, 2.1645853856715944], [41.40669500788951, 2.1657965918867528], [41.404482483760674, 2.1669194196723556], [41.40682236041078, 2.1676517661487122], [41.40502751231435, 2.165617660171989], [41.406873277762145, 2.1649755375690707], [41.40430558824476, 2.1668839250494116], [41.406110217792374, 2.1672397096931766], [41.40622299374222, 2.1677169337223723], [41.40577596702299, 2.166027993683403], [41.405284356205904, 2.167709714735254], [41.40641206926261, 2.167875586817233], [41.405057954513474, 2.165234525493926], [41.4062223474588, 2.1659482666104846], [41.405898621842, 2.164941121393898], [41.40669089674504, 2.16693765691508], [41.40656541188601, 2.1664706648661074], [41.40602952753223, 2.1648115028764474], [41.404808225811635, 2.1675980475845416], [41.40425195767391, 2.166283006830524], [41.404708614028465, 2.1672340193386326], [41.40642993267219, 2.1653533011084534], [41.40426983378339, 2.1664898106349897], [41.40520892636418, 2.1656391938774937], [41.40527620282444, 2.1663599890331584], [41.406599527500795, 2.1654785608279643], [41.40616908263912, 2.1663527902129465], [41.40398553355829, 2.1658226119449475], [41.40466210480678, 2.16528070292125], [41.404743347517886, 2.1670765992034338], [41.4048985221765, 2.166287214994584], [41.38254814735533, 2.14866863847914], [41.38322429528938, 2.1488810957645765], [41.382876047705075, 2.1487767537787854], [41.38237231715159, 2.148605502527378], [41.38285503465002, 2.1494042605252464], [41.3822140502345, 2.1487666674084016], [41.38208058277137, 2.1491087315201916], [41.38231723818029, 2.149490211685754], [41.382088797364645, 2.1494827485968075], [41.38186102236767, 2.148484202971778], [41.38260560560015, 2.149261922823725], [41.382370846528254, 2.1484923571959693], [41.382511451919875, 2.148644470189081], [41.38337167169368, 2.149908320771933], [41.38249395978865, 2.1489675720878676], [41.38322043222982, 2.1485754498816676], [41.38200798659444, 2.148308528204301], [41.38282940770809, 2.149354888973739], [41.38310794605773, 2.1497409229582156], [41.38261813778945, 2.149612961919043], [41.383128143509985, 2.148392906370101], [41.3818201224418, 2.1487216651412737], [41.38318423304153, 2.149813084521691], [41.38322117190838, 2.149209877525135], [41.382636561775804, 2.148457738213644], [41.38235725843532, 2.1493663555964138], [41.382748363082854, 2.1494100572455426], [41.383033720166466, 2.1486200400833635], [41.382074612611234, 2.148944221804646], [41.38222959033015, 2.1494909155891118], [41.38224559718926, 2.149650389681702], [41.381842017234426, 2.1485615923275607], [41.38231163584173, 2.1492289066303347], [41.382244973488724, 2.1491271654027617], [41.38322246110331, 2.1486491256236278], [41.38263378508901, 2.1492032579455356], [41.38194322669234, 2.149299371172041], [41.38264012606545, 2.148454934841273], [41.38298008326831, 2.1496080436778118], [41.382065474709385, 2.1497970886400775], [41.38308106640865, 2.1493586712861026], [41.38214770436972, 2.14946160720728], [41.382188563597886, 2.148481720386517], [41.38189582594846, 2.1490520565886277], [41.382054548886984, 2.1497562528380607], [41.38204488487556, 2.149608224612694], [41.382703275119944, 2.1491136687223977], [41.38196088887331, 2.149487354370374], [41.38195704290411, 2.148884657406874], [41.3833978754109, 2.1494788995250222], [41.382294677616464, 2.148280632698512], [41.38196884823975, 2.14918886322441], [41.38187275172959, 2.148629747370109], [41.38291188371354, 2.14945919381655], [41.3833671000002, 2.148303743163141], [41.38232779498556, 2.149647173544856], [41.383330484452905, 2.1491997124985645], [41.38230487431865, 2.1488587027923964], [41.38302324863577, 2.148889067175988], [41.382339650015474, 2.149249525403295], [41.382698909049196, 2.149389068474316], [41.38235820727357, 2.1496090867033613], [41.38295755140311, 2.148799654923869], [41.383389876792585, 2.1489216071919817], [41.38185904734181, 2.149789182524445], [41.38265464782773, 2.149292276800264], [41.38184680419019, 2.149922966074028], [41.3822917569168, 2.1484629966939846], [41.381858913296604, 2.148357034267884], [41.38184380400248, 2.1486289522228352], [41.38222654094439, 2.1495415916107175], [41.38192459431246, 2.1491591635305802], [41.382486242034005, 2.1485218816893483], [41.382826065099515, 2.149440719300483], [41.39962647072152, 2.164431257600184], [41.39951381538304, 2.1642486231391476], [41.40090163356251, 2.1644072630243407], [41.3987488668783, 2.1656735443641906], [41.399796630361, 2.165479042123609], [41.401207793039305, 2.1642297136696227], [41.40065507536645, 2.1647086239358724], [41.401322241121434, 2.1634785939982186], [41.39974014860265, 2.165396913385181], [41.40076559750891, 2.163776685808352], [41.39984032435599, 2.1646499969402604], [41.400369712528956, 2.1635688797728494], [41.3994661668536, 2.1631548934099953], [41.39933366364594, 2.1653444262219566], [41.39991823950899, 2.1635962586378428], [41.39879756340087, 2.16471596627846], [41.399973227398675, 2.1632758460017003], [41.39929370848679, 2.1639470450451523], [41.399361717027375, 2.163089422897069], [41.3999280042377, 2.1635146562764187], [41.3993428321388, 2.1645330144083665], [41.400312720983756, 2.1651390375254715], [41.40120856499842, 2.16372670307463], [41.39886172243486, 2.1647051260084913], [41.40073716004984, 2.164361958880773], [41.399339055882564, 2.1657820220246253], [41.39890912831167, 2.164868983908292], [41.3998597784776, 2.1633568855731533], [41.40104937068136, 2.164022917435623], [41.400217993096355, 2.1636431110127563], [41.40078412892817, 2.1638508402518353], [41.40067237298969, 2.1637213559860373], [41.401060587334435, 2.164697760159937], [41.39909921276673, 2.1654197976734877], [41.398591803431685, 2.1650272262084966], [41.399961441215346, 2.1645302757125036], [41.40078186852911, 2.1648340393645027], [41.39872154344881, 2.164530737614256], [41.39912499890283, 2.1648184079330077], [41.400537549416484, 2.1653043353772503], [41.40056531999318, 2.1631733516390383], [41.399357217855076, 2.165537950912131], [41.40104080803461, 2.1651311400169075], [41.39939094172184, 2.164092717381851], [41.398848350066736, 2.163013805718617], [41.39994433508549, 2.165230718033453], [41.40005451258134, 2.164807491311724], [41.400383269852234, 2.1637615752253385], [41.401201929300846, 2.164769436330469], [41.40040871775804, 2.1651604422244577], [41.39871176186478, 2.1633099924613277], [41.399436530800415, 2.1643437127230385], [41.39981809532722, 2.163050055014356], [41.40093212081425, 2.163807715967457], [41.40053444865632, 2.163315506308544], [41.399649631387916, 2.163775962969212], [41.40071755924969, 2.1634403323695297], [41.3995079902882, 2.164742915816057], [41.39970089581702, 2.165704144952017], [41.40083266573139, 2.1637208178507854], [41.39901215378508, 2.1649477161798956], [41.39856912203645, 2.1648566542607224], [41.39859437861789, 2.163674316241943], [41.39965356543197, 2.1654404481919265], [41.40127717458684, 2.1657714789851528], [41.39885049103126, 2.164698269154255], [41.39903506093335, 2.1649358688580476], [41.39984179810468, 2.1648564724090895], [41.40031434123379, 2.163307887679662], [41.39971910510249, 2.1641716755320424], [41.39870324946231, 2.165592469438441], [41.400523142236985, 2.165683952668133], [41.39974990694773, 2.1648642261068782], [41.399324971046504, 2.1646758112811164], [41.40011576620488, 2.163181987680229], [41.400631700301304, 2.16364996880753], [41.39999625709111, 2.165776560530389], [41.40119778255598, 2.163472071521089], [41.399721250025216, 2.1657444669717303], [41.39875774758636, 2.163319920242227], [41.39931550021463, 2.1654209017508608], [41.40070037905564, 2.1630507594599115], [41.400940602325754, 2.1646355137514255], [41.400751673159576, 2.164686825491763], [41.40103748943104, 2.163302502481609], [41.40083509676046, 2.164206638757069], [41.39971086575726, 2.1654399064740613], [41.40085568923891, 2.1654000746945434], [41.40673518908499, 2.155790840481171], [41.40697276694026, 2.155738481503105], [41.40715059656539, 2.155936684953764], [41.40713237882208, 2.1557270040872525], [41.40675229156646, 2.1559983322641183], [41.40708036510306, 2.155930024223406], [41.407112051168504, 2.1556561657482476], [41.406913380436414, 2.1555884775697947], [41.406817538278275, 2.1557528835039443], [41.4068580148668, 2.1557596088658886], [41.40713749399097, 2.155602289290291], [41.407140416851, 2.155571520013399], [41.40691244127373, 2.1555549012400865], [41.407095855535516, 2.1558829588259134], [41.406958652614435, 2.155901164490747], [41.40703147260884, 2.1560240005599454], [41.40678646981042, 2.1559694147423794], [41.40701848471708, 2.1557513180214958], [41.407048599167354, 2.1558428007302464], [41.40710681113474, 2.1557993668683735], [41.40709917135803, 2.155805980643562], [41.40712548572927, 2.1559289222921447], [41.40670437240807, 2.1559813125813867], [41.407072523971166, 2.1556162137354353], [41.40711679625486, 2.1556047273477716], [41.40678932614786, 2.155750091889102], [41.40685051976931, 2.1559398793761217], [41.40699523004775, 2.1559356230268385], [41.40703049800349, 2.1557910152433], [41.40696684305908, 2.1556554150669704], [41.40690987007487, 2.155643419375383], [41.40699878554241, 2.1556057286488794], [41.40696166448763, 2.1559540613799184], [41.40715050390937, 2.1556653958148715], [41.4068976958872, 2.1556124369908027], [41.40669723479543, 2.1558245493025527], [41.407015895145385, 2.1559719602765424], [41.40675081176753, 2.155724772387311], [41.406717603448946, 2.155580885873538], [41.406728795426154, 2.1557798250031945], [41.406719393604476, 2.1556560272632623], [41.407094821433894, 2.15571943787908], [41.40681608265166, 2.1558619059418676], [41.40681004438306, 2.1560160374692314], [41.406776322563566, 2.1559057269939315], [41.40697863413111, 2.155693609952528], [41.40711174883081, 2.1559155148853795], [41.40672097162313, 2.1559770327012933], [41.40698415012317, 2.1559389345793742], [41.40602618219207, 2.157739446649868], [41.40410862940735, 2.1572043996931525], [41.40613562766097, 2.155789041208975], [41.405794502408654, 2.155788040104794], [41.40440115064207, 2.155422175655884], [41.406509116694224, 2.157080955965585], [41.405111311942505, 2.158121416460372], [41.4047893763622, 2.157538858286197], [41.4048836940054, 2.15681475126908], [41.40570570763317, 2.1571831972623934], [41.40489327120457, 2.156636419831431], [41.40627654011375, 2.1562358827461496], [41.404776578821064, 2.1573141764476014], [41.404709669254146, 2.1560228056649233], [41.40662501211415, 2.1582535658253543], [41.406111861657024, 2.158022002025082], [41.40586255568529, 2.1573560036786454], [41.405722347979605, 2.1573354084866123], [41.40515914882407, 2.1561098865430375], [41.40668218526677, 2.1574640606594784], [41.40596139244128, 2.158199565778333], [41.40598256874246, 2.156528385753321], [41.403614365543, 2.1555188115530672], [41.406240940287404, 2.1565059411474214], [41.406551583831394, 2.157772502965189], [41.40603746648068, 2.155510393129889], [41.40429827369882, 2.158196969587896], [41.405274101436575, 2.1556494719468255], [41.404241781978556, 2.157523014588472], [41.40416901659271, 2.1580060634493172], [41.40366780548304, 2.1563278064199727], [41.4054876704487, 2.1563810649834894], [41.405587815103935, 2.157796891002201], [41.403855185863435, 2.156024256424009], [41.40666093393132, 2.157794224089135], [41.404693606839835, 2.157124900156873], [41.40660006873234, 2.157153849503203], [41.40668288108467, 2.1555351112662526], [41.403815528825774, 2.156528744945123], [41.40375240686988, 2.155842704207339], [41.40574932914748, 2.1572628362273547], [41.40588780272646, 2.1576108517646784], [41.40538103288962, 2.1553240750533402], [41.405816739004734, 2.157814798856337], [41.40655677323024, 2.1580468635276917], [41.40630420588203, 2.15591035291069], [41.40622078549099, 2.1564897923000137], [41.40571729456278, 2.158254118419331], [41.40502160812092, 2.1557875227425076], [41.40557236543272, 2.1564384803568815], [41.40666769173543, 2.155660833123917], [41.405338322659865, 2.1566592060062115], [41.40664468332631, 2.1563438160245734], [41.40519955728599, 2.1564590204094447], [41.40438271015023, 2.158041427112861], [41.404262472002614, 2.158376246355543], [41.404883631394135, 2.156267949455543], [41.40578807552663, 2.1567495098637406], [41.40415823430535, 2.1573712747740688], [41.40428866573, 2.157661289966079], [41.404360522088076, 2.1571836567608833], [41.40474852617739, 2.156867961978464], [41.404185391566806, 2.156956694008064], [41.40655093152951, 2.157604261418594], [41.40547746494894, 2.1579085995390694], [41.40652308752167, 2.156096612843799], [41.40419266875081, 2.156478377760543], [41.4041117283522, 2.157479999494025], [41.40500996840374, 2.1570849850363976], [41.40600854358823, 2.155805867819336], [41.406482591860176, 2.1571539308663437], [41.40556705191928, 2.1564060134006326], [41.40431591207355, 2.156185306076493], [41.40445685577136, 2.1582337662439834], [41.40439044914166, 2.157479270968398], [41.405782240321514, 2.1574102981454395], [41.403887256125266, 2.15602848060282], [41.40587606157102, 2.1581080850987706], [41.40597757805101, 2.157165372647806], [41.40647774674767, 2.1578924067665453], [41.40453633433409, 2.158041319265449], [41.404443404580014, 2.15750729354094], [41.40603663206524, 2.1565716807557056], [41.40629519135227, 2.15687216342982], [41.40399486113202, 2.1555098640618042], [41.40412277764645, 2.156691827082066], [41.40634827491936, 2.158179003083287], [41.40453815881258, 2.1558335423651176], [41.4054118535722, 2.1577675901808693], [41.40504198278509, 2.1579805286773475], [41.404447882176584, 2.1580062701267777], [41.40365120426445, 2.155857199122262], [41.40742906169672, 2.1637875037650027], [41.40681929055743, 2.1636412230437294], [41.406723837128965, 2.1629147798932125], [41.40739273672229, 2.1635775117096117], [41.407674689864685, 2.16340451107135], [41.40709893560802, 2.162989961081088], [41.407685891708745, 2.164056554563145], [41.40754307654729, 2.1636114213906117], [41.40722353306669, 2.1632843830252946], [41.407201572500455, 2.1629137860310204], [41.40714582669664, 2.1631762877285134], [41.40672437678781, 2.163504971211247], [41.4074702382572, 2.1640665498882345], [41.40732311858256, 2.1639730343561374], [41.407648991674805, 2.163768436522479], [41.407001767795876, 2.163553505301938], [41.407382067729536, 2.1636179818501375], [41.40766606022236, 2.1629323221272787], [41.4073646110023, 2.1634604059789813], [41.40702687062009, 2.1639835047020215], [41.40746960047074, 2.1639588649646764], [41.40707099730346, 2.1632541673696624], [41.407172349899696, 2.163067287846187], [41.4065793424523, 2.163898457216035], [41.40722566801346, 2.1637506702794327], [41.40783040978288, 2.1635593137310734], [41.40766530677695, 2.1636471337445706], [41.406556849878456, 2.163883293606792], [41.40778471592418, 2.1629664456294706], [41.40728205768974, 2.163744064247792], [41.40699568513836, 2.1636140002027506], [41.407301014676946, 2.163588990021206], [41.407558780944626, 2.1637766478146627], [41.40721575778329, 2.1639199886948517], [41.40783612490275, 2.1639424725456657], [41.40695709440104, 2.163260975012667], [41.40733896831492, 2.163971364422659], [41.40723070963183, 2.1640065445286485], [41.40663175388809, 2.1634397706691], [41.407794524194586, 2.1629472861057], [41.40774574293023, 2.1631670676201193], [41.40707659575585, 2.162881877089197], [41.40745511866462, 2.1630920240771054], [41.406570213315185, 2.1635569728425423], [41.406634162381756, 2.1630342457992993], [41.406888990600805, 2.1633821583788113], [41.407453089655924, 2.1632517569518828], [41.40683126495814, 2.1632757354173684], [41.40763474214511, 2.1631533249626855], [41.40679176310829, 2.1633352686374585], [41.406995294011395, 2.163137451778328], [41.40680441509399, 2.1630390392602084], [41.407705574591496, 2.162889519814234], [41.406546339065336, 2.163061802954172], [41.40729157166519, 2.1638547109190993], [41.40776554916198, 2.1637813553974787], [41.407293744302756, 2.163443634212646], [41.407578388442374, 2.1630368694256092], [41.407151529290815, 2.162870518326523], [41.40703844854667, 2.1632827105761874], [41.40687793461138, 2.1633776193285086], [41.407227317111165, 2.163852852242981], [41.40695547040016, 2.1638654894190443], [41.407195962455816, 2.1629742634967117], [41.40760175211571, 2.1628264588693153], [41.40655709820454, 2.1633563374498106], [41.407707399029604, 2.163155243025245], [41.40660366836762, 2.1629773066226763], [41.40679096904558, 2.163893719005033], [41.39504572330448, 2.16086738467594], [41.394963455836255, 2.1611513265214786], [41.39527345815242, 2.1610312454361957], [41.39486239297123, 2.1615507360283495], [41.39492233292668, 2.161197028632621], [41.39542875518509, 2.161655335448455], [41.39479687597, 2.161418979533565], [41.394737556031224, 2.1615230671716215], [41.395376389111355, 2.160862010571929], [41.39458417929024, 2.161156131525031], [41.39530189951037, 2.16091114682005], [41.39480499075697, 2.1614029687102896], [41.39522497565874, 2.1609878637797726], [41.39548274691765, 2.1616867833400235], [41.39462028726116, 2.161646878391801], [41.39494059273974, 2.161050616931242], [41.39459601667239, 2.1614158077536145], [41.395056852851944, 2.161014245056839], [41.39495620842572, 2.161520075919875], [41.39490358399749, 2.160994300704261], [41.39470912025045, 2.1609507017723386], [41.395061656440305, 2.1616597572101885], [41.394758484540205, 2.1611186637543067], [41.39516657372275, 2.1607943190316594], [41.394950499228074, 2.1610918332025646], [41.39531629162145, 2.1609834715413885], [41.39538757354132, 2.1607736226511824], [41.39479802186346, 2.160839991034084], [41.394599304947995, 2.1612157110808647], [41.39454454119274, 2.1611640424328358], [41.3954292426566, 2.1609801763518726], [41.39546696651826, 2.161006621828781], [41.39456110950496, 2.1614407798901207], [41.39532275829158, 2.16167462775104], [41.39505752705955, 2.161701740116863], [41.39489755128674, 2.1612361198243657], [41.394716336541514, 2.1607593673588004], [41.395291153821766, 2.161534059542156], [41.39492045233958, 2.1609503169212423], [41.39484881807466, 2.1609430016619484], [41.395039300600516, 2.161484725259891], [41.3954350300871, 2.160843900132612], [41.39545038627625, 2.161178694723352], [41.39490020548141, 2.1608438222124273], [41.395246942354134, 2.161083302175071], [41.394928280026406, 2.161397184829935], [41.39539849436075, 2.1614430435553404], [41.39544742769243, 2.160904144689153], [41.39501398264134, 2.1613757749643425], [41.3947757574418, 2.16097472567621], [41.39466171410142, 2.161103161211066], [41.39486950332099, 2.160959647607385], [41.39507066758796, 2.1615901292493374], [41.395348920286914, 2.1608773343795744], [41.39505022432894, 2.161688823432168], [41.395397574389136, 2.1615993482747498], [41.39489363356377, 2.161062582222773], [41.395254833898875, 2.1608105234933555], [41.39512042453417, 2.161367704473529], [41.39469202145404, 2.1607907274867335], [41.39481724489328, 2.16081357757976], [41.39533758536627, 2.1610065598492696], [41.404097018732394, 2.1442169351420595], [41.40634510747849, 2.1443486444546735], [41.40481828291539, 2.1441186859538486], [41.406727910435805, 2.144083522039823], [41.40604897220509, 2.141374423021797], [41.4062595271877, 2.1442790378785683], [41.40682982511334, 2.1435072024579136], [41.406425786197374, 2.140740381987816], [41.40417980601626, 2.1414090556957324], [41.407052789536046, 2.1427461142635336], [41.40644420312612, 2.143073868609163], [41.40419995286453, 2.1407932373939222], [41.40736856169279, 2.1433641430860235], [41.4036779316909, 2.143173989399465], [41.40540016044783, 2.1413620332915237], [41.40458727480405, 2.1438571657638685], [41.40720325355831, 2.1424087733095964], [41.40344539596366, 2.1421306794956965], [41.406177037031796, 2.1436481928693643], [41.405616565783966, 2.143830027772143], [41.404131210881175, 2.1420069772772568], [41.40672371226692, 2.142095613099188], [41.40728593440124, 2.1411137063378005], [41.40446420873669, 2.1434505420895062], [41.40499030246595, 2.142549216081541], [41.407134532799276, 2.14084559253625], [41.40382715481015, 2.1436021406045556], [41.40389737537175, 2.1434711306315357], [41.40506308398588, 2.1424652950355494], [41.40728622805026, 2.1443958808458112], [41.40647274626849, 2.142108154057843], [41.40368726245982, 2.140521112443545], [41.40503980792287, 2.1444349643142604], [41.40592311816997, 2.1409571678624646], [41.40367773302272, 2.1436442862972123], [41.404436551765365, 2.1430012028142253], [41.40499940917282, 2.1408667067736267], [41.404119765214325, 2.143438563666975], [41.40624242435973, 2.143907205096761], [41.40531442126202, 2.143008075627878], [41.40404277128199, 2.144198879101151], [41.40446574102853, 2.141845319400163], [41.404409170052226, 2.1406571525478144], [41.403562097835795, 2.1413655780883216], [41.40374528055863, 2.141872428846674], [41.40597098511155, 2.1421668069896387], [41.403893198282155, 2.140820129854265], [41.404191878330316, 2.1442620041007543], [41.40727453137996, 2.1438592418074878], [41.40491726889419, 2.1432511913124106], [41.40733377347808, 2.142365134594433], [41.405682633904675, 2.1430955175414805], [41.40503451176703, 2.144166498528938], [41.403643873733785, 2.1411650489905343], [41.40709396432, 2.1440906202853625], [41.406837250166575, 2.1440439686435355], [41.40653663804196, 2.1424207117897787], [41.405300465247784, 2.142855385146769], [41.40690363752852, 2.1414433812477953], [41.40558996714681, 2.141215568423868], [41.40371992578054, 2.143511411555808], [41.404853759405505, 2.1443261084954166], [41.40504199756301, 2.1424395986036817], [41.40667965579579, 2.14075779232783], [41.405484614405545, 2.140476524963919], [41.40510013875884, 2.1412895255178745], [41.40567629973512, 2.141962638435009], [41.403499218085464, 2.1435764655895198], [41.404301303403514, 2.143916313915937], [41.403769302968335, 2.1436712685037524], [41.40344993251481, 2.141173090449789], [41.406864393324014, 2.1405205302010524], [41.404269929709464, 2.144333225261026], [41.403898402742456, 2.142115165829949], [41.40583544995513, 2.1414853061274703], [41.40484747005528, 2.1435559179308106], [41.406806818663206, 2.14188926198757], [41.406942752066506, 2.1431184089247197], [41.40467437100971, 2.1435178228329397], [41.40602178737928, 2.1408754671622026], [41.40454854442099, 2.1416194175699346], [41.40407612023306, 2.1408456678551118], [41.40609902331399, 2.1404843040664026], [41.40520751547993, 2.1410274796049804], [41.40710634118748, 2.1438427215156795], [41.4065039090507, 2.141380716886714], [41.40727570890659, 2.1421878555129097], [41.405253688879284, 2.1434458870317963], [41.4060995614047, 2.1413546046216676], [41.40614274858162, 2.1430639071897035], [41.40542590534722, 2.1416047944100964], [41.406584428826974, 2.1441152376839576], [41.40490101990538, 2.141853655748334], [41.40582667033571, 2.1430187508699077], [41.40612406862683, 2.14408037864456], [41.405883119229394, 2.1443916387606357], [41.40462848979684, 2.140808652944432], [41.40627333007121, 2.143893801110999], [41.405809599181374, 2.1406859632255846], [41.40641875065466, 2.1440606893783634], [41.39889088942146, 2.1536754140782683], [41.39889214158819, 2.1536846046261076], [41.398901980809164, 2.1536629359060027], [41.39889428191079, 2.1536694513086143], [41.39890118291155, 2.1536675999790376], [41.398914050990534, 2.1536864948775687], [41.39889617148213, 2.153663044350911], [41.398906611676914, 2.1536500413885635], [41.39891722410559, 2.1536652813810955], [41.398900591621796, 2.1536519068922626], [41.39890910906316, 2.153660205979174], [41.39889521316471, 2.153687512006746], [41.39888892287325, 2.153680330459347], [41.398918648870904, 2.153671951399054], [41.3989188603482, 2.153661730564464], [41.39888604999034, 2.153673906899416], [41.39888737974066, 2.1536605403547706], [41.39891039876662, 2.1536545024885956], [41.39888607853806, 2.153678714647461], [41.39890988763441, 2.1536795283468533], [41.39888122776631, 2.1536664438108235], [41.39670262418653, 2.1606994104909307], [41.3964710313751, 2.1594996665749235], [41.39842477884005, 2.1578234462459873], [41.39549299780951, 2.1611680536655973], [41.39872806888814, 2.1589146361100835], [41.395351529982015, 2.1588879066727795], [41.39592668667556, 2.1592541574106887], [41.395067670189235, 2.159100074160528], [41.39647068425935, 2.158005066252855], [41.397007255338515, 2.1608836884792777], [41.398397719604674, 2.1613033109560096], [41.39737588525979, 2.157698793942483], [41.39640552456657, 2.1591028552081024], [41.39643977911705, 2.1612789064221016], [41.395894837620915, 2.160093668508917], [41.39776312898072, 2.1588470126558947], [41.3987644087172, 2.1575588201213622], [41.39850616789657, 2.1592311565776576], [41.39587759834307, 2.1577434892203597], [41.3966787100864, 2.1596558585017314], [41.39771302686244, 2.161416475742277], [41.39530857228537, 2.1601410955535885], [41.39755944019087, 2.1574660523162144], [41.39605867525273, 2.1587660151737307], [41.39878009290586, 2.160758682704458], [41.39859770123924, 2.160010390637435], [41.39871225024301, 2.1608645585900357], [41.39699857961442, 2.159720782634396], [41.39542029582262, 2.1614286276369783], [41.39653049105947, 2.158281034593084], [41.39731911286, 2.160261164619916], [41.39524733676893, 2.159562684506873], [41.3956786157529, 2.1579473678540206], [41.39848252506233, 2.1610477635442207], [41.398084857963724, 2.1587322332252423], [41.3980688853548, 2.158667773274291], [41.3970977662437, 2.1579461437632457], [41.39511936088578, 2.1598696171158522], [41.395979595368615, 2.1606394806505365], [41.39577388900776, 2.1580267443782595], [41.398401736810946, 2.1605720879672012], [41.395489206289, 2.158477158558344], [41.39536655617508, 2.158346191276611], [41.39750453164683, 2.1613044802865273], [41.39806045661536, 2.1612681355708743], [41.3976026664179, 2.158321302582059], [41.398879479825965, 2.159159615039533], [41.39844973581171, 2.160610229801341], [41.39682967961411, 2.1588477477515036], [41.398689634958956, 2.1605682726595448], [41.397254751135854, 2.161126890469183], [41.39588594605653, 2.1600140601862705], [41.39866911028695, 2.159490323253213], [41.39697231143727, 2.1604709189874196], [41.396889795636746, 2.1584270868727677], [41.3968194226387, 2.160640898885843], [41.39764579821502, 2.1577100949645796], [41.39815889217853, 2.1604353763893465], [41.39578670856532, 2.159031726246931], [41.395616011546906, 2.1604112603936585], [41.39871583627161, 2.1579959426101674], [41.3977803831314, 2.161218952756661], [41.39863910776504, 2.160168615710207], [41.395300881603376, 2.159533514827968], [41.39886641173027, 2.1577160996558877], [41.39746760071943, 2.1607621791010243], [41.39566355039911, 2.160397811961645], [41.39724658046189, 2.159808871482451], [41.395894485168, 2.1594475808618583], [41.397698352276784, 2.158787067185035], [41.39497036338885, 2.160804231796431], [41.39686988684407, 2.159623483956613], [41.39811783991166, 2.1576997175518096], [41.39716339486879, 2.158883495574809], [41.39780299539646, 2.157940845142947], [41.396922212245904, 2.157512990931105], [41.3955451043031, 2.159826536007309], [41.397288606706134, 2.1612314289918637], [41.398791873580834, 2.158366703562201], [41.39886836554259, 2.159251648653193], [41.39569999093832, 2.1588593229891235], [41.397237940259004, 2.159403610571613], [41.39657272489176, 2.1613565671653747], [41.39747075936822, 2.1587545777946615], [41.39544291295498, 2.1585135198459215], [41.3967961039575, 2.159708456757902], [41.397740327721195, 2.157817494144303], [41.39511458154069, 2.1576322993149883], [41.39533838904781, 2.1593253165750377], [41.39673712154172, 2.159276864979319], [41.39664519747349, 2.157727807760756], [41.39503253914392, 2.158279995314986], [41.397591998578775, 2.1576350617417055], [41.395147390522894, 2.159314602693537], [41.395352294435476, 2.159731434301624], [41.398726109994726, 2.1577723930032233], [41.39795977077476, 2.160465718650735], [41.39756281843921, 2.1590805821891346], [41.39690200485551, 2.1592834493877087], [41.39794292411754, 2.15753317006657], [41.39787221916373, 2.152517537323243], [41.399173239138044, 2.153276526700623], [41.39836825495523, 2.1536160537996114], [41.398392334037716, 2.1539072652093068], [41.39783741385611, 2.1532309338477313], [41.397594242724544, 2.1532284484695507], [41.3976645998826, 2.153667458438531], [41.39807863532766, 2.153381359223621], [41.398318066531644, 2.1529340580673293], [41.39774546968743, 2.1534088773340647], [41.3992489978117, 2.1533318192310587], [41.39878931016422, 2.1525412183221473], [41.3992282152963, 2.1529096144343014], [41.39920375100786, 2.1523698841947843], [41.3980426374157, 2.1526929111347983], [41.39850435793943, 2.1533210457425294], [41.39755815849529, 2.1523442314973984], [41.39815123861754, 2.1522557337530595], [41.39854493636613, 2.1537572420387145], [41.397843609404624, 2.154038763817791], [41.398542518484916, 2.1539635689614145], [41.39835197836963, 2.1533211241620402], [41.39895499175484, 2.153249674766625], [41.398341233444384, 2.153387122535801], [41.398494607936975, 2.1537605720731134], [41.398017630045814, 2.1526514686263867], [41.3979757894913, 2.152268285761117], [41.39791256881199, 2.1539897399158843], [41.39913597468382, 2.1527414803226907], [41.39809735598218, 2.1525709672331343], [41.397913187416684, 2.1528115886547945], [41.39774984512918, 2.1536783465974327], [41.39808454429156, 2.152373665503355], [41.39837727127185, 2.153144759918536], [41.398686610368145, 2.153013210626711], [41.39883592667514, 2.1528547673769154], [41.39772520023713, 2.153346514057521], [41.39908481578384, 2.153782136433965], [41.39916445242152, 2.153107882747857], [41.39861948859257, 2.1527760184261417], [41.39885038491014, 2.1538312849162353], [41.398467085664585, 2.1538041800931023], [41.39763470783794, 2.152536111081584], [41.397875366450535, 2.1529411600383885], [41.39797347020702, 2.152563319252573], [41.39753627153311, 2.1531311717108443], [41.39757085705006, 2.153614823582096], [41.39878208311022, 2.153882796947744], [41.39882381221276, 2.152917467547198], [41.39758928657125, 2.15236486637482], [41.3977292872103, 2.153559740361084], [41.39803839640071, 2.1530299365585277], [41.39780548289538, 2.153034309997648], [41.39888435732411, 2.1533054125269016], [41.397552112460694, 2.152797346666545], [41.39767748099728, 2.1539624926780196], [41.39883550602244, 2.1525171943853914], [41.39891312373312, 2.1540199867520293], [41.397564317616144, 2.1532126918887804], [41.399162876089605, 2.152748818147188], [41.398456279002076, 2.1539122441245757], [41.39773152917314, 2.1522661386840545], [41.39866331447737, 2.152741597702253], [41.3989932022224, 2.15346522976452], [41.3979250500921, 2.1527129097072097], [41.39919688020793, 2.152969763752507], [41.39833221753272, 2.153167276331903], [41.398529647846466, 2.152661847110726], [41.397963909808354, 2.1524513346274485], [41.39846065568321, 2.1531159111636358], [41.39751286691648, 2.152448104099325], [41.398390925991905, 2.153474306188798], [41.39773450410385, 2.1533538724016177], [41.39792221189593, 2.153179256489425], [41.398611575001446, 2.1532571271029006], [41.3980147658953, 2.1529716850777025]],
                {
                    minOpacity: 0.1,
                    maxZoom: 18,
                    max: 0.8,
                    radius: 50,
                    blur: 70,
                    gradient: null
                    })
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
</script>


<!DOCTYPE html>
<head>    
    <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
    <script>L_PREFER_CANVAS=false; L_NO_TOUCH=false; L_DISABLE_3D=false;</script>
    <script src="https://cdn.jsdelivr.net/npm/leaflet@1.2.0/dist/leaflet.js"></script>
    <script src="https://ajax.googleapis.com/ajax/libs/jquery/1.11.1/jquery.min.js"></script>
    <script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/js/bootstrap.min.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js"></script>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet@1.2.0/dist/leaflet.css"/>
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap.min.css"/>
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap-theme.min.css"/>
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.6.3/css/font-awesome.min.css"/>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css"/>
    <link rel="stylesheet" href="https://rawgit.com/python-visualization/folium/master/folium/templates/leaflet.awesome.rotate.css"/>
    <style>html, body {width: 100%;height: 100%;margin: 0;padding: 0;}</style>
    <style>#map {position:absolute;top:0;bottom:0;right:0;left:0;}</style>
    
    <style>#map_e9a0ac71acee47749a2d0b689bffffa3 {
        position: relative;
        width: 100.0%;
        height: 100.0%;
        left: 0.0%;
        top: 0.0%;
        }
    </style>
    <script src="https://leaflet.github.io/Leaflet.heat/dist/leaflet-heat.js"></script>
</head>
<body>    
    
    <div class="folium-map" id="map_e9a0ac71acee47749a2d0b689bffffa3" ></div>
</body>
<script>    
    
    
        var bounds = null;
    

    var map_e9a0ac71acee47749a2d0b689bffffa3 = L.map(
        'map_e9a0ac71acee47749a2d0b689bffffa3', {
        center: [41.41023, 2.15087],
        zoom: 14,
        maxBounds: bounds,
        layers: [],
        worldCopyJump: false,
        crs: L.CRS.EPSG3857,
        zoomControl: true,
        });

    
    
    var tile_layer_c04ed4ef95b249c58b259d0cd72268ca = L.tileLayer(
        'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png',
        {
        "attribution": null,
        "detectRetina": false,
        "maxNativeZoom": 18,
        "maxZoom": 18,
        "minZoom": 0,
        "noWrap": false,
        "subdomains": "abc"
}).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
    
            var circle_marker_15a058ce23274d019bb69b6807fd4302 = L.circleMarker(
                [41.402535, 2.152519],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_6248037211bc4bec907bf599c10b3e6b = L.popup({maxWidth: '300'
            
            });

            
                var html_ce3c263f66ed4467abab88b6a048cc24 = $('<div id="html_ce3c263f66ed4467abab88b6a048cc24" style="width: 100.0%; height: 100.0%;">221 - GRAN DE GRÀCIA, 155 (METRO FONTANA)</div>')[0];
                popup_6248037211bc4bec907bf599c10b3e6b.setContent(html_ce3c263f66ed4467abab88b6a048cc24);
            

            circle_marker_15a058ce23274d019bb69b6807fd4302.bindPopup(popup_6248037211bc4bec907bf599c10b3e6b)
            ;

            
        
    
        var marker_db8584da64f34f9c874a87deec0d3abe = L.marker(
            [41.402535, 2.152519],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_f0651221fe664695a4f6b0e1af7a31ef = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'green',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_db8584da64f34f9c874a87deec0d3abe.setIcon(icon_f0651221fe664695a4f6b0e1af7a31ef);
            
    
            var popup_671d3095a25947e89788af04350e413e = L.popup({maxWidth: '300'
            
            });

            
                var html_15880d0d68874a36b597fa4e28a0ca3a = $('<div id="html_15880d0d68874a36b597fa4e28a0ca3a" style="width: 100.0%; height: 100.0%;">221 - GRAN DE GRÀCIA, 155 (METRO FONTANA)::250.0</div>')[0];
                popup_671d3095a25947e89788af04350e413e.setContent(html_15880d0d68874a36b597fa4e28a0ca3a);
            

            marker_db8584da64f34f9c874a87deec0d3abe.bindPopup(popup_671d3095a25947e89788af04350e413e)
            ;

            
        
    
            var circle_marker_741f1607a7a74f9f849b7d80cf65ce50 = L.circleMarker(
                [41.40124, 2.157483],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_ad1d6418c8f14393877027ff792b7e5c = L.popup({maxWidth: '300'
            
            });

            
                var html_95427642acbd4710baf757240e69d49d = $('<div id="html_95427642acbd4710baf757240e69d49d" style="width: 100.0%; height: 100.0%;">222 - C/ DEL CANÓ, 1</div>')[0];
                popup_ad1d6418c8f14393877027ff792b7e5c.setContent(html_95427642acbd4710baf757240e69d49d);
            

            circle_marker_741f1607a7a74f9f849b7d80cf65ce50.bindPopup(popup_ad1d6418c8f14393877027ff792b7e5c)
            ;

            
        
    
        var marker_56fe443fcdbe4cd99ab60595b8e8f4bc = L.marker(
            [41.40124, 2.157483],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_bdd04fe069e444abb690d48d5c36b5b3 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'black',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_56fe443fcdbe4cd99ab60595b8e8f4bc.setIcon(icon_bdd04fe069e444abb690d48d5c36b5b3);
            
    
            var popup_65959a05371249f09620e7290b03db13 = L.popup({maxWidth: '300'
            
            });

            
                var html_cccc49b7bf5e4be894b396516cd602f7 = $('<div id="html_cccc49b7bf5e4be894b396516cd602f7" style="width: 100.0%; height: 100.0%;">222 - C/ DEL CANÓ, 1::0.0</div>')[0];
                popup_65959a05371249f09620e7290b03db13.setContent(html_cccc49b7bf5e4be894b396516cd602f7);
            

            marker_56fe443fcdbe4cd99ab60595b8e8f4bc.bindPopup(popup_65959a05371249f09620e7290b03db13)
            ;

            
        
    
            var circle_marker_ae3976c512b047c09d6f2e169550901a = L.circleMarker(
                [41.398311, 2.159865],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_b6b3ef290e644eb488940ac9c34fa9ed = L.popup({maxWidth: '300'
            
            });

            
                var html_c566c2485c2a4e3798c309647df6f4e6 = $('<div id="html_c566c2485c2a4e3798c309647df6f4e6" style="width: 100.0%; height: 100.0%;">223 - C/ DE BONAVISTA, 14</div>')[0];
                popup_b6b3ef290e644eb488940ac9c34fa9ed.setContent(html_c566c2485c2a4e3798c309647df6f4e6);
            

            circle_marker_ae3976c512b047c09d6f2e169550901a.bindPopup(popup_b6b3ef290e644eb488940ac9c34fa9ed)
            ;

            
        
    
        var marker_924d0863e19f477886b97730cd528eb7 = L.marker(
            [41.398311, 2.159865],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_9a91397e5a4549ec92db25324e381f34 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'orange',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_924d0863e19f477886b97730cd528eb7.setIcon(icon_9a91397e5a4549ec92db25324e381f34);
            
    
            var popup_cf66fca2849940b1a2b40a800e8dbf70 = L.popup({maxWidth: '300'
            
            });

            
                var html_c6f842e2c9ba40f294955fb3503b6438 = $('<div id="html_c6f842e2c9ba40f294955fb3503b6438" style="width: 100.0%; height: 100.0%;">223 - C/ DE BONAVISTA, 14::197.0</div>')[0];
                popup_cf66fca2849940b1a2b40a800e8dbf70.setContent(html_c6f842e2c9ba40f294955fb3503b6438);
            

            marker_924d0863e19f477886b97730cd528eb7.bindPopup(popup_cf66fca2849940b1a2b40a800e8dbf70)
            ;

            
        
    
            var circle_marker_a7d4916bd0114decb99aa560481f4acb = L.circleMarker(
                [41.403465, 2.161096],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_1a2c34809517431abee77b1f69685706 = L.popup({maxWidth: '300'
            
            });

            
                var html_8013b0ec05504f72acbbe3e5df959a3a = $('<div id="html_8013b0ec05504f72acbbe3e5df959a3a" style="width: 100.0%; height: 100.0%;">226 - C/MONTMANY, 1</div>')[0];
                popup_1a2c34809517431abee77b1f69685706.setContent(html_8013b0ec05504f72acbbe3e5df959a3a);
            

            circle_marker_a7d4916bd0114decb99aa560481f4acb.bindPopup(popup_1a2c34809517431abee77b1f69685706)
            ;

            
        
    
        var marker_ca5c8d26d7914ca590c8942943ad4216 = L.marker(
            [41.403465, 2.161096],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_4ec295587b534fc6a3f56da58d4ac91e = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'orange',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_ca5c8d26d7914ca590c8942943ad4216.setIcon(icon_4ec295587b534fc6a3f56da58d4ac91e);
            
    
            var popup_492f03f75c4a43f6aca17f7dee899934 = L.popup({maxWidth: '300'
            
            });

            
                var html_c0bd7314c63b4bf9b4590331100d4b0a = $('<div id="html_c0bd7314c63b4bf9b4590331100d4b0a" style="width: 100.0%; height: 100.0%;">226 - C/MONTMANY, 1::200.0</div>')[0];
                popup_492f03f75c4a43f6aca17f7dee899934.setContent(html_c0bd7314c63b4bf9b4590331100d4b0a);
            

            marker_ca5c8d26d7914ca590c8942943ad4216.bindPopup(popup_492f03f75c4a43f6aca17f7dee899934)
            ;

            
        
    
            var circle_marker_a2d18d8f62b7433a8d04e6c081eee1a3 = L.circleMarker(
                [41.405986, 2.151633],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_da360bf86e9d4e61b2adb431cb47dd20 = L.popup({maxWidth: '300'
            
            });

            
                var html_1cc4ea8b9b754b33a6f6937be74a0c2e = $('<div id="html_1cc4ea8b9b754b33a6f6937be74a0c2e" style="width: 100.0%; height: 100.0%;">230 - C/ DE NIL FABRA, 16-20</div>')[0];
                popup_da360bf86e9d4e61b2adb431cb47dd20.setContent(html_1cc4ea8b9b754b33a6f6937be74a0c2e);
            

            circle_marker_a2d18d8f62b7433a8d04e6c081eee1a3.bindPopup(popup_da360bf86e9d4e61b2adb431cb47dd20)
            ;

            
        
    
        var marker_08e6128d2ead4753b470f1dac4208a5b = L.marker(
            [41.405986, 2.151633],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_eda419ded3174081958fe5717affd4cc = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'orange',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_08e6128d2ead4753b470f1dac4208a5b.setIcon(icon_eda419ded3174081958fe5717affd4cc);
            
    
            var popup_f2e40bdfeb2642dc8bff2fa39833aa60 = L.popup({maxWidth: '300'
            
            });

            
                var html_0fef82b9341b4cf9bc9a931a92befef3 = $('<div id="html_0fef82b9341b4cf9bc9a931a92befef3" style="width: 100.0%; height: 100.0%;">230 - C/ DE NIL FABRA, 16-20::232.0</div>')[0];
                popup_f2e40bdfeb2642dc8bff2fa39833aa60.setContent(html_0fef82b9341b4cf9bc9a931a92befef3);
            

            marker_08e6128d2ead4753b470f1dac4208a5b.bindPopup(popup_f2e40bdfeb2642dc8bff2fa39833aa60)
            ;

            
        
    
            var circle_marker_1935b342cb3b42459c8dfde8e823743c = L.circleMarker(
                [41.38551, 2.152],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_24832a2be44f49ae8ae604ae14dc63e0 = L.popup({maxWidth: '300'
            
            });

            
                var html_cd3430c0aeb04883b182657b1ab918f6 = $('<div id="html_cd3430c0aeb04883b182657b1ab918f6" style="width: 100.0%; height: 100.0%;">405 - C/ COMTE BORRELL, 198</div>')[0];
                popup_24832a2be44f49ae8ae604ae14dc63e0.setContent(html_cd3430c0aeb04883b182657b1ab918f6);
            

            circle_marker_1935b342cb3b42459c8dfde8e823743c.bindPopup(popup_24832a2be44f49ae8ae604ae14dc63e0)
            ;

            
        
    
        var marker_3d9c81720e064f24b2cd2c723226bb07 = L.marker(
            [41.38551, 2.152],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_0124b950f540464193c9244081a6b2e4 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'orange',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_3d9c81720e064f24b2cd2c723226bb07.setIcon(icon_0124b950f540464193c9244081a6b2e4);
            
    
            var popup_ec849234a52c4d8e809bde159f776874 = L.popup({maxWidth: '300'
            
            });

            
                var html_f7e1482bda7f417d8406c1bd399d5dc8 = $('<div id="html_f7e1482bda7f417d8406c1bd399d5dc8" style="width: 100.0%; height: 100.0%;">405 - C/ COMTE BORRELL, 198::228.0</div>')[0];
                popup_ec849234a52c4d8e809bde159f776874.setContent(html_f7e1482bda7f417d8406c1bd399d5dc8);
            

            marker_3d9c81720e064f24b2cd2c723226bb07.bindPopup(popup_ec849234a52c4d8e809bde159f776874)
            ;

            
        
    
            var circle_marker_17cc42dfc33e4de8bd1dd8a98001d348 = L.circleMarker(
                [41.407353, 2.151341],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_887a2b26df094bcd955032e64b4f07a1 = L.popup({maxWidth: '300'
            
            });

            
                var html_6509548179a34e36bcd7db7645159f5e = $('<div id="html_6509548179a34e36bcd7db7645159f5e" style="width: 100.0%; height: 100.0%;">472 - (PK) C/ DEL TORRENT DE L´OLLA, 221</div>')[0];
                popup_887a2b26df094bcd955032e64b4f07a1.setContent(html_6509548179a34e36bcd7db7645159f5e);
            

            circle_marker_17cc42dfc33e4de8bd1dd8a98001d348.bindPopup(popup_887a2b26df094bcd955032e64b4f07a1)
            ;

            
        
    
        var marker_623edfa6eb2d41d09976f9c6aea4d100 = L.marker(
            [41.407353, 2.151341],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_0fbaef28e85c4cc1ad64b57ae5c92938 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'blue',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_623edfa6eb2d41d09976f9c6aea4d100.setIcon(icon_0fbaef28e85c4cc1ad64b57ae5c92938);
            
    
            var popup_40f37e4289f64cde8508cb41c3926e89 = L.popup({maxWidth: '300'
            
            });

            
                var html_40e25089959c49d6bbb69168738b8505 = $('<div id="html_40e25089959c49d6bbb69168738b8505" style="width: 100.0%; height: 100.0%;">472 - (PK) C/ DEL TORRENT DE L´OLLA, 221::266.0</div>')[0];
                popup_40f37e4289f64cde8508cb41c3926e89.setContent(html_40e25089959c49d6bbb69168738b8505);
            

            marker_623edfa6eb2d41d09976f9c6aea4d100.bindPopup(popup_40f37e4289f64cde8508cb41c3926e89)
            ;

            
        
    
            var circle_marker_a8c39ff73833459abb1669e5b5b2b727 = L.circleMarker(
                [41.401752, 2.160748],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_b7ecef024b914d0e91bc8e060eea384d = L.popup({maxWidth: '300'
            
            });

            
                var html_211b90a6c8ac4f7ca7f5dd18df3fbf1b = $('<div id="html_211b90a6c8ac4f7ca7f5dd18df3fbf1b" style="width: 100.0%; height: 100.0%;">483 - (PK) C/ DE SIRACUSA, 39-51</div>')[0];
                popup_b7ecef024b914d0e91bc8e060eea384d.setContent(html_211b90a6c8ac4f7ca7f5dd18df3fbf1b);
            

            circle_marker_a8c39ff73833459abb1669e5b5b2b727.bindPopup(popup_b7ecef024b914d0e91bc8e060eea384d)
            ;

            
        
    
        var marker_8f9d47b6b2dc4a0cae17fd19d766ad33 = L.marker(
            [41.401752, 2.160748],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_8120fe7934e94f67a9a03f8ebdf320de = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'blue',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_8f9d47b6b2dc4a0cae17fd19d766ad33.setIcon(icon_8120fe7934e94f67a9a03f8ebdf320de);
            
    
            var popup_1c8e84a02ec94008ae62ba639913decc = L.popup({maxWidth: '300'
            
            });

            
                var html_3112b960b37343a18691ee0411f0da87 = $('<div id="html_3112b960b37343a18691ee0411f0da87" style="width: 100.0%; height: 100.0%;">483 - (PK) C/ DE SIRACUSA, 39-51::266.0</div>')[0];
                popup_1c8e84a02ec94008ae62ba639913decc.setContent(html_3112b960b37343a18691ee0411f0da87);
            

            marker_8f9d47b6b2dc4a0cae17fd19d766ad33.bindPopup(popup_1c8e84a02ec94008ae62ba639913decc)
            ;

            
        
    
            var circle_marker_e7ea295c38144818bec8e5875a414f1f = L.circleMarker(
                [41.384956, 2.150814],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_3a304ea67e474e14a5a3ac910d1193e6 = L.popup({maxWidth: '300'
            
            });

            
                var html_488250e842f848f4b944b2a019bfd3ea = $('<div id="html_488250e842f848f4b944b2a019bfd3ea" style="width: 100.0%; height: 100.0%;">225 - C/MALLORCA, 84</div>')[0];
                popup_3a304ea67e474e14a5a3ac910d1193e6.setContent(html_488250e842f848f4b944b2a019bfd3ea);
            

            circle_marker_e7ea295c38144818bec8e5875a414f1f.bindPopup(popup_3a304ea67e474e14a5a3ac910d1193e6)
            ;

            
        
    
        var marker_bc8ab54f8cba4fa582cf87b8b801c692 = L.marker(
            [41.384956, 2.150814],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_697bd985cc184436bcbeb4b9d0256cec = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'orange',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_bc8ab54f8cba4fa582cf87b8b801c692.setIcon(icon_697bd985cc184436bcbeb4b9d0256cec);
            
    
            var popup_ae49ab20749b4fe1a0b713e13130d18c = L.popup({maxWidth: '300'
            
            });

            
                var html_907951bccfbf433fb016e23d67f7e7e3 = $('<div id="html_907951bccfbf433fb016e23d67f7e7e3" style="width: 100.0%; height: 100.0%;">225 - C/MALLORCA, 84::222.0</div>')[0];
                popup_ae49ab20749b4fe1a0b713e13130d18c.setContent(html_907951bccfbf433fb016e23d67f7e7e3);
            

            marker_bc8ab54f8cba4fa582cf87b8b801c692.bindPopup(popup_ae49ab20749b4fe1a0b713e13130d18c)
            ;

            
        
    
            var circle_marker_aa7d8c5c40ae4ddba4440f32a5979d2c = L.circleMarker(
                [41.395364, 2.157356],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_de59bc3e803245208645fccd1a43bdd6 = L.popup({maxWidth: '300'
            
            });

            
                var html_4123d18ff0874fabadf0e2f8c804c88f = $('<div id="html_4123d18ff0874fabadf0e2f8c804c88f" style="width: 100.0%; height: 100.0%;">68 - RAMBLA CATALUNYA, 133 /  CÒRSEGA</div>')[0];
                popup_de59bc3e803245208645fccd1a43bdd6.setContent(html_4123d18ff0874fabadf0e2f8c804c88f);
            

            circle_marker_aa7d8c5c40ae4ddba4440f32a5979d2c.bindPopup(popup_de59bc3e803245208645fccd1a43bdd6)
            ;

            
        
    
        var marker_fe974fbdbff9406f90244da7fdb4ff6d = L.marker(
            [41.395364, 2.157356],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_266c8e43cb7d4d2c9f65e411e267e02c = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'red',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_fe974fbdbff9406f90244da7fdb4ff6d.setIcon(icon_266c8e43cb7d4d2c9f65e411e267e02c);
            
    
            var popup_c87c1a814b504d06adb0a89815ccfb8c = L.popup({maxWidth: '300'
            
            });

            
                var html_522347c43e6a4f58ba8e61e627f3a0e7 = $('<div id="html_522347c43e6a4f58ba8e61e627f3a0e7" style="width: 100.0%; height: 100.0%;">68 - RAMBLA CATALUNYA, 133 /  CÒRSEGA::182.0</div>')[0];
                popup_c87c1a814b504d06adb0a89815ccfb8c.setContent(html_522347c43e6a4f58ba8e61e627f3a0e7);
            

            marker_fe974fbdbff9406f90244da7fdb4ff6d.bindPopup(popup_c87c1a814b504d06adb0a89815ccfb8c)
            ;

            
        
    
            var circle_marker_bed09b45af494ec6bff37025424ace34 = L.circleMarker(
                [41.384075, 2.153654],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_320c5ccb39ac4f579a64f45dad56f585 = L.popup({maxWidth: '300'
            
            });

            
                var html_9b676f2bec5f4ac09646b2a1e62b91ca = $('<div id="html_9b676f2bec5f4ac09646b2a1e62b91ca" style="width: 100.0%; height: 100.0%;">83 - C/ COMTE BORRELL 177</div>')[0];
                popup_320c5ccb39ac4f579a64f45dad56f585.setContent(html_9b676f2bec5f4ac09646b2a1e62b91ca);
            

            circle_marker_bed09b45af494ec6bff37025424ace34.bindPopup(popup_320c5ccb39ac4f579a64f45dad56f585)
            ;

            
        
    
        var marker_19221659c3454ef6ad4b4b51abb7b7c9 = L.marker(
            [41.384075, 2.153654],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_fa458165727e47468f3306bc767edf85 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'orange',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_19221659c3454ef6ad4b4b51abb7b7c9.setIcon(icon_fa458165727e47468f3306bc767edf85);
            
    
            var popup_098b929284ba4b74a2bb54e2db3134e4 = L.popup({maxWidth: '300'
            
            });

            
                var html_5a15b3a8c83d4c28885a08e729e1b768 = $('<div id="html_5a15b3a8c83d4c28885a08e729e1b768" style="width: 100.0%; height: 100.0%;">83 - C/ COMTE BORRELL 177::220.0</div>')[0];
                popup_098b929284ba4b74a2bb54e2db3134e4.setContent(html_5a15b3a8c83d4c28885a08e729e1b768);
            

            marker_19221659c3454ef6ad4b4b51abb7b7c9.bindPopup(popup_098b929284ba4b74a2bb54e2db3134e4)
            ;

            
        
    
            var circle_marker_bd9bf998ebf84e84921f3f672f8c58dc = L.circleMarker(
                [41.383156, 2.148506],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_10115bea6a6c4abd8c0f9a54fdea6e52 = L.popup({maxWidth: '300'
            
            });

            
                var html_2445034e4cb74935abfd2433048014d9 = $('<div id="html_2445034e4cb74935abfd2433048014d9" style="width: 100.0%; height: 100.0%;">87 - C/ MALLORCA 41-43</div>')[0];
                popup_10115bea6a6c4abd8c0f9a54fdea6e52.setContent(html_2445034e4cb74935abfd2433048014d9);
            

            circle_marker_bd9bf998ebf84e84921f3f672f8c58dc.bindPopup(popup_10115bea6a6c4abd8c0f9a54fdea6e52)
            ;

            
        
    
        var marker_3382f9fbfbf64db493eeb2558756bbc8 = L.marker(
            [41.383156, 2.148506],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_e421c9919d7e42e28159e3bf2e3cb926 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'orange',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_3382f9fbfbf64db493eeb2558756bbc8.setIcon(icon_e421c9919d7e42e28159e3bf2e3cb926);
            
    
            var popup_b1e760ec404a4b9c9ac60c70fdc7f4df = L.popup({maxWidth: '300'
            
            });

            
                var html_63190c78cfe14401a0d78a6e8d757fda = $('<div id="html_63190c78cfe14401a0d78a6e8d757fda" style="width: 100.0%; height: 100.0%;">87 - C/ MALLORCA 41-43::219.0</div>')[0];
                popup_b1e760ec404a4b9c9ac60c70fdc7f4df.setContent(html_63190c78cfe14401a0d78a6e8d757fda);
            

            marker_3382f9fbfbf64db493eeb2558756bbc8.bindPopup(popup_b1e760ec404a4b9c9ac60c70fdc7f4df)
            ;

            
        
    
            var circle_marker_efdd786a997d4b8dafd3b8b261dcfa0b = L.circleMarker(
                [41.387953, 2.150169],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_0f377ef04cec43698c2e41bd58716b77 = L.popup({maxWidth: '300'
            
            });

            
                var html_3eb7f55de0194edc8124f05958d441e8 = $('<div id="html_3eb7f55de0194edc8124f05958d441e8" style="width: 100.0%; height: 100.0%;">89 - C/ ROSSELLÓ, 101</div>')[0];
                popup_0f377ef04cec43698c2e41bd58716b77.setContent(html_3eb7f55de0194edc8124f05958d441e8);
            

            circle_marker_efdd786a997d4b8dafd3b8b261dcfa0b.bindPopup(popup_0f377ef04cec43698c2e41bd58716b77)
            ;

            
        
    
        var marker_73aa914361a3479d8353fb3505a67d7e = L.marker(
            [41.387953, 2.150169],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_ec571cd0696347198d11ee7a34c6b5b5 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'green',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_73aa914361a3479d8353fb3505a67d7e.setIcon(icon_ec571cd0696347198d11ee7a34c6b5b5);
            
    
            var popup_f523e0c8232b4de2b80d7e0af7e0c536 = L.popup({maxWidth: '300'
            
            });

            
                var html_023c7b1111fc44b6821823032d285e71 = $('<div id="html_023c7b1111fc44b6821823032d285e71" style="width: 100.0%; height: 100.0%;">89 - C/ ROSSELLÓ, 101::263.0</div>')[0];
                popup_f523e0c8232b4de2b80d7e0af7e0c536.setContent(html_023c7b1111fc44b6821823032d285e71);
            

            marker_73aa914361a3479d8353fb3505a67d7e.bindPopup(popup_f523e0c8232b4de2b80d7e0af7e0c536)
            ;

            
        
    
            var circle_marker_aa0b93801dad4298b5c827e1eee236cd = L.circleMarker(
                [41.405587, 2.162317],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_b17fb692a3c24380b82bd02c61960b59 = L.popup({maxWidth: '300'
            
            });

            
                var html_7d4cf79c67754f34a0f34d2b683b88f2 = $('<div id="html_7d4cf79c67754f34a0f34d2b683b88f2" style="width: 100.0%; height: 100.0%;">106 - PL. JOANIC- C/ BRUNIQUER, 59</div>')[0];
                popup_b17fb692a3c24380b82bd02c61960b59.setContent(html_7d4cf79c67754f34a0f34d2b683b88f2);
            

            circle_marker_aa0b93801dad4298b5c827e1eee236cd.bindPopup(popup_b17fb692a3c24380b82bd02c61960b59)
            ;

            
        
    
        var marker_98333144b2f9461f96fd754a6d52d23d = L.marker(
            [41.405587, 2.162317],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_d8e9cf92252b48c29b2e2fb7f5f7c333 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'red',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_98333144b2f9461f96fd754a6d52d23d.setIcon(icon_d8e9cf92252b48c29b2e2fb7f5f7c333);
            
    
            var popup_026d7016aecd47028487d605198e4445 = L.popup({maxWidth: '300'
            
            });

            
                var html_ccb343d6bd2545d69cb988847239b4fc = $('<div id="html_ccb343d6bd2545d69cb988847239b4fc" style="width: 100.0%; height: 100.0%;">106 - PL. JOANIC- C/ BRUNIQUER, 59::182.0</div>')[0];
                popup_026d7016aecd47028487d605198e4445.setContent(html_ccb343d6bd2545d69cb988847239b4fc);
            

            marker_98333144b2f9461f96fd754a6d52d23d.bindPopup(popup_026d7016aecd47028487d605198e4445)
            ;

            
        
    
            var circle_marker_e61bced7212d4ecfbf4e3fec4aa10c58 = L.circleMarker(
                [41.402314, 2.164961],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_98de3bb6d5e84f78975ee8e3f8114b94 = L.popup({maxWidth: '300'
            
            });

            
                var html_74f2098239864d34b4b8c8db7c205579 = $('<div id="html_74f2098239864d34b4b8c8db7c205579" style="width: 100.0%; height: 100.0%;">108 - C/ INDÚSTRIA 10</div>')[0];
                popup_98de3bb6d5e84f78975ee8e3f8114b94.setContent(html_74f2098239864d34b4b8c8db7c205579);
            

            circle_marker_e61bced7212d4ecfbf4e3fec4aa10c58.bindPopup(popup_98de3bb6d5e84f78975ee8e3f8114b94)
            ;

            
        
    
        var marker_7f25d92f8a8b461781c06d311a20cfc5 = L.marker(
            [41.402314, 2.164961],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_dac86a68de61417f9d2f2545f9274e69 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'orange',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_7f25d92f8a8b461781c06d311a20cfc5.setIcon(icon_dac86a68de61417f9d2f2545f9274e69);
            
    
            var popup_3578eedc0e58452b9270eb2736388f96 = L.popup({maxWidth: '300'
            
            });

            
                var html_1a499271d3934a8caf4b146a2ff3a368 = $('<div id="html_1a499271d3934a8caf4b146a2ff3a368" style="width: 100.0%; height: 100.0%;">108 - C/ INDÚSTRIA 10::241.0</div>')[0];
                popup_3578eedc0e58452b9270eb2736388f96.setContent(html_1a499271d3934a8caf4b146a2ff3a368);
            

            marker_7f25d92f8a8b461781c06d311a20cfc5.bindPopup(popup_3578eedc0e58452b9270eb2736388f96)
            ;

            
        
    
            var circle_marker_4b91393c685d497a89194f3a7e548dca = L.circleMarker(
                [41.385377, 2.154878],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_f478d3ea519b4b76ae04084af9fb8e70 = L.popup({maxWidth: '300'
            
            });

            
                var html_0b4ab427398c48b59ede2e8957e4998b = $('<div id="html_0b4ab427398c48b59ede2e8957e4998b" style="width: 100.0%; height: 100.0%;">110 - AV. ROMA, 136</div>')[0];
                popup_f478d3ea519b4b76ae04084af9fb8e70.setContent(html_0b4ab427398c48b59ede2e8957e4998b);
            

            circle_marker_4b91393c685d497a89194f3a7e548dca.bindPopup(popup_f478d3ea519b4b76ae04084af9fb8e70)
            ;

            
        
    
        var marker_b02f1248d6fd4c3fad7f5db88be4d534 = L.marker(
            [41.385377, 2.154878],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_4c1f9694b74c415989bd04626af6104a = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'orange',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_b02f1248d6fd4c3fad7f5db88be4d534.setIcon(icon_4c1f9694b74c415989bd04626af6104a);
            
    
            var popup_2a194dfd90f544b3bc0f80f2213b1853 = L.popup({maxWidth: '300'
            
            });

            
                var html_ecb2a21b1c0442359f1261f72480ef70 = $('<div id="html_ecb2a21b1c0442359f1261f72480ef70" style="width: 100.0%; height: 100.0%;">110 - AV. ROMA, 136::231.0</div>')[0];
                popup_2a194dfd90f544b3bc0f80f2213b1853.setContent(html_ecb2a21b1c0442359f1261f72480ef70);
            

            marker_b02f1248d6fd4c3fad7f5db88be4d534.bindPopup(popup_2a194dfd90f544b3bc0f80f2213b1853)
            ;

            
        
    
            var circle_marker_453e0fa78c1c4f98885c0ef20277d92b = L.circleMarker(
                [41.405435, 2.16624],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_f9991f48aa9c456092318adce3e25b6d = L.popup({maxWidth: '300'
            
            });

            
                var html_3ec80a8584bc48f3a56aa724e9e13745 = $('<div id="html_3ec80a8584bc48f3a56aa724e9e13745" style="width: 100.0%; height: 100.0%;">122 - C/NÀPOLS, 344</div>')[0];
                popup_f9991f48aa9c456092318adce3e25b6d.setContent(html_3ec80a8584bc48f3a56aa724e9e13745);
            

            circle_marker_453e0fa78c1c4f98885c0ef20277d92b.bindPopup(popup_f9991f48aa9c456092318adce3e25b6d)
            ;

            
        
    
        var marker_62fd33f968b14f1ab7126dfff87b9d77 = L.marker(
            [41.405435, 2.16624],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_929abd3b707b40b1bc453976ffd434ed = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'green',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_62fd33f968b14f1ab7126dfff87b9d77.setIcon(icon_929abd3b707b40b1bc453976ffd434ed);
            
    
            var popup_1122cdad70364edcb296f76d373bc7d7 = L.popup({maxWidth: '300'
            
            });

            
                var html_644a49afa1be49d2a6394bb848c16b2c = $('<div id="html_644a49afa1be49d2a6394bb848c16b2c" style="width: 100.0%; height: 100.0%;">122 - C/NÀPOLS, 344::251.0</div>')[0];
                popup_1122cdad70364edcb296f76d373bc7d7.setContent(html_644a49afa1be49d2a6394bb848c16b2c);
            

            marker_62fd33f968b14f1ab7126dfff87b9d77.bindPopup(popup_1122cdad70364edcb296f76d373bc7d7)
            ;

            
        
    
            var circle_marker_e205671c44804ffa8a8e7cfeb3caa6e6 = L.circleMarker(
                [41.382564, 2.149094],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_10184c34c0b640d4b2426b09ff37eef4 = L.popup({maxWidth: '300'
            
            });

            
                var html_00720cb458164160a449df328aec62c0 = $('<div id="html_00720cb458164160a449df328aec62c0" style="width: 100.0%; height: 100.0%;">191 -C/ ROCAFORT, 167</div>')[0];
                popup_10184c34c0b640d4b2426b09ff37eef4.setContent(html_00720cb458164160a449df328aec62c0);
            

            circle_marker_e205671c44804ffa8a8e7cfeb3caa6e6.bindPopup(popup_10184c34c0b640d4b2426b09ff37eef4)
            ;

            
        
    
        var marker_ddf21f649a13474e8f2242ff9779f6c0 = L.marker(
            [41.382564, 2.149094],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_624afdfcaa2e4a1488f1ea357bff4962 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'orange',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_ddf21f649a13474e8f2242ff9779f6c0.setIcon(icon_624afdfcaa2e4a1488f1ea357bff4962);
            
    
            var popup_4c476f60d0244a6b907f70049dc491d2 = L.popup({maxWidth: '300'
            
            });

            
                var html_cc49e136e3be446e92a900a55cddfbd6 = $('<div id="html_cc49e136e3be446e92a900a55cddfbd6" style="width: 100.0%; height: 100.0%;">191 -C/ ROCAFORT, 167::199.0</div>')[0];
                popup_4c476f60d0244a6b907f70049dc491d2.setContent(html_cc49e136e3be446e92a900a55cddfbd6);
            

            marker_ddf21f649a13474e8f2242ff9779f6c0.bindPopup(popup_4c476f60d0244a6b907f70049dc491d2)
            ;

            
        
    
            var circle_marker_b889fbe7a6bb440984097554774023d3 = L.circleMarker(
                [41.399945, 2.164389],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_a312dd922a0a469babea91c9c4f0b9b2 = L.popup({maxWidth: '300'
            
            });

            
                var html_7cc2cb5fffff447fa4c061544de4488f = $('<div id="html_7cc2cb5fffff447fa4c061544de4488f" style="width: 100.0%; height: 100.0%;">224- C /GIRONA, 176</div>')[0];
                popup_a312dd922a0a469babea91c9c4f0b9b2.setContent(html_7cc2cb5fffff447fa4c061544de4488f);
            

            circle_marker_b889fbe7a6bb440984097554774023d3.bindPopup(popup_a312dd922a0a469babea91c9c4f0b9b2)
            ;

            
        
    
        var marker_107693e9da2549558c2c08c5a9f91a0f = L.marker(
            [41.399945, 2.164389],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_5fe74d5bfca647d1b8b2863ae474529c = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'orange',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_107693e9da2549558c2c08c5a9f91a0f.setIcon(icon_5fe74d5bfca647d1b8b2863ae474529c);
            
    
            var popup_a8cab2e15cc84016ac9386746e9e4c03 = L.popup({maxWidth: '300'
            
            });

            
                var html_b1a3ed0b840749ab8d5a31a18b88507e = $('<div id="html_b1a3ed0b840749ab8d5a31a18b88507e" style="width: 100.0%; height: 100.0%;">224- C /GIRONA, 176::236.0</div>')[0];
                popup_a8cab2e15cc84016ac9386746e9e4c03.setContent(html_b1a3ed0b840749ab8d5a31a18b88507e);
            

            marker_107693e9da2549558c2c08c5a9f91a0f.bindPopup(popup_a8cab2e15cc84016ac9386746e9e4c03)
            ;

            
        
    
            var circle_marker_583fefe99206455981c75650328ccc9b = L.circleMarker(
                [41.40694, 2.155794],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_574a206e71f148fcbe77f857e556b3ad = L.popup({maxWidth: '300'
            
            });

            
                var html_a736f6c9d8424e56ba86ed6ec55b49f6 = $('<div id="html_a736f6c9d8424e56ba86ed6ec55b49f6" style="width: 100.0%; height: 100.0%;">228 - PL.DEL NORD, 5</div>')[0];
                popup_574a206e71f148fcbe77f857e556b3ad.setContent(html_a736f6c9d8424e56ba86ed6ec55b49f6);
            

            circle_marker_583fefe99206455981c75650328ccc9b.bindPopup(popup_574a206e71f148fcbe77f857e556b3ad)
            ;

            
        
    
        var marker_6a73453597f1460fa48024273e2bffbc = L.marker(
            [41.40694, 2.155794],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_e1693b65897348ed929977f542c46928 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'red',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_6a73453597f1460fa48024273e2bffbc.setIcon(icon_e1693b65897348ed929977f542c46928);
            
    
            var popup_d5a3bf29e5064d4183b7fc44694b5301 = L.popup({maxWidth: '300'
            
            });

            
                var html_b1ab408e5452403da5bd48850fed8da7 = $('<div id="html_b1ab408e5452403da5bd48850fed8da7" style="width: 100.0%; height: 100.0%;">228 - PL.DEL NORD, 5::132.0</div>')[0];
                popup_d5a3bf29e5064d4183b7fc44694b5301.setContent(html_b1ab408e5452403da5bd48850fed8da7);
            

            marker_6a73453597f1460fa48024273e2bffbc.bindPopup(popup_d5a3bf29e5064d4183b7fc44694b5301)
            ;

            
        
    
            var circle_marker_0ce94d376a3f4157ae791ae3c38407f6 = L.circleMarker(
                [41.405107, 2.156874],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_280035ca626f4074a7867abd58bf384e = L.popup({maxWidth: '300'
            
            });

            
                var html_b3e44f60e4aa4f7e87f8afc1867fed78 = $('<div id="html_b3e44f60e4aa4f7e87f8afc1867fed78" style="width: 100.0%; height: 100.0%;">229 - C/ DE LA SANTACREU, 2 (PL.DE LA VIRREINA)</div>')[0];
                popup_280035ca626f4074a7867abd58bf384e.setContent(html_b3e44f60e4aa4f7e87f8afc1867fed78);
            

            circle_marker_0ce94d376a3f4157ae791ae3c38407f6.bindPopup(popup_280035ca626f4074a7867abd58bf384e)
            ;

            
        
    
        var marker_6ec0a88d6d1546028d1ad3b742a5203c = L.marker(
            [41.405107, 2.156874],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_eb728e21c1644d39bbb9f1059d949f41 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'orange',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_6ec0a88d6d1546028d1ad3b742a5203c.setIcon(icon_eb728e21c1644d39bbb9f1059d949f41);
            
    
            var popup_35a66624c8e04ad79885ef48b3385cea = L.popup({maxWidth: '300'
            
            });

            
                var html_75a035ef177b4fb19f7088107def74dc = $('<div id="html_75a035ef177b4fb19f7088107def74dc" style="width: 100.0%; height: 100.0%;">229 - C/ DE LA SANTACREU, 2 (PL.DE LA VIRREINA)::246.0</div>')[0];
                popup_35a66624c8e04ad79885ef48b3385cea.setContent(html_75a035ef177b4fb19f7088107def74dc);
            

            marker_6ec0a88d6d1546028d1ad3b742a5203c.bindPopup(popup_35a66624c8e04ad79885ef48b3385cea)
            ;

            
        
    
            var circle_marker_308899a02c5343e3b7fb1e7d088f1d54 = L.circleMarker(
                [41.407202, 2.163476],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_0fadae42d0644d01b529613e099e2f62 = L.popup({maxWidth: '300'
            
            });

            
                var html_b76ce519adc148e28ba2a3470667a02e = $('<div id="html_b76ce519adc148e28ba2a3470667a02e" style="width: 100.0%; height: 100.0%;">231- C/PI I MARGALL, 38</div>')[0];
                popup_0fadae42d0644d01b529613e099e2f62.setContent(html_b76ce519adc148e28ba2a3470667a02e);
            

            circle_marker_308899a02c5343e3b7fb1e7d088f1d54.bindPopup(popup_0fadae42d0644d01b529613e099e2f62)
            ;

            
        
    
        var marker_54a64402bd194a0491b868a41e2f96d3 = L.marker(
            [41.407202, 2.163476],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_d112d3b198bf4d01b190aa8ece04b077 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'red',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_54a64402bd194a0491b868a41e2f96d3.setIcon(icon_d112d3b198bf4d01b190aa8ece04b077);
            
    
            var popup_894ba2a604a84a9a8e254bd6dc7c3d1b = L.popup({maxWidth: '300'
            
            });

            
                var html_1f71133a44a74c159e9122a3089f45a5 = $('<div id="html_1f71133a44a74c159e9122a3089f45a5" style="width: 100.0%; height: 100.0%;">231- C/PI I MARGALL, 38::184.0</div>')[0];
                popup_894ba2a604a84a9a8e254bd6dc7c3d1b.setContent(html_1f71133a44a74c159e9122a3089f45a5);
            

            marker_54a64402bd194a0491b868a41e2f96d3.bindPopup(popup_894ba2a604a84a9a8e254bd6dc7c3d1b)
            ;

            
        
    
            var circle_marker_b45971a45c434c958f9c010f7cf2549f = L.circleMarker(
                [41.395, 2.16123],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_23380722c35f4c559daaefa471578198 = L.popup({maxWidth: '300'
            
            });

            
                var html_7fa254678ba14fb98f4b9ed1c2d39484 = $('<div id="html_7fa254678ba14fb98f4b9ed1c2d39484" style="width: 100.0%; height: 100.0%;">374 - PG. DE GRÀCIA, 89</div>')[0];
                popup_23380722c35f4c559daaefa471578198.setContent(html_7fa254678ba14fb98f4b9ed1c2d39484);
            

            circle_marker_b45971a45c434c958f9c010f7cf2549f.bindPopup(popup_23380722c35f4c559daaefa471578198)
            ;

            
        
    
        var marker_ffe30c79b54841a88e4d42e3df183f6d = L.marker(
            [41.395, 2.16123],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_e8903e531d664ec58e7d53e4ffef43e0 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'red',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_ffe30c79b54841a88e4d42e3df183f6d.setIcon(icon_e8903e531d664ec58e7d53e4ffef43e0);
            
    
            var popup_ebba61010c5e4d88bfa20a1dc084120c = L.popup({maxWidth: '300'
            
            });

            
                var html_e70f6128fc3a421c871602d14c4c430a = $('<div id="html_e70f6128fc3a421c871602d14c4c430a" style="width: 100.0%; height: 100.0%;">374 - PG. DE GRÀCIA, 89::167.0</div>')[0];
                popup_ebba61010c5e4d88bfa20a1dc084120c.setContent(html_e70f6128fc3a421c871602d14c4c430a);
            

            marker_ffe30c79b54841a88e4d42e3df183f6d.bindPopup(popup_ebba61010c5e4d88bfa20a1dc084120c)
            ;

            
        
    
            var circle_marker_84a97642112c42b6854319ee566713fb = L.circleMarker(
                [41.40542, 2.142456],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_5f0445e9993344ff9cfff743dd289c2e = L.popup({maxWidth: '300'
            
            });

            
                var html_6bab3e73aeb9485d9d8265477346ea46 = $('<div id="html_6bab3e73aeb9485d9d8265477346ea46" style="width: 100.0%; height: 100.0%;">465 - (PK) RDA. DEL GENERAL MITRE, 203</div>')[0];
                popup_5f0445e9993344ff9cfff743dd289c2e.setContent(html_6bab3e73aeb9485d9d8265477346ea46);
            

            circle_marker_84a97642112c42b6854319ee566713fb.bindPopup(popup_5f0445e9993344ff9cfff743dd289c2e)
            ;

            
        
    
        var marker_377dad61bfa14388a6d3b8ccc1208d90 = L.marker(
            [41.40542, 2.142456],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_2e0bdc4233254472bbf19b985d15ed0b = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'blue',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_377dad61bfa14388a6d3b8ccc1208d90.setIcon(icon_2e0bdc4233254472bbf19b985d15ed0b);
            
    
            var popup_7b04070791cf4da48d4c25f72bf56957 = L.popup({maxWidth: '300'
            
            });

            
                var html_316bfeb038bb48e597b6fc41c0fab9ff = $('<div id="html_316bfeb038bb48e597b6fc41c0fab9ff" style="width: 100.0%; height: 100.0%;">465 - (PK) RDA. DEL GENERAL MITRE, 203::266.0</div>')[0];
                popup_7b04070791cf4da48d4c25f72bf56957.setContent(html_316bfeb038bb48e597b6fc41c0fab9ff);
            

            marker_377dad61bfa14388a6d3b8ccc1208d90.bindPopup(popup_7b04070791cf4da48d4c25f72bf56957)
            ;

            
        
    
            var circle_marker_32b4ead969414845b0d71922ffee3710 = L.circleMarker(
                [41.3989, 2.153668],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_63d0a455c94c44c8b0ddfb136b1d8d8e = L.popup({maxWidth: '300'
            
            });

            
                var html_8624dea7b8664c0bb9359c09643a9197 = $('<div id="html_8624dea7b8664c0bb9359c09643a9197" style="width: 100.0%; height: 100.0%;">474 - (PK) PL. GAL·LA PLACÍDIA, 2</div>')[0];
                popup_63d0a455c94c44c8b0ddfb136b1d8d8e.setContent(html_8624dea7b8664c0bb9359c09643a9197);
            

            circle_marker_32b4ead969414845b0d71922ffee3710.bindPopup(popup_63d0a455c94c44c8b0ddfb136b1d8d8e)
            ;

            
        
    
        var marker_15604df242a149eba5c58acbc407d5d9 = L.marker(
            [41.3989, 2.153668],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_b74de9f7a40a4dcb96ee01d990bc0790 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'red',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_15604df242a149eba5c58acbc407d5d9.setIcon(icon_b74de9f7a40a4dcb96ee01d990bc0790);
            
    
            var popup_b35a717781fc4b6692ab230c716f17bf = L.popup({maxWidth: '300'
            
            });

            
                var html_96a45861c2ea484b81a3417bed1a9c00 = $('<div id="html_96a45861c2ea484b81a3417bed1a9c00" style="width: 100.0%; height: 100.0%;">474 - (PK) PL. GAL·LA PLACÍDIA, 2::57.0</div>')[0];
                popup_b35a717781fc4b6692ab230c716f17bf.setContent(html_96a45861c2ea484b81a3417bed1a9c00);
            

            marker_15604df242a149eba5c58acbc407d5d9.bindPopup(popup_b35a717781fc4b6692ab230c716f17bf)
            ;

            
        
    
            var circle_marker_60977951ef694ac29f6fd75dcc52127f = L.circleMarker(
                [41.396934, 2.159445],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_5a563dcfed1a41d59a32890ffe9a47d7 = L.popup({maxWidth: '300'
            
            });

            
                var html_7a77f88a51314a6ab4ce7061ecde1a24 = $('<div id="html_7a77f88a51314a6ab4ce7061ecde1a24" style="width: 100.0%; height: 100.0%;">486 - (PK) PG. DE GRÀCIA - DIAGONAL</div>')[0];
                popup_5a563dcfed1a41d59a32890ffe9a47d7.setContent(html_7a77f88a51314a6ab4ce7061ecde1a24);
            

            circle_marker_60977951ef694ac29f6fd75dcc52127f.bindPopup(popup_5a563dcfed1a41d59a32890ffe9a47d7)
            ;

            
        
    
        var marker_996fde0ab97d46c6acd78054c70de50d = L.marker(
            [41.396934, 2.159445],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_c40daf6959f04b778bd78c6465d4356c = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'blue',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_996fde0ab97d46c6acd78054c70de50d.setIcon(icon_c40daf6959f04b778bd78c6465d4356c);
            
    
            var popup_798ad9b66b5148669e00b97b68553edd = L.popup({maxWidth: '300'
            
            });

            
                var html_1c84136aff3041cd95f4e85428b0b1aa = $('<div id="html_1c84136aff3041cd95f4e85428b0b1aa" style="width: 100.0%; height: 100.0%;">486 - (PK) PG. DE GRÀCIA - DIAGONAL::266.0</div>')[0];
                popup_798ad9b66b5148669e00b97b68553edd.setContent(html_1c84136aff3041cd95f4e85428b0b1aa);
            

            marker_996fde0ab97d46c6acd78054c70de50d.bindPopup(popup_798ad9b66b5148669e00b97b68553edd)
            ;

            
        
    
            var circle_marker_20cfc2379e944615be4e5e08425b5ab2 = L.circleMarker(
                [41.398373, 2.153154],
                {
  "bubblingMouseEvents": true,
  "color": "#3388ff",
  "dashArray": null,
  "dashOffset": null,
  "fill": true,
  "fillColor": "#3db7e4",
  "fillOpacity": 0.2,
  "fillRule": "evenodd",
  "lineCap": "round",
  "lineJoin": "round",
  "opacity": 1.0,
  "radius": 15,
  "stroke": true,
  "weight": 3
}
                )
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
            
    
            var popup_84d7499f76cd4ab6ae553dc939c82027 = L.popup({maxWidth: '300'
            
            });

            
                var html_f1e15a802ce1419590db3e4c61b27710 = $('<div id="html_f1e15a802ce1419590db3e4c61b27710" style="width: 100.0%; height: 100.0%;">107 - TRAV. DE GRÀCIA 92 / VIA AUGUSTA</div>')[0];
                popup_84d7499f76cd4ab6ae553dc939c82027.setContent(html_f1e15a802ce1419590db3e4c61b27710);
            

            circle_marker_20cfc2379e944615be4e5e08425b5ab2.bindPopup(popup_84d7499f76cd4ab6ae553dc939c82027)
            ;

            
        
    
        var marker_0649ef54b50148b3a2f0653fa262d71b = L.marker(
            [41.398373, 2.153154],
            {
                icon: new L.Icon.Default()
                }
            ).addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
    

                var icon_7c3cec38bedd4534ae065e4edc23e570 = L.AwesomeMarkers.icon({
                    icon: 'info-sign',
                    iconColor: 'white',
                    markerColor: 'orange',
                    prefix: 'glyphicon',
                    extraClasses: 'fa-rotate-0'
                    });
                marker_0649ef54b50148b3a2f0653fa262d71b.setIcon(icon_7c3cec38bedd4534ae065e4edc23e570);
            
    
            var popup_01b849c2370a4a59b43e460849c22456 = L.popup({maxWidth: '300'
            
            });

            
                var html_5ac13f255b0a450d9cd01c959d448c62 = $('<div id="html_5ac13f255b0a450d9cd01c959d448c62" style="width: 100.0%; height: 100.0%;">107 - TRAV. DE GRÀCIA 92 / VIA AUGUSTA::204.0</div>')[0];
                popup_01b849c2370a4a59b43e460849c22456.setContent(html_5ac13f255b0a450d9cd01c959d448c62);
            

            marker_0649ef54b50148b3a2f0653fa262d71b.bindPopup(popup_01b849c2370a4a59b43e460849c22456)
            ;

            
        
    
            var heat_map_aa07254fbe13437aae717ac9e1034219 = L.heatLayer(
                [[41.403309859099664, 2.1521850379155514], [41.40181954961136, 2.15212354191599], [41.40252399485162, 2.1541074417935593], [41.4024991371333, 2.1541473478159165], [41.40172818421211, 2.151513949455388], [41.401073322999046, 2.152520908777154], [41.40381775999537, 2.153006325000218], [41.4024752234313, 2.151270034155539], [41.40341477324329, 2.151849548113032], [41.40177956014948, 2.1521191904077686], [41.40124150335853, 2.1522693271467213], [41.40199086812038, 2.1527794968345293], [41.402512501976396, 2.1535180082218726], [41.40234178189931, 2.15393102246908], [41.40336871641285, 2.151830882377483], [41.402704144394, 2.1538624165962013], [41.40269006227144, 2.151228468035271], [41.40256584269451, 2.151930942148317], [41.40390440744452, 2.15376289185034], [41.40152557431332, 2.1512047908509038], [41.40417861419942, 2.1527943320111484], [41.40402335463647, 2.151119956383269], [41.40088556429481, 2.152851771627446], [41.403870996081174, 2.150928507050988], [41.401154864381525, 2.15093501258914], [41.402868347182576, 2.152039864814522], [41.40321518089324, 2.1514661614552417], [41.40246960867502, 2.1528308590154253], [41.401799655887125, 2.152605100637584], [41.40093041069562, 2.1519797122827447], [41.402084030009235, 2.1512196531035515], [41.40170678059006, 2.153326507679404], [41.40235308812968, 2.152975855356589], [41.4041155316387, 2.1520267345246813], [41.402720879373355, 2.152718839014712], [41.403046567443745, 2.151612466144095], [41.40261408553937, 2.151993191464785], [41.40210018359861, 2.1527742188234473], [41.40305347167135, 2.1521741508497083], [41.40399406740089, 2.1518146794810615], [41.40103476894876, 2.1522192298753384], [41.40129071027865, 2.1522980517433], [41.40274162137902, 2.151238841249528], [41.40187316629966, 2.1525639023204146], [41.403987123712064, 2.151623502839703], [41.402817730451574, 2.1532532558441098], [41.404155084888096, 2.15295941733257], [41.40217456715852, 2.1538625192274243], [41.40260855718433, 2.151188402016893], [41.40266261046022, 2.152361272798773], [41.40353899335425, 2.152716049697949], [41.40266948635823, 2.1519659240716846], [41.404156528915756, 2.1528235973870853], [41.401731335938486, 2.152025437248911], [41.40411070140945, 2.151587025023517], [41.40147778078129, 2.1525516907510607], [41.403271654594185, 2.1508616823957487], [41.404030328760825, 2.1514030851209665], [41.40155101875454, 2.151074085950952], [41.40091882971642, 2.1532590570671237], [41.4039593567602, 2.153612700229997], [41.40405361120166, 2.1529243303049137], [41.40132935993195, 2.1516978394780106], [41.401842912754105, 2.152474056872414], [41.40105745669867, 2.153977480323412], [41.4009547156994, 2.1514634633813094], [41.40329367798736, 2.1522563297388175], [41.40331321399714, 2.1528011439396715], [41.400958602820396, 2.1520950056345742], [41.40228665792718, 2.153062171471235], [41.40094474979203, 2.152455803283539], [41.40400155982582, 2.1541178104013183], [41.40161977442402, 2.152814286109641], [41.401867928814596, 2.1525530656931147], [41.402590963519884, 2.1533239412220553], [41.40310179366495, 2.152184115906431], [41.40176845107955, 2.153077086353103], [41.400899877818006, 2.153357635233846], [41.40158017043975, 2.1522692218565855], [41.40381717906632, 2.1513015174229735], [41.40088235104758, 2.154005044038152], [41.402627624984504, 2.153853674234475], [41.402233290484624, 2.152979707383645], [41.40412292917655, 2.1539464135892215], [41.40122264666583, 2.152170438273102], [41.40235304122522, 2.150979916553486], [41.401142553465, 2.153533205497393], [41.40207799361685, 2.1529830395492566], [41.40121674401017, 2.1537512181171015], [41.40265310305926, 2.1524354717826317], [41.401039735239415, 2.1515744426298147], [41.403409986164725, 2.151757958389885], [41.40240721329135, 2.151173348845519], [41.39845526958453, 2.1600417013435527], [41.398887316056424, 2.1592022007328113], [41.397721637933884, 2.159918250177366], [41.397963808016335, 2.1597314494859376], [41.397506523568325, 2.160667273366283], [41.398988384831306, 2.1605066748618045], [41.39899389806905, 2.1605099979967126], [41.39788912150108, 2.1591304805727973], [41.39908976089943, 2.1597906972673533], [41.39892867515554, 2.1603283239762527], [41.398403626903374, 2.160341477506525], [41.3986985218633, 2.1591323098533475], [41.39869578391833, 2.1592693094822986], [41.39832566270166, 2.159424050239344], [41.398897677872256, 2.1599083091265494], [41.39899567368082, 2.1600596583352503], [41.39804112468801, 2.1602091264061993], [41.3977754819771, 2.159688603810698], [41.39752680164683, 2.1595846562226355], [41.39788617236652, 2.1598442688316624], [41.39810686631593, 2.160478472220072], [41.397909350747106, 2.1600366269461895], [41.39862942549333, 2.160487286856497], [41.39860592488847, 2.159573464738476], [41.39802638638719, 2.1592792524463165], [41.39903067995754, 2.160534915297682], [41.397789173246295, 2.1591086853580297], [41.399045663980644, 2.1590828701545806], [41.39777558157899, 2.160026455088131], [41.398850168226325, 2.160347902477521], [41.39750693753295, 2.1605865880381523], [41.39859822731395, 2.160192728474962], [41.398515147924726, 2.159137438310477], [41.39892032054575, 2.1604667337572607], [41.39857024920867, 2.160390519439808], [41.39886806379018, 2.160019802427627], [41.39905459361096, 2.159976627996302], [41.39881288820952, 2.159700473387994], [41.398813563005916, 2.160089926738341], [41.3984131348692, 2.159656927539897], [41.39906257560975, 2.159263556882262], [41.39826861569694, 2.159314351880283], [41.39853797771344, 2.1597728035282255], [41.397760596130006, 2.1600080324946775], [41.39752043784951, 2.1592903500027343], [41.39779459469743, 2.1602170099896014], [41.39749917864044, 2.1594099751736295], [41.397901273813936, 2.1603910651676452], [41.39851668537966, 2.159613084176475], [41.398812490208826, 2.159079877952134], [41.39822565157083, 2.1595071720436487], [41.397626731453286, 2.1595492481624765], [41.39901858996959, 2.159427258676234], [41.39761359410468, 2.1599956084678262], [41.3984215985504, 2.1604235356710095], [41.397592319941865, 2.1596856350430125], [41.39859310733627, 2.1605259506480636], [41.39882529676395, 2.1604878403761796], [41.39909538326118, 2.1593648509554915], [41.398443789647786, 2.1599968991804297], [41.397588027764144, 2.1601374577931027], [41.39751603247476, 2.1592881472772376], [41.398883385545936, 2.1594683958345153], [41.398049210523, 2.160471134625439], [41.3975199217682, 2.159922342651037], [41.397739063382176, 2.1597215663168856], [41.39759096148397, 2.159741159967619], [41.39842435373546, 2.160357026162134], [41.39778771112276, 2.160262151243042], [41.398416206883475, 2.1604235691283673], [41.39873324759809, 2.1603899937470374], [41.398087351809494, 2.1595620197755605], [41.398139335142346, 2.1600898854864568], [41.39804724331322, 2.1603875571077817], [41.404277614004535, 2.1604788489831694], [41.40354022176881, 2.161782925726851], [41.40277761000748, 2.1609814455718057], [41.403309595976324, 2.161572046871532], [41.403176066923606, 2.161427925735381], [41.40300683057193, 2.16143823101957], [41.40360114412787, 2.1612944735960675], [41.40319490323098, 2.161174880291111], [41.40366809731147, 2.1606079227618324], [41.40382755050379, 2.161539407059219], [41.40404428527836, 2.1611835844984975], [41.40351207139945, 2.1607819395138455], [41.40360824118682, 2.161674548636279], [41.40291398076509, 2.161770900975857], [41.403143746088816, 2.161445119040361], [41.404237142591136, 2.160566502814751], [41.40380727231513, 2.1603514947280424], [41.40414644982953, 2.1607824057016782], [41.403098671339016, 2.1608763444334693], [41.40391135561831, 2.160673685319634], [41.40421541882773, 2.161732790871693], [41.403285742810176, 2.1604848567354327], [41.40360493966746, 2.1606563643028105], [41.40363060325563, 2.1609046367141103], [41.403116840226744, 2.161845915027155], [41.403876040313456, 2.161829363732375], [41.40281887899247, 2.1608679660259753], [41.40287228674918, 2.1613825961881017], [41.40318457528152, 2.1612096254402737], [41.40354901308208, 2.1613204831129766], [41.402891778446396, 2.1617114310539796], [41.40328677337129, 2.1607532750930716], [41.4031419678135, 2.160655216811301], [41.40316802267215, 2.1608813816567887], [41.40261715707497, 2.161936252705061], [41.40279912020073, 2.160489616509368], [41.40315786963061, 2.161284004987996], [41.40321814246047, 2.16165343882502], [41.40394830842676, 2.1603827971282055], [41.404000626274545, 2.1609410792105948], [41.40349742220965, 2.161742156531714], [41.40302704639066, 2.160290756903996], [41.4033455822812, 2.1605267943353055], [41.40340956376378, 2.161169156336684], [41.402637922285685, 2.1612786883277955], [41.40301648255696, 2.160656493412763], [41.403566171787624, 2.160928304512482], [41.40348227197941, 2.1613133841985634], [41.40300639573747, 2.16062449969351], [41.40374616065591, 2.1612790357395673], [41.40393801941969, 2.1603297585369896], [41.40382960554657, 2.161630288719332], [41.40274130176612, 2.1614929115649364], [41.40334256376648, 2.161068308724775], [41.40361554872653, 2.160485399843948], [41.40385925191359, 2.16051197285974], [41.403091231771505, 2.160528020287294], [41.40261655308726, 2.1603753254687064], [41.40379594513486, 2.161296387470128], [41.40335843540264, 2.161732152785018], [41.403650818514684, 2.161268493808056], [41.40331733812931, 2.1605547141737054], [41.40331903185501, 2.1602897454278334], [41.40397491056315, 2.161311575471427], [41.40361534595212, 2.160441087105113], [41.403722702898364, 2.1615504508065366], [41.40370165395605, 2.1603105982851725], [41.40353313929478, 2.1606347505057655], [41.40317278807604, 2.16168835372347], [41.40305732370609, 2.160306594962537], [41.40279976631259, 2.161311614169262], [41.403557078602994, 2.161422784265717], [41.40390808696355, 2.16193391274229], [41.404015035282555, 2.1619177859890084], [41.40297931579231, 2.161933295575228], [41.40644152611839, 2.1519194173459706], [41.4068324006764, 2.151878882429393], [41.40677041089853, 2.1526170727710214], [41.40523963844399, 2.1516720566421883], [41.4060321591056, 2.1526426365817914], [41.40586172668308, 2.1522331471282574], [41.40659198157717, 2.1520444474285703], [41.40705342226779, 2.151259072287588], [41.404709726800675, 2.150735259875619], [41.40551423326826, 2.152634048327058], [41.405252074544606, 2.151771680957149], [41.40477981526898, 2.151590169675974], [41.40701283455285, 2.150836752263629], [41.407185694014494, 2.1523331210116803], [41.40513925046628, 2.1524246729872063], [41.406911878734434, 2.1520188458390153], [41.405538060026714, 2.152651167412334], [41.405940419092055, 2.1517260618024574], [41.405479796690216, 2.150374795276415], [41.406644289674844, 2.1504992834176893], [41.40636081051026, 2.1528899723712045], [41.406326353493164, 2.151280764429729], [41.40652722440244, 2.151729134255275], [41.40653974425502, 2.1519454603000234], [41.40618713341051, 2.1504107226743714], [41.40486045420844, 2.1518275344403506], [41.40726417155395, 2.1520832434857997], [41.40695561563293, 2.1521543159434855], [41.40559495309653, 2.1511927583035937], [41.40621206454818, 2.1507862272856006], [41.40501207728905, 2.151357738386269], [41.404762698272975, 2.1515850981294835], [41.40501785216648, 2.1503735997921773], [41.40524036693033, 2.1527940007381647], [41.40504684000262, 2.1510030026142757], [41.40677979085238, 2.152652786328624], [41.404905346887986, 2.1511993143985033], [41.40471576495196, 2.150553812259905], [41.405944585930115, 2.15226664660277], [41.40639378616196, 2.15181253413186], [41.405103750419705, 2.152732483450278], [41.40507989574046, 2.1522883051765693], [41.40523540706458, 2.1519829395939643], [41.40566008262545, 2.1520802289750076], [41.40630663644352, 2.1518127365475697], [41.40730363091751, 2.1509614495941136], [41.405481680895164, 2.151363717488739], [41.40646047924659, 2.151350667592398], [41.40493934924103, 2.152505660546456], [41.40631022032517, 2.1509506263542675], [41.40670512395795, 2.1508201053045353], [41.40667700467811, 2.150760447165571], [41.40620750954298, 2.1508710242221345], [41.404795432278924, 2.151693480836934], [41.4063557714409, 2.1505827441941396], [41.40624610199736, 2.1520185857306133], [41.40659964950556, 2.150857831957577], [41.40570571130774, 2.1508838908004315], [41.40556281756326, 2.1511403170102437], [41.405400842342985, 2.151730637162761], [41.406333939685645, 2.152369254246332], [41.407131130460904, 2.1503554077966176], [41.405819832891616, 2.152937945473346], [41.40572468475542, 2.1522132973194563], [41.40473076218164, 2.1517909598390594], [41.404884266125734, 2.1516170947244873], [41.40642695970224, 2.152789771740792], [41.40606033484315, 2.1504637625740535], [41.40683841813715, 2.151572716596562], [41.405330891808774, 2.151626874309297], [41.406657494927735, 2.1528795379889623], [41.40631343510872, 2.1521315960859777], [41.40472173141101, 2.151525655514101], [41.40615503570398, 2.151889266908237], [41.4065871979082, 2.1517801898137], [41.40477449925095, 2.15229022573639], [41.40605162737675, 2.1524542437965426], [41.406089952294465, 2.151638799138341], [41.40706101672839, 2.1520992366977265], [41.406088634698115, 2.1512833223495385], [41.407269183807614, 2.152640970329308], [41.4067075478512, 2.1518216234523724], [41.405144279740085, 2.152733811790108], [41.40492308723074, 2.1526254702270213], [41.405522416986635, 2.1523511643000814], [41.40634349945524, 2.1526062160735444], [41.40669716346892, 2.1527099085539354], [41.38639809698555, 2.150970269147942], [41.385822930721964, 2.152737671378118], [41.38488459525908, 2.152774157910541], [41.38657684695827, 2.1514389388174915], [41.385159832201744, 2.151141112745494], [41.38575469921295, 2.1511700879373334], [41.384982910329924, 2.152934261649244], [41.38582309804549, 2.1512332857614065], [41.384417102200835, 2.1517587642996188], [41.38672257933406, 2.150851983738501], [41.38598027412942, 2.1507583340722563], [41.38633995928271, 2.1513715477590383], [41.38468209446866, 2.153132476469322], [41.3857689154052, 2.1532231853771884], [41.386492824736635, 2.15301513561729], [41.385738847025436, 2.1508664234674426], [41.38529118663718, 2.1532469385133295], [41.38656279159795, 2.1509432728013502], [41.38524994619641, 2.1516252640299682], [41.386057015601615, 2.153105744200442], [41.38671475210165, 2.1515916846743965], [41.38568962184647, 2.1516766050160627], [41.3849343474301, 2.1524276399875464], [41.38508463962558, 2.1516747581157696], [41.38667630544541, 2.153028560885883], [41.385216623232814, 2.153248882490933], [41.38493234908667, 2.152565956193447], [41.38568507644199, 2.151949523276346], [41.38526167093402, 2.1519775028305745], [41.384273057333886, 2.1509672140562106], [41.38546706537274, 2.153252194101209], [41.38514199888721, 2.1527890078431753], [41.385818661512054, 2.151625854528319], [41.385080556485605, 2.1518867213761426], [41.38455433001542, 2.1507757439797692], [41.38548587746705, 2.151758841183193], [41.38536924094048, 2.1513656534401044], [41.38493583217013, 2.1523592692675146], [41.384550097526635, 2.1507674913910857], [41.384386044760646, 2.153014983744717], [41.385357238435475, 2.1531802089256122], [41.38448743920101, 2.15152951426539], [41.38458645347369, 2.1510711429739313], [41.38649142078677, 2.152652755978154], [41.38542762925847, 2.1522168634540226], [41.38492857616967, 2.151719033703386], [41.38636967297341, 2.1530514892178063], [41.38522724854057, 2.150815542747494], [41.3853117299056, 2.1515637350066137], [41.38639052336978, 2.152038982038518], [41.38530477758146, 2.152584975228786], [41.386380794617686, 2.153056780103792], [41.38439620478474, 2.1510321112738735], [41.3859219635179, 2.151042646089798], [41.38585033233765, 2.152797041207778], [41.38629966608191, 2.151306963716314], [41.38668349858138, 2.1526147115284444], [41.38458288996203, 2.151400457961625], [41.385664879892595, 2.152082563382859], [41.38472596756561, 2.1514746731424883], [41.38619413465804, 2.1512629250173054], [41.3860645406936, 2.150842674209626], [41.38437786149917, 2.152431605254636], [41.38573002974374, 2.1514191443015647], [41.38487535740216, 2.152360648357365], [41.38609724855852, 2.1514443434304966], [41.38519228781621, 2.1514377581411157], [41.38438153657525, 2.152083008900039], [41.38427762947717, 2.151519406166777], [41.38473047924632, 2.1520797684905353], [41.38576914232958, 2.151454129899747], [41.38439487840945, 2.1509091543109213], [41.38531710530126, 2.1514099005938205], [41.38625086436089, 2.152056060777713], [41.38448734035936, 2.153228615508118], [41.38664319928271, 2.1531335244913783], [41.38465172088706, 2.152453144670268], [41.386628017204885, 2.151658553389696], [41.38533503054039, 2.1522743831216444], [41.38610250968993, 2.1523182310849003], [41.3853700946535, 2.1511800759111437], [41.38509395618504, 2.152102545189919], [41.384440120492116, 2.1526452318616944], [41.38466432307042, 2.15112349852162], [41.386315666989226, 2.1526432235990502], [41.4090644764057, 2.15082114756546], [41.40758974096342, 2.152086842456519], [41.40723332911488, 2.150952124912809], [41.406050742373644, 2.149649640033535], [41.40859271100594, 2.1506990505650694], [41.40900968204539, 2.153311901871669], [41.40774326706659, 2.151513481168723], [41.40662984393028, 2.1505849150755396], [41.40731671599635, 2.1505490831054623], [41.406723014599805, 2.1510260838859647], [41.406866865972226, 2.1518094295581727], [41.40829401431292, 2.1528608742990896], [41.40602666214014, 2.15164928017658], [41.40675276084461, 2.151335774475861], [41.40583911399572, 2.1502755864108023], [41.406068756305686, 2.1494067639432384], [41.40561031732419, 2.150539623561849], [41.407750046435275, 2.149612573752862], [41.40865842101285, 2.1494830196053516], [41.40896858184225, 2.1500411074863033], [41.40884832735874, 2.1503621139642757], [41.40861945767488, 2.150364483248976], [41.40865794933562, 2.1497311341918026], [41.407078445297714, 2.1520707800208076], [41.406660427719885, 2.150245343654168], [41.405520095139074, 2.152102978102844], [41.40636485340515, 2.152332741351455], [41.40898990264832, 2.150434495593709], [41.406744125634376, 2.1527730071374216], [41.40917602130824, 2.152916476435834], [41.40665966674197, 2.1529771943157145], [41.4066488343029, 2.151925887422803], [41.40599915262918, 2.151489090634049], [41.40693874873123, 2.150233749447014], [41.40800167429386, 2.14983332180792], [41.40694817967958, 2.1510619469031607], [41.40777241009424, 2.152486996155591], [41.40564691478877, 2.1521572024252973], [41.408006452653225, 2.15059167668007], [41.40598760126814, 2.1510718946451877], [41.40753025342291, 2.153144784497639], [41.406351368027664, 2.152082568135293], [41.406569801951264, 2.1510522771696916], [41.4093005212873, 2.1528710129882067], [41.40728544422018, 2.1530229168859427], [41.40840635774715, 2.1520889991371863], [41.40788488332105, 2.152677487848509], [41.4068013898334, 2.150086024916505], [41.409201989209755, 2.1532814135943377], [41.40683383967815, 2.151196775251741], [41.40555043958583, 2.150794298776757], [41.407792663642546, 2.151758693799443], [41.40662157605115, 2.1511027864833174], [41.407885234697645, 2.1511790726889912], [41.406217667243624, 2.1505024834076245], [41.40631071017763, 2.152445924391683], [41.4058148276367, 2.1497874708167544], [41.40832049046985, 2.151820592399873], [41.40729045998657, 2.14976196607203], [41.4060168541695, 2.150289280910973], [41.407918201855836, 2.1504055662077888], [41.40654389555119, 2.1509640072643625], [41.405566636449095, 2.149752371311225], [41.40679610698638, 2.149354366744953], [41.40923596255834, 2.1526191399825962], [41.40865455960387, 2.1529629762691287], [41.40736981897624, 2.1508051000230233], [41.40538887595449, 2.1529952839221873], [41.40916211048581, 2.1526781320308257], [41.40809455794592, 2.1527752376423837], [41.408692203186625, 2.153025933974529], [41.4068852902402, 2.1508793519840936], [41.4082713049766, 2.1520245281187154], [41.405593288200194, 2.152972051462695], [41.40758628409939, 2.1528237975449396], [41.406902501953056, 2.1529691550561605], [41.40609955452806, 2.1494591034279606], [41.4061716435809, 2.1499562009186275], [41.40822643664389, 2.1499577863439026], [41.406827658632004, 2.1516135024608802], [41.40765816838085, 2.1518998723309655], [41.405593990380574, 2.1498176167881664], [41.40795913971757, 2.152791135033763], [41.40739843171946, 2.1530122880138096], [41.4056323893208, 2.1498877385178785], [41.40822918101179, 2.150194264289694], [41.408565106134354, 2.150190699130879], [41.40697069474083, 2.1508519691156995], [41.40576930598777, 2.1495931109721482], [41.405974970743145, 2.1521683301400043], [41.40595546778112, 2.1500088977967247], [41.40669299957717, 2.1520915436416623], [41.4057804298561, 2.150534451142126], [41.40742780903431, 2.1512652054940142], [41.40596207090792, 2.1514942944496758], [41.40685802596177, 2.15074616998031], [41.406757361781416, 2.1522331469713856], [41.40922951362684, 2.1500255821778103], [41.40616764567058, 2.153209054209534], [41.40832521890961, 2.152318373389824], [41.401692363977595, 2.1625115386512825], [41.4015889136725, 2.160176480758623], [41.40227850317439, 2.1616141881365833], [41.40125445036982, 2.162498757204996], [41.40064375196943, 2.16254027490165], [41.401458291803735, 2.1604388831419135], [41.400965195878086, 2.1595264668616965], [41.402134113146055, 2.162636942722862], [41.40206235309173, 2.160356381948657], [41.40205792013363, 2.1596179963640294], [41.39979374234992, 2.159157958788122], [41.400037653612614, 2.1622382703103464], [41.40353096479848, 2.1601332463717355], [41.40318349875727, 2.1594019951852386], [41.40041715814241, 2.16088549020131], [41.40186418567178, 2.1623965302531474], [41.40094433164971, 2.1601627875332774], [41.4033795406082, 2.1606007973889154], [41.403110676328396, 2.162013410171255], [41.40024995100698, 2.159780814204227], [41.40365737463012, 2.16038158714599], [41.40268759712093, 2.1589576575235143], [41.40002627169764, 2.161457251994103], [41.40007087807684, 2.1621105224688457], [41.40261735731224, 2.162731528071257], [41.40281521666265, 2.1588294726115302], [41.400194617976894, 2.1622804479586204], [41.40051729972471, 2.16244631824359], [41.400430148211164, 2.162373448546154], [41.402375014239524, 2.1621306192673124], [41.402545031433924, 2.161149687946661], [41.40302588461524, 2.159476331102117], [41.40304288421597, 2.15952430377338], [41.402914706041656, 2.1606001058533955], [41.40198481815594, 2.159155383737232], [41.40173449038666, 2.1591401759899074], [41.39976118291463, 2.160825989362543], [41.4003044065289, 2.1597774896236483], [41.40199519371632, 2.159181654094821], [41.40143413456022, 2.1607328476568908], [41.40305360973829, 2.159860345823285], [41.40029625820713, 2.1627112207271013], [41.40066860841116, 2.159023385017818], [41.402529075370076, 2.160431015282612], [41.40337740392873, 2.161596792658619], [41.40313689507681, 2.160929990630885], [41.40353705815204, 2.162700959933349], [41.40152496774689, 2.161729195598278], [41.40180319039955, 2.1603357873002254], [41.40246215941571, 2.160656553363525], [41.400621630726874, 2.1605495356858815], [41.402062922134284, 2.162201868118134], [41.403587919155406, 2.160637020921058], [41.40264269351178, 2.162118907579622], [41.399943752455314, 2.160240744416638], [41.40364732993185, 2.161522834140067], [41.402326457517496, 2.1619821877096106], [41.40215623072723, 2.161473018569561], [41.402618721650065, 2.1604441683024938], [41.402725947938066, 2.16204092400555], [41.40026566793471, 2.158916310856001], [41.403374202073564, 2.1591536218236165], [41.40016457926327, 2.1618229180335193], [41.40338021529842, 2.1603233600066223], [41.40228260819478, 2.1599262689496563], [41.402732240806465, 2.160460065644676], [41.40154629189065, 2.158897402791057], [41.40253240537602, 2.1590131336354914], [41.40050680298047, 2.1605068192384036], [41.40054949468522, 2.162159616204635], [41.40271711732323, 2.159910927309682], [41.39979010644024, 2.1619257725337273], [41.400275587694004, 2.1594924738836645], [41.39982779814061, 2.1598967955152024], [41.399950723235136, 2.1593093824455987], [41.402976104814236, 2.1598971555700297], [41.40308803666979, 2.161152230209658], [41.403672596146706, 2.159555497677951], [41.40237290813808, 2.1601449400886263], [41.400125424041654, 2.16097163380595], [41.40143821929778, 2.161048011980262], [41.40104031397159, 2.1600172323596687], [41.40264361020152, 2.162368438653974], [41.403024299362976, 2.1603396499977285], [41.40014204419718, 2.159902161485765], [41.40252828692287, 2.162385487478122], [41.39983134635002, 2.1590671793309344], [41.40217306878345, 2.159292516122339], [41.40121557035502, 2.1622879879893877], [41.402914301663735, 2.1590955495017483], [41.40187442586569, 2.1606483428974195], [41.40357522647546, 2.1611864911852883], [41.400873394809146, 2.1602152414266254], [41.40239077096485, 2.162327143455709], [41.40208279794269, 2.1622652488801344], [41.40307859712762, 2.1621202538212065], [41.40013243850288, 2.1591237309589144], [41.40155640061185, 2.159340111085743], [41.40127061946413, 2.1598306456300733], [41.4005913662181, 2.1625659921795273], [41.38393028926368, 2.1518102204342404], [41.38455808172941, 2.151353231889968], [41.3858033270716, 2.1513730393938446], [41.38387568233463, 2.1517279929538025], [41.38395162382429, 2.1505839628674797], [41.384011456445386, 2.151536217530365], [41.38498460933166, 2.1509045940883564], [41.3848387613741, 2.149921810775092], [41.3855407606747, 2.150605607622149], [41.384767035637324, 2.151190219230952], [41.38395975720587, 2.1508858889001083], [41.38507556884559, 2.150269492942123], [41.384381037097135, 2.1503459328002346], [41.385179089963565, 2.1507073333673556], [41.38394305068935, 2.151742852619631], [41.385218938664735, 2.1517703874517284], [41.38578932599804, 2.1512314072126277], [41.38422576351659, 2.1497479055045603], [41.38511395983895, 2.150175400399138], [41.38421790249894, 2.151328270256411], [41.38424553831361, 2.1500846101764304], [41.3838246355796, 2.150224590328858], [41.384856933660565, 2.151533181783975], [41.38605450755226, 2.151804788936643], [41.384345016312764, 2.1503316607282557], [41.384494452247615, 2.1498315293860597], [41.38538942301937, 2.1509968864293487], [41.38398991021511, 2.1505139403178317], [41.383970282980044, 2.1499358862402955], [41.385250902147874, 2.1519500372146734], [41.38522097050899, 2.1499308600737366], [41.38594024789524, 2.1503360242453513], [41.385569126637996, 2.1496786965142287], [41.38607792848373, 2.150446672102504], [41.385189714738196, 2.1512760297605795], [41.385053336099745, 2.1517427013165147], [41.38504489787627, 2.1496769681141936], [41.38381471977092, 2.1504571237530348], [41.38407896601705, 2.151450881037276], [41.38438605276072, 2.151333730277656], [41.38404757281374, 2.1518497324136985], [41.38450546083624, 2.151136425991354], [41.3845604274104, 2.1514231311454677], [41.38514103258125, 2.1517535402667227], [41.38473382330806, 2.150989748744724], [41.385043188451576, 2.1507670594632122], [41.38408178411824, 2.149725009063788], [41.384591549677666, 2.15057088682769], [41.38452029809895, 2.1507265494720427], [41.38596655523638, 2.150789195243689], [41.386107328908, 2.1519130352886995], [41.38506430047135, 2.151046859912803], [41.38557863109135, 2.1516137820108816], [41.38451973754507, 2.1500888449888893], [41.38474100718659, 2.150033915351992], [41.385654122518716, 2.1501728243604443], [41.3850389355734, 2.151943360787425], [41.385503810471945, 2.1507013166168125], [41.38410300570509, 2.1503408494022964], [41.38485678238364, 2.149782147606375], [41.385262350019815, 2.151892543628514], [41.38579895891355, 2.149870522685095], [41.38439636703485, 2.1513176851018954], [41.384182450943904, 2.150763979937224], [41.385034535583365, 2.150492197540509], [41.3842926116573, 2.149652470639626], [41.385917116826406, 2.151190741889649], [41.38538699485498, 2.150371208494978], [41.383830433246985, 2.1511641755964437], [41.383927179152565, 2.1515104629051836], [41.38453302074247, 2.150859347845946], [41.38501150628814, 2.1510100132777343], [41.38387804054819, 2.1509283363949385], [41.38418465565478, 2.1500259546466167], [41.38508136414244, 2.151956016956066], [41.38416538348963, 2.1501729883633924], [41.38448238619017, 2.1505433216636693], [41.38462508366733, 2.151233251056251], [41.38400366883734, 2.1498287663619418], [41.38406446823931, 2.1516767920553566], [41.38574497527331, 2.151803587126755], [41.38522775427808, 2.151381204024877], [41.38453151242592, 2.1503395286417386], [41.39532783429778, 2.1576782630143083], [41.39482262342383, 2.1579860675890385], [41.39578678474565, 2.1568835287682386], [41.39492624608169, 2.1574111914855116], [41.39492413305222, 2.1577315021939207], [41.395904940752196, 2.157498579430298], [41.395063848929865, 2.1569286451214436], [41.3950111298865, 2.1574561423367666], [41.39490415348241, 2.157502773312913], [41.394941768955086, 2.1571303257615995], [41.394934550336274, 2.1569512622934237], [41.3947636773067, 2.157048739217961], [41.3947956264771, 2.1578984898550093], [41.394895259726326, 2.1568963717296854], [41.39535175483755, 2.157752549352411], [41.395430713679275, 2.1569755413370135], [41.39494738694469, 2.1567163853852964], [41.39513823152707, 2.1573463692978376], [41.395589754735425, 2.1578343755672647], [41.39577623326167, 2.1567807758720297], [41.395127715170894, 2.1569184395644756], [41.39477017687174, 2.1568909842676507], [41.39512107894621, 2.1567712987232612], [41.39473037157458, 2.156855355187201], [41.3955220200431, 2.1567558616413103], [41.39498903430368, 2.157625735069528], [41.3959084438074, 2.1576322356689133], [41.39520067584723, 2.1567291265478468], [41.395152034460196, 2.15744304918096], [41.39520413766349, 2.156887130299096], [41.39598742455084, 2.157229499982962], [41.39550900480644, 2.1567827869345657], [41.39503420617952, 2.157758002505526], [41.39582405488116, 2.157506167804096], [41.395391236219645, 2.1571699604189574], [41.39585780743692, 2.1570853382496167], [41.39479662670574, 2.1569313774743857], [41.39539418156373, 2.1578842539258916], [41.395435713354956, 2.1575227828818435], [41.395962058186704, 2.1576426711199583], [41.39587417637631, 2.156824934804435], [41.39517920716624, 2.1574354968749825], [41.39552641439873, 2.157063703204839], [41.39546395654558, 2.1568295468587753], [41.39517560785497, 2.157993677233774], [41.3947789837891, 2.157178915162419], [41.39541099388491, 2.1570641317769663], [41.39484120116789, 2.157693099670216], [41.39598385104655, 2.1576195674456073], [41.39478592623573, 2.1568232631696795], [41.39559902525574, 2.15764438552359], [41.39476767963173, 2.1573414654871077], [41.3950515748566, 2.157656507997112], [41.395253068416125, 2.156922257444084], [41.39581709385949, 2.1573900089718885], [41.39513027478309, 2.1578604626739044], [41.39595059855193, 2.1569000888524066], [41.395915671317525, 2.1579208696186565], [41.39578375214913, 2.1569990068023084], [41.39573077424698, 2.157211912151458], [41.39504272437058, 2.1576965696677695], [41.39596835375692, 2.157187899729404], [41.395004723858015, 2.1575342061291556], [41.39501094673379, 2.1569259232447164], [41.395528316533046, 2.156813656611792], [41.39531122948785, 2.157639257986013], [41.394734835833866, 2.1569902494580924], [41.39500286419651, 2.1570981760599537], [41.384997896721565, 2.15271967020053], [41.383884918297746, 2.1538982088269787], [41.38335395899965, 2.1546041250565455], [41.38361356181728, 2.1543978327181956], [41.38395056201935, 2.153040188542278], [41.383199871502704, 2.153447804223878], [41.38395837524731, 2.1537191145850674], [41.382975285528225, 2.152637176490939], [41.38448830383702, 2.1546237365740404], [41.38493434212314, 2.154241076244234], [41.38408659982919, 2.153669896418885], [41.38315323246468, 2.1526576323847277], [41.384471880400376, 2.1537503518977266], [41.38410471814908, 2.154425809739155], [41.38512173568216, 2.152568508874874], [41.384173777092506, 2.153657306292062], [41.38405173023132, 2.1527864871367446], [41.38490658746542, 2.154176967143779], [41.3843438355811, 2.1525327177338447], [41.38364838731042, 2.154227827536262], [41.38340437154412, 2.153986629511136], [41.38352113489181, 2.1539149705559466], [41.3830547360433, 2.1542549833619753], [41.38513835188892, 2.1534751988709337], [41.38428854122369, 2.154719734489195], [41.384109054127045, 2.153108202925007], [41.38460040822372, 2.1525829460504706], [41.38308973094531, 2.153462271196155], [41.38472419033926, 2.15298959666566], [41.383324596611864, 2.154426378326719], [41.38397525514562, 2.152886589670657], [41.38477368620151, 2.1533000097174777], [41.38397478163354, 2.153321877212769], [41.3844745688744, 2.1536760155789847], [41.384346665166575, 2.1540340427906703], [41.38347697833036, 2.1528170338139683], [41.384268279577725, 2.154267406700565], [41.38305950879386, 2.1537603490321184], [41.383556993592926, 2.1529909731282704], [41.38305168742027, 2.152799370946888], [41.38422493785108, 2.1526838694447803], [41.38412783712101, 2.1527818181300966], [41.38489691270322, 2.154075862492245], [41.384304333417006, 2.154329425647938], [41.38424370499685, 2.1528533355869266], [41.38517299208012, 2.1540227482495045], [41.38449433226019, 2.153613375005953], [41.38294572868408, 2.1534254866201907], [41.38372328745217, 2.15389081847049], [41.38321785122016, 2.154458516906962], [41.38470837850217, 2.152882077233971], [41.383013031253604, 2.1535001052699028], [41.384271471922204, 2.154374591770233], [41.38515103647622, 2.152564247318627], [41.38514938436187, 2.152882690935731], [41.38495934837687, 2.1527490319551603], [41.38497632665914, 2.154286696676409], [41.383722446545754, 2.1526505137318237], [41.38391030151948, 2.1541942140120414], [41.383262444534104, 2.153373823468045], [41.38357192600622, 2.152691079772575], [41.38472969907779, 2.1538957103356897], [41.38492438761373, 2.1545824880136983], [41.38472817530172, 2.1543830942170406], [41.3831768121001, 2.1545783644052747], [41.382999549875684, 2.1544853279964573], [41.38357961915313, 2.1536588065043296], [41.383247921556354, 2.1533287439670135], [41.38500964171338, 2.154339854589544], [41.384940170050875, 2.153695171432588], [41.38429179737036, 2.1540018696854126], [41.38333086182757, 2.154676096003429], [41.383583520700995, 2.1541635522489533], [41.38416394630566, 2.154495570213682], [41.383155371670284, 2.1544118634758176], [41.38483388055928, 2.1538556353667895], [41.38456373619284, 2.1530681289712232], [41.3838453865146, 2.1527098652744097], [41.38386719880285, 2.153048786431952], [41.3840932997208, 2.1533139792186105], [41.384055759204685, 2.153163486530541], [41.38486813694107, 2.1540170992992387], [41.38348077803072, 2.148128275688064], [41.38375022672473, 2.1486355183534354], [41.38240637980803, 2.1489933158906784], [41.382629643822646, 2.1481552331094798], [41.38288286988423, 2.1494531716141076], [41.383763283903676, 2.1493487817845427], [41.383874497340365, 2.148378609440122], [41.38293355357173, 2.14770442094871], [41.383533642576275, 2.147630104743727], [41.383410609227305, 2.149125868997377], [41.384118987359535, 2.147708611792642], [41.38245506000408, 2.1492041075879538], [41.38378231321412, 2.1492058591516754], [41.382258746196314, 2.1479033143860664], [41.383516545347156, 2.1495966560898783], [41.38335262373602, 2.147549416119961], [41.38297439502167, 2.147908514160222], [41.38259766519415, 2.149298305364932], [41.38390567237961, 2.1495955367005912], [41.38299875489475, 2.1484091497390403], [41.3829533599069, 2.1489799491358723], [41.382425065094694, 2.1495914618836722], [41.38213991660848, 2.147885742568186], [41.382400175126186, 2.148962180746362], [41.38297828593865, 2.1493661546799485], [41.38211249424851, 2.1489897673586245], [41.384174537739064, 2.1489521263850073], [41.382811758094306, 2.1486619748318367], [41.38375078051241, 2.148338799857859], [41.38287378834145, 2.149495709297374], [41.38265622637084, 2.1475752361971043], [41.38372020713633, 2.148113838389707], [41.3822058494135, 2.1474252164152117], [41.383734423995165, 2.148788519927032], [41.38393882164276, 2.1491911908350136], [41.38245848628382, 2.149220089606582], [41.38355964433477, 2.1479763881272502], [41.38346369460033, 2.149230468712136], [41.38295815466131, 2.1486061474175076], [41.38285144779709, 2.147639070025329], [41.38330213420787, 2.148228769130548], [41.38301979583528, 2.148652664421473], [41.38246911252586, 2.1484906679815463], [41.38299383115146, 2.148696151056272], [41.38362301556757, 2.1476622548754163], [41.38285185627066, 2.149242752060289], [41.3837249528373, 2.1494947689746047], [41.3836845928257, 2.1480660327740173], [41.38350897120191, 2.1475844589732778], [41.38258198690647, 2.148745843998723], [41.38401527848203, 2.1475240356486927], [41.38425592025659, 2.147576472877046], [41.383294114193326, 2.1475838452688665], [41.38341501804786, 2.148744331969941], [41.38334877671011, 2.147660494354395], [41.38282362022673, 2.1492244225672064], [41.38372313308362, 2.1485034085735912], [41.38335360604296, 2.149121823318764], [41.38280163433562, 2.1482690053550644], [41.382443515975176, 2.1495277197050866], [41.382249349726685, 2.148812003608653], [41.382598029262425, 2.1487072480725824], [41.38391692836138, 2.1480833145275176], [41.38389380852462, 2.148955919897333], [41.382149729416454, 2.1486192112771683], [41.384127503200496, 2.1484591021242765], [41.38389248270652, 2.1478941858228824], [41.383961286664096, 2.1478019822612473], [41.384263985844846, 2.147485580403581], [41.38337161397756, 2.1485891119224485], [41.38222036900825, 2.149575903316754], [41.38247588337667, 2.148686956320442], [41.38305675654744, 2.148917290771397], [41.38266424373938, 2.1477481488372283], [41.38264728315513, 2.1485246476881388], [41.38266640333685, 2.147719438317053], [41.38285225571949, 2.1493819138193944], [41.38382196552959, 2.1494825694814597], [41.38380735727004, 2.148567501089209], [41.383677333784455, 2.1490301310350826], [41.3822730240953, 2.147906745403656], [41.38251731776526, 2.148049211830514], [41.387684283687555, 2.1517877593933057], [41.3863595784784, 2.1492716871758533], [41.38812011540935, 2.1520483322555695], [41.387301298295796, 2.151003719780469], [41.38871007922742, 2.151331727593681], [41.38925037354475, 2.148956850263399], [41.387210241829045, 2.1513033379100888], [41.38664408591056, 2.151446061904314], [41.38781232800874, 2.1513450941866084], [41.387709163693636, 2.1520662399344204], [41.38920496521987, 2.1500332472893904], [41.388134509135384, 2.1483542571595042], [41.38666851092061, 2.1511996429853086], [41.386549041448134, 2.148532688042867], [41.38714430047806, 2.150976751176303], [41.38795372578085, 2.149825001188147], [41.389466260773716, 2.1502993193593785], [41.387422792286785, 2.1518352660557936], [41.386256375072804, 2.150844449309474], [41.38960432051394, 2.1495504545945683], [41.38729108702476, 2.1484138860507302], [41.38688159423733, 2.1516539243189414], [41.38873409424344, 2.1516718027394917], [41.38774552185618, 2.148893773711933], [41.38660418092731, 2.1515848561653654], [41.387440495432884, 2.148682280157276], [41.38896265817389, 2.1505780128050547], [41.38645117187577, 2.1495978290678197], [41.389545069004534, 2.1488835692968458], [41.38793123738561, 2.149533801629975], [41.38935068181464, 2.150572833797786], [41.38895892340069, 2.151905625265586], [41.38718540460314, 2.148895829063947], [41.38684377208543, 2.1516074936029024], [41.38637946229026, 2.1491769908540514], [41.387646316541336, 2.15069620305469], [41.38899857544307, 2.1511756962446205], [41.38622466122569, 2.151208199582778], [41.38622758402493, 2.149601634551385], [41.38714922087815, 2.150967402873025], [41.386717446632666, 2.148870665944493], [41.38924933395708, 2.150469046638094], [41.38894001957235, 2.15065252485759], [41.38977783510647, 2.1492361827306294], [41.38693632145782, 2.1484168485212694], [41.3883399945847, 2.149965090790962], [41.38734612021046, 2.1492850688099745], [41.38878846256289, 2.1483837614692995], [41.38952550144266, 2.1488924706587516], [41.38975591055159, 2.150407415568574], [41.38789074587904, 2.14927719159138], [41.38774077795059, 2.149646427648081], [41.386389542584844, 2.151682876396899], [41.38831327143455, 2.1484060794842748], [41.38839896079798, 2.151562295249873], [41.3885859880525, 2.15067640828312], [41.38705253736624, 2.148444236152823], [41.38665178046837, 2.151782931480179], [41.38672572319349, 2.1510944972581396], [41.387847916138654, 2.148498571303741], [41.386026536350414, 2.151547941562474], [41.38754532564917, 2.148701087744358], [41.38854458291911, 2.150666041028397], [41.386528877799606, 2.1494511593589034], [41.388177633102686, 2.1513718076728146], [41.38765545011077, 2.148940189369788], [41.3898381793297, 2.1482628767191994], [41.38735262098322, 2.14875117965881], [41.38878983906163, 2.1484675601638985], [41.388553107007105, 2.1503045304249238], [41.389139118893965, 2.1509241241825134], [41.38782582485769, 2.15120268309401], [41.38758746580195, 2.1493568180739078], [41.38867754568362, 2.150114689095235], [41.38986913554144, 2.150573369557211], [41.38964679753313, 2.1487289782104844], [41.3863130590335, 2.1486051447238275], [41.387777898292164, 2.151779935603198], [41.38838432443998, 2.149345133268114], [41.38920603021619, 2.1490486067192074], [41.38752208082132, 2.150317005891797], [41.38739539881854, 2.1484752597300223], [41.3885624471688, 2.151842967798945], [41.38798361093047, 2.15158175060003], [41.38607614891181, 2.150276353274533], [41.38884397838541, 2.1500236052160355], [41.38915361616459, 2.15105177165963], [41.387327087357946, 2.1519730542241673], [41.387927963023344, 2.150993314761709], [41.38700064561571, 2.1491282987825606], [41.386972545339084, 2.151665303018579], [41.38607162620114, 2.151053092719308], [41.38809037555289, 2.150660842381524], [41.38982904125283, 2.1485738183417453], [41.38927220158052, 2.151770707109309], [41.38713704205798, 2.1489946835085116], [41.38659078742794, 2.151969914909502], [41.38932064520307, 2.14963575089561], [41.405089963690635, 2.1624431522918304], [41.405545555631065, 2.1619697879066075], [41.405890012129454, 2.1619689413589263], [41.40577345974004, 2.1624357364140683], [41.40554979826474, 2.162678355250147], [41.40582265966811, 2.1617515752547805], [41.40584201592207, 2.1629430035823556], [41.4051842805721, 2.1628267293354386], [41.40593401596514, 2.1628086539161653], [41.40610920641195, 2.16267591733439], [41.40509107214006, 2.16218030989481], [41.4052121932538, 2.162220784284645], [41.406105982730274, 2.162303284211298], [41.40519988090159, 2.1621463283135465], [41.4060339505887, 2.1618176519170844], [41.4054978694399, 2.161754418735891], [41.40547189198935, 2.162200006917457], [41.40533450079913, 2.1618737510587795], [41.40539816744165, 2.1621183421614782], [41.40510943906011, 2.16187362171336], [41.405748725955796, 2.1624554142877894], [41.40548512270079, 2.162800434624612], [41.405599629189695, 2.162206693487306], [41.40596657671265, 2.1622984959148526], [41.4061432039774, 2.1618776865277463], [41.405033748147616, 2.1629460712262727], [41.40565774876997, 2.1622731823350994], [41.405547864242266, 2.1625719048434418], [41.40588678594056, 2.1622305820962424], [41.40575549182293, 2.162388362341375], [41.405823227773716, 2.162417841330207], [41.40614307634671, 2.161796912452633], [41.405262749358535, 2.162113654756958], [41.40605120719754, 2.162294038490071], [41.405186093231414, 2.1618872521762844], [41.405971208292, 2.1619455430450123], [41.40529207865931, 2.1626726871214212], [41.406120515849594, 2.1617987745693377], [41.40553689757118, 2.1621654877353054], [41.40588163253869, 2.162072612569354], [41.4061759121927, 2.161965958531352], [41.40543323550306, 2.161974640590162], [41.40564638658725, 2.1623522110136424], [41.40520205704773, 2.162751479716668], [41.405167665348245, 2.16261900960845], [41.406061677447305, 2.161890028856988], [41.405174018123844, 2.1620664691160276], [41.40604237319169, 2.162392384168631], [41.40616178320376, 2.1620910564081672], [41.40588558639663, 2.162822879263733], [41.405213782301, 2.1624233499153127], [41.405625937519254, 2.1619635267328907], [41.40619521747, 2.1629458051426687], [41.405799397546915, 2.1622532341988774], [41.404995786552924, 2.1621326962502248], [41.40550296086867, 2.16224621842213], [41.40582486343713, 2.1623802205086298], [41.405067241464586, 2.1624778606769217], [41.40538360059803, 2.1625589501545126], [41.40553164610022, 2.16294767505531], [41.40553646866778, 2.1624884302679894], [41.40520768103388, 2.1628207572154543], [41.40497062369509, 2.1619869272924555], [41.405492640479245, 2.1618027759725713], [41.40555875458569, 2.161838600656754], [41.406166891523455, 2.162503598022723], [41.405261349751854, 2.162491980690358], [41.405694056191656, 2.1628543642916487], [41.40108934255754, 2.1638860101263417], [41.40242299011954, 2.164701786902239], [41.40356750568431, 2.164383337947326], [41.40350214132941, 2.1656576336207243], [41.40292097769716, 2.1651180078696886], [41.40115526593091, 2.164138179634494], [41.40273540537094, 2.165083484612163], [41.40242830457865, 2.1648091573710713], [41.403281251522735, 2.165842306942827], [41.401328908289535, 2.163498904816967], [41.400847711606225, 2.16491701520013], [41.40202012396602, 2.1646276442509067], [41.40346894345011, 2.1660965389777935], [41.40300007577445, 2.1663145086393105], [41.40290722543817, 2.165739409548243], [41.40103955725934, 2.1649420355650393], [41.40095467093675, 2.164198032977687], [41.40183839595398, 2.166177018254002], [41.402598019426634, 2.163503393439449], [41.40219901409379, 2.166426041932876], [41.40187535953329, 2.1648766520108906], [41.401873135089446, 2.165110924078689], [41.40261528398456, 2.1642831865946994], [41.40261563674831, 2.16413823277103], [41.40106156456939, 2.165919758142986], [41.4021477819589, 2.1648602823718437], [41.401345352276394, 2.164189388450132], [41.40148848448519, 2.165176694270238], [41.40246635764607, 2.163937055038512], [41.40263405059047, 2.1654085958746028], [41.40296575325895, 2.166404358832419], [41.401981259858864, 2.164540352954386], [41.40181504215631, 2.1661735751377886], [41.40277756655422, 2.165376636925188], [41.40253636698965, 2.165094992718711], [41.40227511787097, 2.166340779229924], [41.40316012726436, 2.1637724098177644], [41.401867706859015, 2.1661284123962847], [41.402453541003936, 2.165842292921682], [41.40283165151643, 2.16546383301113], [41.401621627631755, 2.1644395110091743], [41.40099329445602, 2.1641180028302536], [41.401184548476266, 2.1636114316232926], [41.40131069275259, 2.165814278365023], [41.40083679162514, 2.165139695132233], [41.402539755631125, 2.1660599624381778], [41.40144079781553, 2.1659399927643297], [41.40375036745629, 2.164594445237823], [41.4025523891912, 2.1645867412083177], [41.403716273396746, 2.165342627632336], [41.40286291607262, 2.1656311280593203], [41.40365438073718, 2.1636781374585827], [41.40347707464566, 2.1657325397321983], [41.40102458467217, 2.1646525958000336], [41.40269338188512, 2.1647198139789747], [41.40224582135592, 2.1655577397928107], [41.40142786729042, 2.164431635899843], [41.40105123332717, 2.1647604325538077], [41.40170230528706, 2.1654641790907196], [41.40340227250802, 2.1661863703271975], [41.40108911275365, 2.1642056022455316], [41.40281790466956, 2.1662690203850494], [41.400860436613115, 2.1661604573770252], [41.40174616267218, 2.1650135692485843], [41.4033775301201, 2.1652885262193258], [41.402293537341116, 2.1645535787766645], [41.401873620767645, 2.163691157546173], [41.40239670511588, 2.165061475426948], [41.40379724715796, 2.1648480317296785], [41.402856981009656, 2.165572240659971], [41.40085056610504, 2.164631733078535], [41.403708191599776, 2.1647107070870244], [41.40093733851351, 2.1653159787235476], [41.40124557676544, 2.1662966247900757], [41.40286606554497, 2.1647054591020245], [41.403755464657785, 2.1641353280292184], [41.40153194254201, 2.1650784869882496], [41.40166178576177, 2.1635633417449265], [41.4023623122332, 2.1661942310578106], [41.40161789385782, 2.1650079403900486], [41.40365941696122, 2.1656314136110133], [41.402125058960465, 2.165324173951647], [41.40093226457452, 2.1648788478679575], [41.40146131000618, 2.165224536804995], [41.403780872150804, 2.165841302710562], [41.40338841552764, 2.1652408444880766], [41.401884317430124, 2.164721531628147], [41.40229948531538, 2.1662837220669204], [41.402508446391195, 2.1647536773617513], [41.40233520161491, 2.1640169095481547], [41.385631680637, 2.15458521791634], [41.38612884994634, 2.1547142212551273], [41.38645703490063, 2.154880723292298], [41.384131582184786, 2.1561803976864424], [41.385949985034955, 2.1543617324617177], [41.38485062458755, 2.155985203355961], [41.38461300178085, 2.15561351481663], [41.38473605546653, 2.1553891455038388], [41.38513806625, 2.1535803888390816], [41.38600888429466, 2.1538782285505533], [41.3858157779398, 2.1548116082254283], [41.38415584898839, 2.1544787705802944], [41.384133164153575, 2.154899198887689], [41.38451809960519, 2.1541596856327305], [41.385210477438704, 2.1554128794290364], [41.385765493811164, 2.155284294697209], [41.385936723098325, 2.1556310825936316], [41.38469212556021, 2.1550029024027824], [41.38429683413784, 2.154361767220602], [41.38544546057901, 2.153787197750198], [41.38416865330433, 2.1542873838516665], [41.3856809997659, 2.156065592898019], [41.38615610616707, 2.1556495482232343], [41.38581827887302, 2.1540222019399504], [41.38544688887032, 2.155156484374986], [41.384451465865, 2.154875318886347], [41.38668202163823, 2.155157442265244], [41.38663380718313, 2.1548758595662756], [41.38597442842512, 2.1555218004394745], [41.38469262262564, 2.1546410044606588], [41.386076732803296, 2.154869769729852], [41.38477238463497, 2.1548841290260494], [41.38521961681386, 2.1561691274467605], [41.3862134600769, 2.154325703302971], [41.384730869554396, 2.1554950153015806], [41.38652391903616, 2.1549091950222863], [41.384144597363466, 2.155283035053862], [41.38630556694017, 2.1543028668806157], [41.386004098288865, 2.1550881934575266], [41.38520724294867, 2.1553390948352247], [41.38455758996167, 2.154273604932883], [41.386343419847485, 2.1557853110540623], [41.38532497040522, 2.1546803458351973], [41.3863067063313, 2.154414812929439], [41.38445055526004, 2.155303677652091], [41.38633397118568, 2.155418073136225], [41.384716107225515, 2.155369738652125], [41.38407458092276, 2.1560127424977593], [41.386464012492546, 2.1553890430571934], [41.384493771772874, 2.153750635024193], [41.38525908832407, 2.153938196660829], [41.38415052522266, 2.154057593961519], [41.38662007364596, 2.1560497223420994], [41.38408487409436, 2.153864384357558], [41.38454229060136, 2.1553279255469495], [41.38464583158076, 2.154332334853333], [41.38654099701304, 2.154001218030851], [41.3860225751608, 2.15533717931182], [41.38466404684529, 2.1554073739134596], [41.38511631672342, 2.15430222631893], [41.385628884448806, 2.1541460871811946], [41.385843387679635, 2.154672541790621], [41.384745639628726, 2.1559313011344674], [41.38445303008433, 2.153603431783226], [41.38525874367262, 2.15383608414138], [41.38415245578681, 2.1559543374510026], [41.385405935471695, 2.1554741500791055], [41.38562124462412, 2.1545444947747523], [41.38594229645577, 2.155676358035085], [41.38467935755983, 2.155625140365478], [41.38449006103142, 2.154799668883813], [41.385468993098364, 2.154010750844251], [41.38656384932567, 2.1556991645131705], [41.38416543028207, 2.155217376370775], [41.38534463670532, 2.153631141125586], [41.384691511812534, 2.154850174868302], [41.38576486279982, 2.154537018188586], [41.38606524764228, 2.1536808928195073], [41.385399716407406, 2.1560158227507964], [41.3859628248566, 2.1543290394683687], [41.38424953748949, 2.155379214449925], [41.38476719939723, 2.154260782324928], [41.38577725805496, 2.155014103629798], [41.386463797187496, 2.1549462739973495], [41.38545755744255, 2.1555318366677354], [41.385387743467, 2.155169372604818], [41.40417905555304, 2.1660901473371763], [41.40577696673579, 2.167116278794125], [41.40606743470728, 2.1656358815781322], [41.40416088626925, 2.1647645225928107], [41.403820557660524, 2.16500332217701], [41.403812245927526, 2.1655048583115017], [41.40595714727225, 2.16520480649687], [41.40641656361575, 2.165682004195314], [41.40632641737219, 2.1669140616167275], [41.40420258718072, 2.1661525000268593], [41.40635147141503, 2.165014292246319], [41.40627732075876, 2.1656191045614306], [41.406293563458235, 2.164894270448981], [41.40407760946411, 2.167603546714501], [41.404684212488526, 2.165630543185883], [41.40651692455149, 2.1654922501636795], [41.40403321728363, 2.166607053965583], [41.404786794694125, 2.164869295225643], [41.404878385809006, 2.1657300566267397], [41.40393219860822, 2.1660223260610234], [41.4068764517892, 2.1660648949069956], [41.40446383496805, 2.1646160075164156], [41.40623713364174, 2.1677678397655766], [41.40470110074484, 2.16589956795757], [41.40544269306792, 2.1667046175116593], [41.404154435184296, 2.165324419442971], [41.405119234348625, 2.1656300702267353], [41.40506468866634, 2.16662976561736], [41.40519237898422, 2.16646269621438], [41.40534750747971, 2.167006718961366], [41.4053753961116, 2.1655148262111066], [41.40663345374183, 2.1656101128807257], [41.40563004983071, 2.164996393092457], [41.40453539187263, 2.165569575442283], [41.40489052554624, 2.16512200746976], [41.405602248187115, 2.1656115310200956], [41.40451018520382, 2.166313932392314], [41.404874094770186, 2.16716729486256], [41.406297777379734, 2.1663627817529982], [41.40529105577293, 2.1654652050190526], [41.4056965628644, 2.1650584326767497], [41.40563694621762, 2.1671271591498673], [41.405838405133345, 2.1651967697813133], [41.40435633209634, 2.16658589430116], [41.40419993779646, 2.166036417153116], [41.40523904740748, 2.1658106883898167], [41.40465675554736, 2.165550642382777], [41.406765444366606, 2.1646379816777634], [41.406386240893916, 2.166223137520266], [41.405462542322745, 2.1674490804664144], [41.40381139386361, 2.167784536055248], [41.40687713461707, 2.1671215089029277], [41.40650605638285, 2.166537273557297], [41.40693507867236, 2.167326147285945], [41.40667901576514, 2.167433479783558], [41.4064987092468, 2.1659532331029374], [41.40558548916092, 2.1651765089281496], [41.40540236598874, 2.1660344089107335], [41.4038445381034, 2.1661693938341635], [41.40704579432614, 2.166558307689344], [41.40614429430211, 2.165464844335671], [41.404162704850286, 2.1656799654737195], [41.40620759406137, 2.1650213469213337], [41.405248579418384, 2.1645853856715944], [41.40669500788951, 2.1657965918867528], [41.404482483760674, 2.1669194196723556], [41.40682236041078, 2.1676517661487122], [41.40502751231435, 2.165617660171989], [41.406873277762145, 2.1649755375690707], [41.40430558824476, 2.1668839250494116], [41.406110217792374, 2.1672397096931766], [41.40622299374222, 2.1677169337223723], [41.40577596702299, 2.166027993683403], [41.405284356205904, 2.167709714735254], [41.40641206926261, 2.167875586817233], [41.405057954513474, 2.165234525493926], [41.4062223474588, 2.1659482666104846], [41.405898621842, 2.164941121393898], [41.40669089674504, 2.16693765691508], [41.40656541188601, 2.1664706648661074], [41.40602952753223, 2.1648115028764474], [41.404808225811635, 2.1675980475845416], [41.40425195767391, 2.166283006830524], [41.404708614028465, 2.1672340193386326], [41.40642993267219, 2.1653533011084534], [41.40426983378339, 2.1664898106349897], [41.40520892636418, 2.1656391938774937], [41.40527620282444, 2.1663599890331584], [41.406599527500795, 2.1654785608279643], [41.40616908263912, 2.1663527902129465], [41.40398553355829, 2.1658226119449475], [41.40466210480678, 2.16528070292125], [41.404743347517886, 2.1670765992034338], [41.4048985221765, 2.166287214994584], [41.38254814735533, 2.14866863847914], [41.38322429528938, 2.1488810957645765], [41.382876047705075, 2.1487767537787854], [41.38237231715159, 2.148605502527378], [41.38285503465002, 2.1494042605252464], [41.3822140502345, 2.1487666674084016], [41.38208058277137, 2.1491087315201916], [41.38231723818029, 2.149490211685754], [41.382088797364645, 2.1494827485968075], [41.38186102236767, 2.148484202971778], [41.38260560560015, 2.149261922823725], [41.382370846528254, 2.1484923571959693], [41.382511451919875, 2.148644470189081], [41.38337167169368, 2.149908320771933], [41.38249395978865, 2.1489675720878676], [41.38322043222982, 2.1485754498816676], [41.38200798659444, 2.148308528204301], [41.38282940770809, 2.149354888973739], [41.38310794605773, 2.1497409229582156], [41.38261813778945, 2.149612961919043], [41.383128143509985, 2.148392906370101], [41.3818201224418, 2.1487216651412737], [41.38318423304153, 2.149813084521691], [41.38322117190838, 2.149209877525135], [41.382636561775804, 2.148457738213644], [41.38235725843532, 2.1493663555964138], [41.382748363082854, 2.1494100572455426], [41.383033720166466, 2.1486200400833635], [41.382074612611234, 2.148944221804646], [41.38222959033015, 2.1494909155891118], [41.38224559718926, 2.149650389681702], [41.381842017234426, 2.1485615923275607], [41.38231163584173, 2.1492289066303347], [41.382244973488724, 2.1491271654027617], [41.38322246110331, 2.1486491256236278], [41.38263378508901, 2.1492032579455356], [41.38194322669234, 2.149299371172041], [41.38264012606545, 2.148454934841273], [41.38298008326831, 2.1496080436778118], [41.382065474709385, 2.1497970886400775], [41.38308106640865, 2.1493586712861026], [41.38214770436972, 2.14946160720728], [41.382188563597886, 2.148481720386517], [41.38189582594846, 2.1490520565886277], [41.382054548886984, 2.1497562528380607], [41.38204488487556, 2.149608224612694], [41.382703275119944, 2.1491136687223977], [41.38196088887331, 2.149487354370374], [41.38195704290411, 2.148884657406874], [41.3833978754109, 2.1494788995250222], [41.382294677616464, 2.148280632698512], [41.38196884823975, 2.14918886322441], [41.38187275172959, 2.148629747370109], [41.38291188371354, 2.14945919381655], [41.3833671000002, 2.148303743163141], [41.38232779498556, 2.149647173544856], [41.383330484452905, 2.1491997124985645], [41.38230487431865, 2.1488587027923964], [41.38302324863577, 2.148889067175988], [41.382339650015474, 2.149249525403295], [41.382698909049196, 2.149389068474316], [41.38235820727357, 2.1496090867033613], [41.38295755140311, 2.148799654923869], [41.383389876792585, 2.1489216071919817], [41.38185904734181, 2.149789182524445], [41.38265464782773, 2.149292276800264], [41.38184680419019, 2.149922966074028], [41.3822917569168, 2.1484629966939846], [41.381858913296604, 2.148357034267884], [41.38184380400248, 2.1486289522228352], [41.38222654094439, 2.1495415916107175], [41.38192459431246, 2.1491591635305802], [41.382486242034005, 2.1485218816893483], [41.382826065099515, 2.149440719300483], [41.39962647072152, 2.164431257600184], [41.39951381538304, 2.1642486231391476], [41.40090163356251, 2.1644072630243407], [41.3987488668783, 2.1656735443641906], [41.399796630361, 2.165479042123609], [41.401207793039305, 2.1642297136696227], [41.40065507536645, 2.1647086239358724], [41.401322241121434, 2.1634785939982186], [41.39974014860265, 2.165396913385181], [41.40076559750891, 2.163776685808352], [41.39984032435599, 2.1646499969402604], [41.400369712528956, 2.1635688797728494], [41.3994661668536, 2.1631548934099953], [41.39933366364594, 2.1653444262219566], [41.39991823950899, 2.1635962586378428], [41.39879756340087, 2.16471596627846], [41.399973227398675, 2.1632758460017003], [41.39929370848679, 2.1639470450451523], [41.399361717027375, 2.163089422897069], [41.3999280042377, 2.1635146562764187], [41.3993428321388, 2.1645330144083665], [41.400312720983756, 2.1651390375254715], [41.40120856499842, 2.16372670307463], [41.39886172243486, 2.1647051260084913], [41.40073716004984, 2.164361958880773], [41.399339055882564, 2.1657820220246253], [41.39890912831167, 2.164868983908292], [41.3998597784776, 2.1633568855731533], [41.40104937068136, 2.164022917435623], [41.400217993096355, 2.1636431110127563], [41.40078412892817, 2.1638508402518353], [41.40067237298969, 2.1637213559860373], [41.401060587334435, 2.164697760159937], [41.39909921276673, 2.1654197976734877], [41.398591803431685, 2.1650272262084966], [41.399961441215346, 2.1645302757125036], [41.40078186852911, 2.1648340393645027], [41.39872154344881, 2.164530737614256], [41.39912499890283, 2.1648184079330077], [41.400537549416484, 2.1653043353772503], [41.40056531999318, 2.1631733516390383], [41.399357217855076, 2.165537950912131], [41.40104080803461, 2.1651311400169075], [41.39939094172184, 2.164092717381851], [41.398848350066736, 2.163013805718617], [41.39994433508549, 2.165230718033453], [41.40005451258134, 2.164807491311724], [41.400383269852234, 2.1637615752253385], [41.401201929300846, 2.164769436330469], [41.40040871775804, 2.1651604422244577], [41.39871176186478, 2.1633099924613277], [41.399436530800415, 2.1643437127230385], [41.39981809532722, 2.163050055014356], [41.40093212081425, 2.163807715967457], [41.40053444865632, 2.163315506308544], [41.399649631387916, 2.163775962969212], [41.40071755924969, 2.1634403323695297], [41.3995079902882, 2.164742915816057], [41.39970089581702, 2.165704144952017], [41.40083266573139, 2.1637208178507854], [41.39901215378508, 2.1649477161798956], [41.39856912203645, 2.1648566542607224], [41.39859437861789, 2.163674316241943], [41.39965356543197, 2.1654404481919265], [41.40127717458684, 2.1657714789851528], [41.39885049103126, 2.164698269154255], [41.39903506093335, 2.1649358688580476], [41.39984179810468, 2.1648564724090895], [41.40031434123379, 2.163307887679662], [41.39971910510249, 2.1641716755320424], [41.39870324946231, 2.165592469438441], [41.400523142236985, 2.165683952668133], [41.39974990694773, 2.1648642261068782], [41.399324971046504, 2.1646758112811164], [41.40011576620488, 2.163181987680229], [41.400631700301304, 2.16364996880753], [41.39999625709111, 2.165776560530389], [41.40119778255598, 2.163472071521089], [41.399721250025216, 2.1657444669717303], [41.39875774758636, 2.163319920242227], [41.39931550021463, 2.1654209017508608], [41.40070037905564, 2.1630507594599115], [41.400940602325754, 2.1646355137514255], [41.400751673159576, 2.164686825491763], [41.40103748943104, 2.163302502481609], [41.40083509676046, 2.164206638757069], [41.39971086575726, 2.1654399064740613], [41.40085568923891, 2.1654000746945434], [41.40673518908499, 2.155790840481171], [41.40697276694026, 2.155738481503105], [41.40715059656539, 2.155936684953764], [41.40713237882208, 2.1557270040872525], [41.40675229156646, 2.1559983322641183], [41.40708036510306, 2.155930024223406], [41.407112051168504, 2.1556561657482476], [41.406913380436414, 2.1555884775697947], [41.406817538278275, 2.1557528835039443], [41.4068580148668, 2.1557596088658886], [41.40713749399097, 2.155602289290291], [41.407140416851, 2.155571520013399], [41.40691244127373, 2.1555549012400865], [41.407095855535516, 2.1558829588259134], [41.406958652614435, 2.155901164490747], [41.40703147260884, 2.1560240005599454], [41.40678646981042, 2.1559694147423794], [41.40701848471708, 2.1557513180214958], [41.407048599167354, 2.1558428007302464], [41.40710681113474, 2.1557993668683735], [41.40709917135803, 2.155805980643562], [41.40712548572927, 2.1559289222921447], [41.40670437240807, 2.1559813125813867], [41.407072523971166, 2.1556162137354353], [41.40711679625486, 2.1556047273477716], [41.40678932614786, 2.155750091889102], [41.40685051976931, 2.1559398793761217], [41.40699523004775, 2.1559356230268385], [41.40703049800349, 2.1557910152433], [41.40696684305908, 2.1556554150669704], [41.40690987007487, 2.155643419375383], [41.40699878554241, 2.1556057286488794], [41.40696166448763, 2.1559540613799184], [41.40715050390937, 2.1556653958148715], [41.4068976958872, 2.1556124369908027], [41.40669723479543, 2.1558245493025527], [41.407015895145385, 2.1559719602765424], [41.40675081176753, 2.155724772387311], [41.406717603448946, 2.155580885873538], [41.406728795426154, 2.1557798250031945], [41.406719393604476, 2.1556560272632623], [41.407094821433894, 2.15571943787908], [41.40681608265166, 2.1558619059418676], [41.40681004438306, 2.1560160374692314], [41.406776322563566, 2.1559057269939315], [41.40697863413111, 2.155693609952528], [41.40711174883081, 2.1559155148853795], [41.40672097162313, 2.1559770327012933], [41.40698415012317, 2.1559389345793742], [41.40602618219207, 2.157739446649868], [41.40410862940735, 2.1572043996931525], [41.40613562766097, 2.155789041208975], [41.405794502408654, 2.155788040104794], [41.40440115064207, 2.155422175655884], [41.406509116694224, 2.157080955965585], [41.405111311942505, 2.158121416460372], [41.4047893763622, 2.157538858286197], [41.4048836940054, 2.15681475126908], [41.40570570763317, 2.1571831972623934], [41.40489327120457, 2.156636419831431], [41.40627654011375, 2.1562358827461496], [41.404776578821064, 2.1573141764476014], [41.404709669254146, 2.1560228056649233], [41.40662501211415, 2.1582535658253543], [41.406111861657024, 2.158022002025082], [41.40586255568529, 2.1573560036786454], [41.405722347979605, 2.1573354084866123], [41.40515914882407, 2.1561098865430375], [41.40668218526677, 2.1574640606594784], [41.40596139244128, 2.158199565778333], [41.40598256874246, 2.156528385753321], [41.403614365543, 2.1555188115530672], [41.406240940287404, 2.1565059411474214], [41.406551583831394, 2.157772502965189], [41.40603746648068, 2.155510393129889], [41.40429827369882, 2.158196969587896], [41.405274101436575, 2.1556494719468255], [41.404241781978556, 2.157523014588472], [41.40416901659271, 2.1580060634493172], [41.40366780548304, 2.1563278064199727], [41.4054876704487, 2.1563810649834894], [41.405587815103935, 2.157796891002201], [41.403855185863435, 2.156024256424009], [41.40666093393132, 2.157794224089135], [41.404693606839835, 2.157124900156873], [41.40660006873234, 2.157153849503203], [41.40668288108467, 2.1555351112662526], [41.403815528825774, 2.156528744945123], [41.40375240686988, 2.155842704207339], [41.40574932914748, 2.1572628362273547], [41.40588780272646, 2.1576108517646784], [41.40538103288962, 2.1553240750533402], [41.405816739004734, 2.157814798856337], [41.40655677323024, 2.1580468635276917], [41.40630420588203, 2.15591035291069], [41.40622078549099, 2.1564897923000137], [41.40571729456278, 2.158254118419331], [41.40502160812092, 2.1557875227425076], [41.40557236543272, 2.1564384803568815], [41.40666769173543, 2.155660833123917], [41.405338322659865, 2.1566592060062115], [41.40664468332631, 2.1563438160245734], [41.40519955728599, 2.1564590204094447], [41.40438271015023, 2.158041427112861], [41.404262472002614, 2.158376246355543], [41.404883631394135, 2.156267949455543], [41.40578807552663, 2.1567495098637406], [41.40415823430535, 2.1573712747740688], [41.40428866573, 2.157661289966079], [41.404360522088076, 2.1571836567608833], [41.40474852617739, 2.156867961978464], [41.404185391566806, 2.156956694008064], [41.40655093152951, 2.157604261418594], [41.40547746494894, 2.1579085995390694], [41.40652308752167, 2.156096612843799], [41.40419266875081, 2.156478377760543], [41.4041117283522, 2.157479999494025], [41.40500996840374, 2.1570849850363976], [41.40600854358823, 2.155805867819336], [41.406482591860176, 2.1571539308663437], [41.40556705191928, 2.1564060134006326], [41.40431591207355, 2.156185306076493], [41.40445685577136, 2.1582337662439834], [41.40439044914166, 2.157479270968398], [41.405782240321514, 2.1574102981454395], [41.403887256125266, 2.15602848060282], [41.40587606157102, 2.1581080850987706], [41.40597757805101, 2.157165372647806], [41.40647774674767, 2.1578924067665453], [41.40453633433409, 2.158041319265449], [41.404443404580014, 2.15750729354094], [41.40603663206524, 2.1565716807557056], [41.40629519135227, 2.15687216342982], [41.40399486113202, 2.1555098640618042], [41.40412277764645, 2.156691827082066], [41.40634827491936, 2.158179003083287], [41.40453815881258, 2.1558335423651176], [41.4054118535722, 2.1577675901808693], [41.40504198278509, 2.1579805286773475], [41.404447882176584, 2.1580062701267777], [41.40365120426445, 2.155857199122262], [41.40742906169672, 2.1637875037650027], [41.40681929055743, 2.1636412230437294], [41.406723837128965, 2.1629147798932125], [41.40739273672229, 2.1635775117096117], [41.407674689864685, 2.16340451107135], [41.40709893560802, 2.162989961081088], [41.407685891708745, 2.164056554563145], [41.40754307654729, 2.1636114213906117], [41.40722353306669, 2.1632843830252946], [41.407201572500455, 2.1629137860310204], [41.40714582669664, 2.1631762877285134], [41.40672437678781, 2.163504971211247], [41.4074702382572, 2.1640665498882345], [41.40732311858256, 2.1639730343561374], [41.407648991674805, 2.163768436522479], [41.407001767795876, 2.163553505301938], [41.407382067729536, 2.1636179818501375], [41.40766606022236, 2.1629323221272787], [41.4073646110023, 2.1634604059789813], [41.40702687062009, 2.1639835047020215], [41.40746960047074, 2.1639588649646764], [41.40707099730346, 2.1632541673696624], [41.407172349899696, 2.163067287846187], [41.4065793424523, 2.163898457216035], [41.40722566801346, 2.1637506702794327], [41.40783040978288, 2.1635593137310734], [41.40766530677695, 2.1636471337445706], [41.406556849878456, 2.163883293606792], [41.40778471592418, 2.1629664456294706], [41.40728205768974, 2.163744064247792], [41.40699568513836, 2.1636140002027506], [41.407301014676946, 2.163588990021206], [41.407558780944626, 2.1637766478146627], [41.40721575778329, 2.1639199886948517], [41.40783612490275, 2.1639424725456657], [41.40695709440104, 2.163260975012667], [41.40733896831492, 2.163971364422659], [41.40723070963183, 2.1640065445286485], [41.40663175388809, 2.1634397706691], [41.407794524194586, 2.1629472861057], [41.40774574293023, 2.1631670676201193], [41.40707659575585, 2.162881877089197], [41.40745511866462, 2.1630920240771054], [41.406570213315185, 2.1635569728425423], [41.406634162381756, 2.1630342457992993], [41.406888990600805, 2.1633821583788113], [41.407453089655924, 2.1632517569518828], [41.40683126495814, 2.1632757354173684], [41.40763474214511, 2.1631533249626855], [41.40679176310829, 2.1633352686374585], [41.406995294011395, 2.163137451778328], [41.40680441509399, 2.1630390392602084], [41.407705574591496, 2.162889519814234], [41.406546339065336, 2.163061802954172], [41.40729157166519, 2.1638547109190993], [41.40776554916198, 2.1637813553974787], [41.407293744302756, 2.163443634212646], [41.407578388442374, 2.1630368694256092], [41.407151529290815, 2.162870518326523], [41.40703844854667, 2.1632827105761874], [41.40687793461138, 2.1633776193285086], [41.407227317111165, 2.163852852242981], [41.40695547040016, 2.1638654894190443], [41.407195962455816, 2.1629742634967117], [41.40760175211571, 2.1628264588693153], [41.40655709820454, 2.1633563374498106], [41.407707399029604, 2.163155243025245], [41.40660366836762, 2.1629773066226763], [41.40679096904558, 2.163893719005033], [41.39504572330448, 2.16086738467594], [41.394963455836255, 2.1611513265214786], [41.39527345815242, 2.1610312454361957], [41.39486239297123, 2.1615507360283495], [41.39492233292668, 2.161197028632621], [41.39542875518509, 2.161655335448455], [41.39479687597, 2.161418979533565], [41.394737556031224, 2.1615230671716215], [41.395376389111355, 2.160862010571929], [41.39458417929024, 2.161156131525031], [41.39530189951037, 2.16091114682005], [41.39480499075697, 2.1614029687102896], [41.39522497565874, 2.1609878637797726], [41.39548274691765, 2.1616867833400235], [41.39462028726116, 2.161646878391801], [41.39494059273974, 2.161050616931242], [41.39459601667239, 2.1614158077536145], [41.395056852851944, 2.161014245056839], [41.39495620842572, 2.161520075919875], [41.39490358399749, 2.160994300704261], [41.39470912025045, 2.1609507017723386], [41.395061656440305, 2.1616597572101885], [41.394758484540205, 2.1611186637543067], [41.39516657372275, 2.1607943190316594], [41.394950499228074, 2.1610918332025646], [41.39531629162145, 2.1609834715413885], [41.39538757354132, 2.1607736226511824], [41.39479802186346, 2.160839991034084], [41.394599304947995, 2.1612157110808647], [41.39454454119274, 2.1611640424328358], [41.3954292426566, 2.1609801763518726], [41.39546696651826, 2.161006621828781], [41.39456110950496, 2.1614407798901207], [41.39532275829158, 2.16167462775104], [41.39505752705955, 2.161701740116863], [41.39489755128674, 2.1612361198243657], [41.394716336541514, 2.1607593673588004], [41.395291153821766, 2.161534059542156], [41.39492045233958, 2.1609503169212423], [41.39484881807466, 2.1609430016619484], [41.395039300600516, 2.161484725259891], [41.3954350300871, 2.160843900132612], [41.39545038627625, 2.161178694723352], [41.39490020548141, 2.1608438222124273], [41.395246942354134, 2.161083302175071], [41.394928280026406, 2.161397184829935], [41.39539849436075, 2.1614430435553404], [41.39544742769243, 2.160904144689153], [41.39501398264134, 2.1613757749643425], [41.3947757574418, 2.16097472567621], [41.39466171410142, 2.161103161211066], [41.39486950332099, 2.160959647607385], [41.39507066758796, 2.1615901292493374], [41.395348920286914, 2.1608773343795744], [41.39505022432894, 2.161688823432168], [41.395397574389136, 2.1615993482747498], [41.39489363356377, 2.161062582222773], [41.395254833898875, 2.1608105234933555], [41.39512042453417, 2.161367704473529], [41.39469202145404, 2.1607907274867335], [41.39481724489328, 2.16081357757976], [41.39533758536627, 2.1610065598492696], [41.404097018732394, 2.1442169351420595], [41.40634510747849, 2.1443486444546735], [41.40481828291539, 2.1441186859538486], [41.406727910435805, 2.144083522039823], [41.40604897220509, 2.141374423021797], [41.4062595271877, 2.1442790378785683], [41.40682982511334, 2.1435072024579136], [41.406425786197374, 2.140740381987816], [41.40417980601626, 2.1414090556957324], [41.407052789536046, 2.1427461142635336], [41.40644420312612, 2.143073868609163], [41.40419995286453, 2.1407932373939222], [41.40736856169279, 2.1433641430860235], [41.4036779316909, 2.143173989399465], [41.40540016044783, 2.1413620332915237], [41.40458727480405, 2.1438571657638685], [41.40720325355831, 2.1424087733095964], [41.40344539596366, 2.1421306794956965], [41.406177037031796, 2.1436481928693643], [41.405616565783966, 2.143830027772143], [41.404131210881175, 2.1420069772772568], [41.40672371226692, 2.142095613099188], [41.40728593440124, 2.1411137063378005], [41.40446420873669, 2.1434505420895062], [41.40499030246595, 2.142549216081541], [41.407134532799276, 2.14084559253625], [41.40382715481015, 2.1436021406045556], [41.40389737537175, 2.1434711306315357], [41.40506308398588, 2.1424652950355494], [41.40728622805026, 2.1443958808458112], [41.40647274626849, 2.142108154057843], [41.40368726245982, 2.140521112443545], [41.40503980792287, 2.1444349643142604], [41.40592311816997, 2.1409571678624646], [41.40367773302272, 2.1436442862972123], [41.404436551765365, 2.1430012028142253], [41.40499940917282, 2.1408667067736267], [41.404119765214325, 2.143438563666975], [41.40624242435973, 2.143907205096761], [41.40531442126202, 2.143008075627878], [41.40404277128199, 2.144198879101151], [41.40446574102853, 2.141845319400163], [41.404409170052226, 2.1406571525478144], [41.403562097835795, 2.1413655780883216], [41.40374528055863, 2.141872428846674], [41.40597098511155, 2.1421668069896387], [41.403893198282155, 2.140820129854265], [41.404191878330316, 2.1442620041007543], [41.40727453137996, 2.1438592418074878], [41.40491726889419, 2.1432511913124106], [41.40733377347808, 2.142365134594433], [41.405682633904675, 2.1430955175414805], [41.40503451176703, 2.144166498528938], [41.403643873733785, 2.1411650489905343], [41.40709396432, 2.1440906202853625], [41.406837250166575, 2.1440439686435355], [41.40653663804196, 2.1424207117897787], [41.405300465247784, 2.142855385146769], [41.40690363752852, 2.1414433812477953], [41.40558996714681, 2.141215568423868], [41.40371992578054, 2.143511411555808], [41.404853759405505, 2.1443261084954166], [41.40504199756301, 2.1424395986036817], [41.40667965579579, 2.14075779232783], [41.405484614405545, 2.140476524963919], [41.40510013875884, 2.1412895255178745], [41.40567629973512, 2.141962638435009], [41.403499218085464, 2.1435764655895198], [41.404301303403514, 2.143916313915937], [41.403769302968335, 2.1436712685037524], [41.40344993251481, 2.141173090449789], [41.406864393324014, 2.1405205302010524], [41.404269929709464, 2.144333225261026], [41.403898402742456, 2.142115165829949], [41.40583544995513, 2.1414853061274703], [41.40484747005528, 2.1435559179308106], [41.406806818663206, 2.14188926198757], [41.406942752066506, 2.1431184089247197], [41.40467437100971, 2.1435178228329397], [41.40602178737928, 2.1408754671622026], [41.40454854442099, 2.1416194175699346], [41.40407612023306, 2.1408456678551118], [41.40609902331399, 2.1404843040664026], [41.40520751547993, 2.1410274796049804], [41.40710634118748, 2.1438427215156795], [41.4065039090507, 2.141380716886714], [41.40727570890659, 2.1421878555129097], [41.405253688879284, 2.1434458870317963], [41.4060995614047, 2.1413546046216676], [41.40614274858162, 2.1430639071897035], [41.40542590534722, 2.1416047944100964], [41.406584428826974, 2.1441152376839576], [41.40490101990538, 2.141853655748334], [41.40582667033571, 2.1430187508699077], [41.40612406862683, 2.14408037864456], [41.405883119229394, 2.1443916387606357], [41.40462848979684, 2.140808652944432], [41.40627333007121, 2.143893801110999], [41.405809599181374, 2.1406859632255846], [41.40641875065466, 2.1440606893783634], [41.39889088942146, 2.1536754140782683], [41.39889214158819, 2.1536846046261076], [41.398901980809164, 2.1536629359060027], [41.39889428191079, 2.1536694513086143], [41.39890118291155, 2.1536675999790376], [41.398914050990534, 2.1536864948775687], [41.39889617148213, 2.153663044350911], [41.398906611676914, 2.1536500413885635], [41.39891722410559, 2.1536652813810955], [41.398900591621796, 2.1536519068922626], [41.39890910906316, 2.153660205979174], [41.39889521316471, 2.153687512006746], [41.39888892287325, 2.153680330459347], [41.398918648870904, 2.153671951399054], [41.3989188603482, 2.153661730564464], [41.39888604999034, 2.153673906899416], [41.39888737974066, 2.1536605403547706], [41.39891039876662, 2.1536545024885956], [41.39888607853806, 2.153678714647461], [41.39890988763441, 2.1536795283468533], [41.39888122776631, 2.1536664438108235], [41.39670262418653, 2.1606994104909307], [41.3964710313751, 2.1594996665749235], [41.39842477884005, 2.1578234462459873], [41.39549299780951, 2.1611680536655973], [41.39872806888814, 2.1589146361100835], [41.395351529982015, 2.1588879066727795], [41.39592668667556, 2.1592541574106887], [41.395067670189235, 2.159100074160528], [41.39647068425935, 2.158005066252855], [41.397007255338515, 2.1608836884792777], [41.398397719604674, 2.1613033109560096], [41.39737588525979, 2.157698793942483], [41.39640552456657, 2.1591028552081024], [41.39643977911705, 2.1612789064221016], [41.395894837620915, 2.160093668508917], [41.39776312898072, 2.1588470126558947], [41.3987644087172, 2.1575588201213622], [41.39850616789657, 2.1592311565776576], [41.39587759834307, 2.1577434892203597], [41.3966787100864, 2.1596558585017314], [41.39771302686244, 2.161416475742277], [41.39530857228537, 2.1601410955535885], [41.39755944019087, 2.1574660523162144], [41.39605867525273, 2.1587660151737307], [41.39878009290586, 2.160758682704458], [41.39859770123924, 2.160010390637435], [41.39871225024301, 2.1608645585900357], [41.39699857961442, 2.159720782634396], [41.39542029582262, 2.1614286276369783], [41.39653049105947, 2.158281034593084], [41.39731911286, 2.160261164619916], [41.39524733676893, 2.159562684506873], [41.3956786157529, 2.1579473678540206], [41.39848252506233, 2.1610477635442207], [41.398084857963724, 2.1587322332252423], [41.3980688853548, 2.158667773274291], [41.3970977662437, 2.1579461437632457], [41.39511936088578, 2.1598696171158522], [41.395979595368615, 2.1606394806505365], [41.39577388900776, 2.1580267443782595], [41.398401736810946, 2.1605720879672012], [41.395489206289, 2.158477158558344], [41.39536655617508, 2.158346191276611], [41.39750453164683, 2.1613044802865273], [41.39806045661536, 2.1612681355708743], [41.3976026664179, 2.158321302582059], [41.398879479825965, 2.159159615039533], [41.39844973581171, 2.160610229801341], [41.39682967961411, 2.1588477477515036], [41.398689634958956, 2.1605682726595448], [41.397254751135854, 2.161126890469183], [41.39588594605653, 2.1600140601862705], [41.39866911028695, 2.159490323253213], [41.39697231143727, 2.1604709189874196], [41.396889795636746, 2.1584270868727677], [41.3968194226387, 2.160640898885843], [41.39764579821502, 2.1577100949645796], [41.39815889217853, 2.1604353763893465], [41.39578670856532, 2.159031726246931], [41.395616011546906, 2.1604112603936585], [41.39871583627161, 2.1579959426101674], [41.3977803831314, 2.161218952756661], [41.39863910776504, 2.160168615710207], [41.395300881603376, 2.159533514827968], [41.39886641173027, 2.1577160996558877], [41.39746760071943, 2.1607621791010243], [41.39566355039911, 2.160397811961645], [41.39724658046189, 2.159808871482451], [41.395894485168, 2.1594475808618583], [41.397698352276784, 2.158787067185035], [41.39497036338885, 2.160804231796431], [41.39686988684407, 2.159623483956613], [41.39811783991166, 2.1576997175518096], [41.39716339486879, 2.158883495574809], [41.39780299539646, 2.157940845142947], [41.396922212245904, 2.157512990931105], [41.3955451043031, 2.159826536007309], [41.397288606706134, 2.1612314289918637], [41.398791873580834, 2.158366703562201], [41.39886836554259, 2.159251648653193], [41.39569999093832, 2.1588593229891235], [41.397237940259004, 2.159403610571613], [41.39657272489176, 2.1613565671653747], [41.39747075936822, 2.1587545777946615], [41.39544291295498, 2.1585135198459215], [41.3967961039575, 2.159708456757902], [41.397740327721195, 2.157817494144303], [41.39511458154069, 2.1576322993149883], [41.39533838904781, 2.1593253165750377], [41.39673712154172, 2.159276864979319], [41.39664519747349, 2.157727807760756], [41.39503253914392, 2.158279995314986], [41.397591998578775, 2.1576350617417055], [41.395147390522894, 2.159314602693537], [41.395352294435476, 2.159731434301624], [41.398726109994726, 2.1577723930032233], [41.39795977077476, 2.160465718650735], [41.39756281843921, 2.1590805821891346], [41.39690200485551, 2.1592834493877087], [41.39794292411754, 2.15753317006657], [41.39787221916373, 2.152517537323243], [41.399173239138044, 2.153276526700623], [41.39836825495523, 2.1536160537996114], [41.398392334037716, 2.1539072652093068], [41.39783741385611, 2.1532309338477313], [41.397594242724544, 2.1532284484695507], [41.3976645998826, 2.153667458438531], [41.39807863532766, 2.153381359223621], [41.398318066531644, 2.1529340580673293], [41.39774546968743, 2.1534088773340647], [41.3992489978117, 2.1533318192310587], [41.39878931016422, 2.1525412183221473], [41.3992282152963, 2.1529096144343014], [41.39920375100786, 2.1523698841947843], [41.3980426374157, 2.1526929111347983], [41.39850435793943, 2.1533210457425294], [41.39755815849529, 2.1523442314973984], [41.39815123861754, 2.1522557337530595], [41.39854493636613, 2.1537572420387145], [41.397843609404624, 2.154038763817791], [41.398542518484916, 2.1539635689614145], [41.39835197836963, 2.1533211241620402], [41.39895499175484, 2.153249674766625], [41.398341233444384, 2.153387122535801], [41.398494607936975, 2.1537605720731134], [41.398017630045814, 2.1526514686263867], [41.3979757894913, 2.152268285761117], [41.39791256881199, 2.1539897399158843], [41.39913597468382, 2.1527414803226907], [41.39809735598218, 2.1525709672331343], [41.397913187416684, 2.1528115886547945], [41.39774984512918, 2.1536783465974327], [41.39808454429156, 2.152373665503355], [41.39837727127185, 2.153144759918536], [41.398686610368145, 2.153013210626711], [41.39883592667514, 2.1528547673769154], [41.39772520023713, 2.153346514057521], [41.39908481578384, 2.153782136433965], [41.39916445242152, 2.153107882747857], [41.39861948859257, 2.1527760184261417], [41.39885038491014, 2.1538312849162353], [41.398467085664585, 2.1538041800931023], [41.39763470783794, 2.152536111081584], [41.397875366450535, 2.1529411600383885], [41.39797347020702, 2.152563319252573], [41.39753627153311, 2.1531311717108443], [41.39757085705006, 2.153614823582096], [41.39878208311022, 2.153882796947744], [41.39882381221276, 2.152917467547198], [41.39758928657125, 2.15236486637482], [41.3977292872103, 2.153559740361084], [41.39803839640071, 2.1530299365585277], [41.39780548289538, 2.153034309997648], [41.39888435732411, 2.1533054125269016], [41.397552112460694, 2.152797346666545], [41.39767748099728, 2.1539624926780196], [41.39883550602244, 2.1525171943853914], [41.39891312373312, 2.1540199867520293], [41.397564317616144, 2.1532126918887804], [41.399162876089605, 2.152748818147188], [41.398456279002076, 2.1539122441245757], [41.39773152917314, 2.1522661386840545], [41.39866331447737, 2.152741597702253], [41.3989932022224, 2.15346522976452], [41.3979250500921, 2.1527129097072097], [41.39919688020793, 2.152969763752507], [41.39833221753272, 2.153167276331903], [41.398529647846466, 2.152661847110726], [41.397963909808354, 2.1524513346274485], [41.39846065568321, 2.1531159111636358], [41.39751286691648, 2.152448104099325], [41.398390925991905, 2.153474306188798], [41.39773450410385, 2.1533538724016177], [41.39792221189593, 2.153179256489425], [41.398611575001446, 2.1532571271029006], [41.3980147658953, 2.1529716850777025]],
                {
                    minOpacity: 0.1,
                    maxZoom: 18,
                    max: 0.8,
                    radius: 50,
                    blur: 70,
                    gradient: null
                    })
                .addTo(map_e9a0ac71acee47749a2d0b689bffffa3);
        
</script>